In [2]:
%pip install numpy
%pip install sympy
%pip install scipy
%load_ext autoreload
%autoreload 2
from simplex_two_phases import simplex as simplex_two_phases, _simplex_find_feasible_initial_basis
#from simplex_tableau import simplex as simplex_tableau
from simplex_tableau import markdown_repr_T
from simplex import simplex
from revised_simplex_tableau import revised_simplex_tableau, markdown_repr_T
from utils import array_to_markdown

    
import numpy as np
from pprint import pprint
from IPython.display import Markdown, display
from itertools import combinations
from sympy import symbols, Matrix, solve
from scipy.optimize import linprog


solution_types = {
    -1: 'Unfeasible',
    1: 'Optimal finite solution found',
    2: 'Multiple optimal solutions found',
    3: 'Unbounded'
}

# 1)

## letra a)

I) Solução do Primal

In [106]:
C = np.array([-2, -1] +  [0] * 4)
A = np.array([
    [2, 1, 1, 0, 0, 0],
    [2, 3, 0, 1, 0, 0],
    [4, 1, 0, 0, 1, 0],
    [1, 5, 0, 0, 0,-1]
])
B = np.array([
    4,
    3,
    5,
    1
])

result = linprog(C, A_eq = A, b_eq = B, method='highs')
display(result)

display(result.x)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -2.6
              x: [ 1.200e+00  2.000e-01  1.400e+00  0.000e+00  0.000e+00
                   1.200e+00]
            nit: 4
          lower:  residual: [ 1.200e+00  2.000e-01  1.400e+00  0.000e+00
                              0.000e+00  1.200e+00]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  2.000e-01
                              4.000e-01  0.000e+00]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          eqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00]
                 marginals: [-0.000e+00 -2.000e-01 -4.000e-01 -0.000e+00]
        ineqlin:  residual: []
                 marginals: []
 mip_node_count: 0
 mip_

array([1.2, 0.2, 1.4, 0. , 0. , 1.2])

II) solução do dual

In [107]:
C = np.array([4, 3, 5, -1, 0, 0])
A = np.array([
    [2, 2, 4, -1, -1, 0],
    [1, 3, 1, -5, 0, -1]
])
B = np.array([
    2,
    1
])

result = linprog(C, A_eq = A, b_eq = B, method='highs')
display(result)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 2.600000000000001
              x: [ 0.000e+00  2.000e-01  4.000e-01  0.000e+00  0.000e+00
                   0.000e+00]
            nit: 4
          lower:  residual: [ 0.000e+00  2.000e-01  4.000e-01  0.000e+00
                              0.000e+00  0.000e+00]
                 marginals: [ 1.400e+00  0.000e+00  0.000e+00  1.200e+00
                              1.200e+00  2.000e-01]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          eqlin:  residual: [ 0.000e+00  0.000e+00]
                 marginals: [ 1.200e+00  2.000e-01]
        ineqlin:  residual: []
                 marginals: []
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap

## Resolvendo duas fases com tableaus

In [108]:
C = np.array([4, 3, 5, -1, 0, 0])
A = np.array([
    [2, 2, 4, -1,-1, 0],
    [1, 3, 1, -5, 0, -1]
])
B = np.array([
    2,
    1
])

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = [6, 7]

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | 1.000 | 1.000 | 3.000 |
| \\(X_{6}\\) | 1.000 | 0.000 | 2.000 |
| \\(X_{7}\\) | 0.000 | 1.000 | 1.000 |


Extended Tableau

|   |  | \\(A_I^{-1}\\) | RHS | \\(X_{1}\\)
|---|---|---|---|---|
| Z | 1.000 | 1.000 | 3.000 | 5.000 |
| \\(X_{6}\\) | 1.000 | 0.000 | 2.000 | 2.000 |
| \\(X_{7}\\) | 0.000 | 1.000 | 1.000 | 3.000 |


\\(Ĉ_J = \ \\)array([ 3.,  5.,  5., -6., -1., -1.])


\\(J = {[0, 1, 2, 3, 4, 5]}\\)

### Pivot Operations

Variable to enter: \\(X_{1}\\)
Variable to leave: \\(X_{7}\\)
#### Row Operations:

1. \\(R_{X_{7}} \leftarrow \frac{R_{X_{7}}}{3.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 5.000R_{X_{7}}\\)
2. \\(R_{X_{6}} \leftarrow R_{X_{6}} - 2.000R_{X_{7}}\\)


Computed Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | 1.000 | -0.667 | 1.333 |
| \\(X_{6}\\) | 1.000 | -0.667 | 1.333 |
| \\(X_{1}\\) | 0.000 | 0.333 | 0.333 |


## Iteration 1

Starting Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | 1.000 | -0.667 | 1.333 |
| \\(X_{6}\\) | 1.000 | -0.667 | 1.333 |
| \\(X_{1}\\) | 0.000 | 0.333 | 0.333 |


Extended Tableau

|   |  | \\(A_I^{-1}\\) | RHS | \\(X_{2}\\)
|---|---|---|---|---|
| Z | 1.000 | -0.667 | 1.333 | 3.333 |
| \\(X_{6}\\) | 1.000 | -0.667 | 1.333 | 3.333 |
| \\(X_{1}\\) | 0.000 | 0.333 | 0.333 | 0.333 |


\\(Ĉ_J = \ \\)array([ 1.333333, -1.666667,  3.333333,  2.333333, -1.      ,  0.666667])


\\(J = {[0, 7, 2, 3, 4, 5]}\\)

### Pivot Operations

Variable to enter: \\(X_{2}\\)
Variable to leave: \\(X_{6}\\)
#### Row Operations:

1. \\(R_{X_{6}} \leftarrow \frac{R_{X_{6}}}{3.333}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 3.333R_{X_{6}}\\)
2. \\(R_{X_{1}} \leftarrow R_{X_{1}} - 0.333R_{X_{6}}\\)


Computed Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | 0.000 | -0.000 | 0.000 |
| \\(X_{2}\\) | 0.300 | -0.200 | 0.400 |
| \\(X_{1}\\) | -0.100 | 0.400 | 0.200 |


## Iteration 2

Starting Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | 0.000 | -0.000 | 0.000 |
| \\(X_{2}\\) | 0.300 | -0.200 | 0.400 |
| \\(X_{1}\\) | -0.100 | 0.400 | 0.200 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
2.000 & 2.000 & 4.000 & -1.000 & -1.000 & 0.000 & 1.000 & 0.000 \\
1.000 & 3.000 & 1.000 & -5.000 & 0.000 & -1.000 & 0.000 & 1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
2.000\\
1.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 1.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}\\
X_{7}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal multiple solutions

Optimal Solution: \\(X^{*} = ['0.000', '0.200', '0.400', '0.000', '0.000', '0.000', '0.000', '0.000']\\)

Optimal Value: \\(Z^{*} = 0.000\\)

Optimal Basis: \\([2, 1]\\)

Final non-basic variables set J: \\([0, 7, 6, 3, 4, 5]\\)

\\(\hat{C}_{J}\\): \\([0.0, -1.0, -1.0, 0.0, -0.0, 0.0]\\)

Number of Iterations: 3



## Resolução do problema original

In [109]:
I = [2, 1]

# resolvendo problema original com o simplex tableau revisado
solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | 1.200 | 0.200 | 2.600 |
| \\(X_{2}\\) | 0.300 | -0.200 | 0.400 |
| \\(X_{1}\\) | -0.100 | 0.400 | 0.200 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
2.000 & 2.000 & 4.000 & -1.000 & -1.000 & 0.000 \\
1.000 & 3.000 & 1.000 & -5.000 & 0.000 & -1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
2.000\\
1.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
4.000 & 3.000 & 5.000 & -1.000 & 0.000 & 0.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal unique solution

Optimal Solution: \\(X^{*} = ['0.000', '0.200', '0.400', '0.000', '0.000', '0.000']\\)

Optimal Value: \\(Z^{*} = 2.600\\)

Optimal Basis: \\([2, 1]\\)

Final non-basic variables set J: \\([0, 3, 4, 5]\\)

\\(\hat{C}_{J}\\): \\([-1.4, -1.2, -1.2, -0.2]\\)

Number of Iterations: 1



## letra b)

In [110]:
A = np.array([
    [1, -1, -1, 1, 1, -1, 1, -1, -1, 0, 0, 0, 0, 0],
    [2, -2, 1, -1, 3, -3, 0,  0,  0, 1, 0, 0, 0, 0],
    [1, -1, 0,  0, 0,  0, 0,  0,  0, 0, 1, 0, 0, 0],
    [0,  0, 1, -1, 0,  0, 0,  0,  0, 0, 0, 1, 0, 0],
    [0,  0, 0,  0, 1, -1, 0,  0,  0, 0, 0, 0, 1, 0],
    [0,  0, 0,  0, 0,  0, 1, -1,  0, 0, 0, 0, 0, 1]
])

B = np.array([
    2,
    3,
    0,
    0,
    0,
    0
])
C = -np.array([
    4, -4, 5, -5, 18, -18, 2, -2, 0, 0, 0, 0, 0, 0
])

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = [14] + list(range(9,14))

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{14}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{9}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.000 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS | \\(X_{0}\\)
|---|---|---|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 | 1.000 |
| \\(X_{14}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 | 1.000 |
| \\(X_{9}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.000 | 2.000 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |


\\(Ĉ_J = \ \\)array([ 1., -1., -1.,  1.,  1., -1.,  1., -1., -1.])


\\(J = {[0, 1, 2, 3, 4, 5, 6, 7, 8]}\\)

### Pivot Operations

Variable to enter: \\(X_{0}\\)
Variable to leave: \\(X_{10}\\)
#### Row Operations:

1. \\(R_{X_{10}} \leftarrow \frac{R_{X_{10}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 1.000R_{X_{10}}\\)
2. \\(R_{X_{14}} \leftarrow R_{X_{14}} - 1.000R_{X_{10}}\\)
3. \\(R_{X_{9}} \leftarrow R_{X_{9}} - 2.000R_{X_{10}}\\)
4. \\(R_{X_{11}} \leftarrow R_{X_{11}} - 0.000R_{X_{10}}\\)
5. \\(R_{X_{12}} \leftarrow R_{X_{12}} - 0.000R_{X_{10}}\\)
6. \\(R_{X_{13}} \leftarrow R_{X_{13}} - 0.000R_{X_{10}}\\)


Computed Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | -1.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{14}\\) | 1.000 | 0.000 | -1.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{9}\\) | 0.000 | 1.000 | -2.000 | 0.000 | 0.000 | 0.000 | 3.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Iteration 1

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | -1.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{14}\\) | 1.000 | 0.000 | -1.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{9}\\) | 0.000 | 1.000 | -2.000 | 0.000 | 0.000 | 0.000 | 3.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS | \\(X_{3}\\)
|---|---|---|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | -1.000 | 0.000 | 0.000 | 0.000 | 2.000 | 1.000 |
| \\(X_{14}\\) | 1.000 | 0.000 | -1.000 | 0.000 | 0.000 | 0.000 | 2.000 | 1.000 |
| \\(X_{9}\\) | 0.000 | 1.000 | -2.000 | 0.000 | 0.000 | 0.000 | 3.000 | -1.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -1.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |


\\(Ĉ_J = \ \\)array([-1.,  0., -1.,  1.,  1., -1.,  1., -1., -1.])


\\(J = {[10, 1, 2, 3, 4, 5, 6, 7, 8]}\\)

### Pivot Operations

Variable to enter: \\(X_{3}\\)
Variable to leave: \\(X_{14}\\)
#### Row Operations:

1. \\(R_{X_{14}} \leftarrow \frac{R_{X_{14}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 1.000R_{X_{14}}\\)
2. \\(R_{X_{9}} \leftarrow R_{X_{9}} + 1.000R_{X_{14}}\\)
3. \\(R_{X_{0}} \leftarrow R_{X_{0}} - 0.000R_{X_{14}}\\)
4. \\(R_{X_{11}} \leftarrow R_{X_{11}} + 1.000R_{X_{14}}\\)
5. \\(R_{X_{12}} \leftarrow R_{X_{12}} - 0.000R_{X_{14}}\\)
6. \\(R_{X_{13}} \leftarrow R_{X_{13}} - 0.000R_{X_{14}}\\)


Computed Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{3}\\) | 1.000 | 0.000 | -1.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{9}\\) | 1.000 | 1.000 | -3.000 | 0.000 | 0.000 | 0.000 | 5.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{11}\\) | 1.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Iteration 2

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{3}\\) | 1.000 | 0.000 | -1.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{9}\\) | 1.000 | 1.000 | -3.000 | 0.000 | 0.000 | 0.000 | 5.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{11}\\) | 1.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
1.000 & -1.000 & -1.000 & 1.000 & 1.000 & -1.000 & 1.000 & -1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 \\
2.000 & -2.000 & 1.000 & -1.000 & 3.000 & -3.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 \\
0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 \\
0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
2.000\\
3.000\\
0.000\\
0.000\\
0.000\\
0.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}\\
X_{7}\\
X_{8}\\
X_{9}\\
X_{10}\\
X_{11}\\
X_{12}\\
X_{13}\\
X_{14}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal multiple solutions

Optimal Solution: \\(X^{*} = ['0.000', '0.000', '0.000', '2.000', '0.000', '0.000', '0.000', '0.000', '0.000', '5.000', '0.000', '2.000', '0.000', '0.000', '0.000']\\)

Optimal Value: \\(Z^{*} = 0.000\\)

Optimal Basis: \\([3, 9, 0, 11, 12, 13]\\)

Final non-basic variables set J: \\([10, 1, 2, 14, 4, 5, 6, 7, 8]\\)

\\(\hat{C}_{J}\\): \\([0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0]\\)

Number of Iterations: 3



In [111]:
# resolvendo problema original com o simplex tableau revisado

I = [3, 9, 0, 11, 4, 6]
solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|---|
| Z | 5.000 | 0.000 | -9.000 | 0.000 | -23.000 | -7.000 | 10.000 |
| \\(X_{3}\\) | 1.000 | 0.000 | -1.000 | 0.000 | -1.000 | -1.000 | 2.000 |
| \\(X_{9}\\) | 1.000 | 1.000 | -3.000 | 0.000 | -4.000 | -1.000 | 5.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{11}\\) | 1.000 | 0.000 | -1.000 | 1.000 | -1.000 | -1.000 | 2.000 |
| \\(X_{4}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{6}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
1.000 & -1.000 & -1.000 & 1.000 & 1.000 & -1.000 & 1.000 & -1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
2.000 & -2.000 & 1.000 & -1.000 & 3.000 & -3.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 \\
0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 \\
0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 \\
0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
2.000\\
3.000\\
0.000\\
0.000\\
0.000\\
0.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
-4.000 & 4.000 & -5.000 & 5.000 & -18.000 & 18.000 & -2.000 & 2.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}\\
X_{7}\\
X_{8}\\
X_{9}\\
X_{10}\\
X_{11}\\
X_{12}\\
X_{13}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal multiple solutions

Optimal Solution: \\(X^{*} = ['0.000', '0.000', '0.000', '2.000', '0.000', '0.000', '0.000', '0.000', '0.000', '5.000', '0.000', '2.000', '0.000', '0.000']\\)

Optimal Value: \\(Z^{*} = 10.000\\)

Optimal Basis: \\([3, 9, 0, 11, 4, 6]\\)

Final non-basic variables set J: \\([1, 2, 5, 7, 8, 10, 12, 13]\\)

\\(\hat{C}_{J}\\): \\([0.0, 0.0, 0.0, 0.0, -5.0, -9.0, -23.0, -7.0]\\)

Number of Iterations: 1



In [112]:
result = linprog(C, A_eq = A, b_eq = B, method='highs')
display(result)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 10.0
              x: [ 0.000e+00 -0.000e+00 ...  0.000e+00  0.000e+00]
            nit: 0
          lower:  residual: [ 0.000e+00 -0.000e+00 ...  0.000e+00
                              0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00 ...  2.300e+01
                              7.000e+00]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
                 marginals: [ 5.000e+00 -0.000e+00 -9.000e+00 -0.000e+00
                             -2.300e+01 -7.000e+00]
        ineqlin:  residual: []
                 marginals: []
 mip_node_count: 0
 mip_dual_b

## letra c)

In [113]:
basis = np.eye(7)
basis[0, 0] *= -1
basis[2, 2] *= -1

A = np.array([
    [-2, -2, -1, 1],
    [-1, -1,  1, 3],
    [ 1,  1,  0, -5],
    [-1,  0,  0, 0],
    [ 0,  1,  0, 0],
    [ 0,  0,  1, 0],
    [ 0,  0, 0, -1]
])

f = lambda x: np.array([x, -x])
aux = []
for row in A:
    aux.append(
        np.array([f(r) for r in row]).ravel()
    )

A = np.hstack((aux, basis))

B = np.array([
    2,
    3,
    4,
    0,
    0,
    0,
    0
])
C = -np.array(
    [-2, -2, -12, -9] + [0] * 7
)

aux = []
for c in C[0:4]:
    aux.append(f(c))
C = np.hstack((np.array(aux).ravel(), [0] * 7))

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = np.arange(8, 15)
I[0] = 15
I[2] = 16
I = I.tolist()

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 6.000 |
| \\(X_{15}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{9}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.000 |
| \\(X_{16}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 4.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{14}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS | \\(X_{7}\\)
|---|---|---|---|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 6.000 | 4.000 |
| \\(X_{15}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 | -1.000 |
| \\(X_{9}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.000 | -3.000 |
| \\(X_{16}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 4.000 | 5.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{14}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 1.000 |


\\(Ĉ_J = \ \\)array([-1.,  1., -1.,  1., -1.,  1., -4.,  4., -1., -1.])


\\(J = {[0, 1, 2, 3, 4, 5, 6, 7, 8, 10]}\\)

### Pivot Operations

Variable to enter: \\(X_{7}\\)
Variable to leave: \\(X_{14}\\)
#### Row Operations:

1. \\(R_{X_{14}} \leftarrow \frac{R_{X_{14}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 4.000R_{X_{14}}\\)
2. \\(R_{X_{15}} \leftarrow R_{X_{15}} + 1.000R_{X_{14}}\\)
3. \\(R_{X_{9}} \leftarrow R_{X_{9}} + 3.000R_{X_{14}}\\)
4. \\(R_{X_{16}} \leftarrow R_{X_{16}} - 5.000R_{X_{14}}\\)
5. \\(R_{X_{11}} \leftarrow R_{X_{11}} - 0.000R_{X_{14}}\\)
6. \\(R_{X_{12}} \leftarrow R_{X_{12}} - 0.000R_{X_{14}}\\)
7. \\(R_{X_{13}} \leftarrow R_{X_{13}} - 0.000R_{X_{14}}\\)


Computed Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -4.000 | 6.000 |
| \\(X_{15}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 |
| \\(X_{9}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.000 | 3.000 |
| \\(X_{16}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -5.000 | 4.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Iteration 1

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -4.000 | 6.000 |
| \\(X_{15}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 |
| \\(X_{9}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.000 | 3.000 |
| \\(X_{16}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -5.000 | 4.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS | \\(X_{1}\\)
|---|---|---|---|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -4.000 | 6.000 | 1.000 |
| \\(X_{15}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 | 2.000 |
| \\(X_{9}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.000 | 3.000 | 1.000 |
| \\(X_{16}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -5.000 | 4.000 | -1.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |


\\(Ĉ_J = \ \\)array([-1.,  1., -1.,  1., -1.,  1.,  0., -4., -1., -1.])


\\(J = {[0, 1, 2, 3, 4, 5, 6, 14, 8, 10]}\\)

### Pivot Operations

Variable to enter: \\(X_{1}\\)
Variable to leave: \\(X_{11}\\)
#### Row Operations:

1. \\(R_{X_{11}} \leftarrow \frac{R_{X_{11}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 1.000R_{X_{11}}\\)
2. \\(R_{X_{15}} \leftarrow R_{X_{15}} - 2.000R_{X_{11}}\\)
3. \\(R_{X_{9}} \leftarrow R_{X_{9}} - 1.000R_{X_{11}}\\)
4. \\(R_{X_{16}} \leftarrow R_{X_{16}} + 1.000R_{X_{11}}\\)
5. \\(R_{X_{12}} \leftarrow R_{X_{12}} - 0.000R_{X_{11}}\\)
6. \\(R_{X_{13}} \leftarrow R_{X_{13}} - 0.000R_{X_{11}}\\)
7. \\(R_{X_{7}} \leftarrow R_{X_{7}} - 0.000R_{X_{11}}\\)


Computed Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | -4.000 | 6.000 |
| \\(X_{15}\\) | 1.000 | 0.000 | 0.000 | -2.000 | 0.000 | 0.000 | 1.000 | 2.000 |
| \\(X_{9}\\) | 0.000 | 1.000 | 0.000 | -1.000 | 0.000 | 0.000 | 3.000 | 3.000 |
| \\(X_{16}\\) | 0.000 | 0.000 | 1.000 | 1.000 | 0.000 | 0.000 | -5.000 | 4.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Iteration 2

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | -4.000 | 6.000 |
| \\(X_{15}\\) | 1.000 | 0.000 | 0.000 | -2.000 | 0.000 | 0.000 | 1.000 | 2.000 |
| \\(X_{9}\\) | 0.000 | 1.000 | 0.000 | -1.000 | 0.000 | 0.000 | 3.000 | 3.000 |
| \\(X_{16}\\) | 0.000 | 0.000 | 1.000 | 1.000 | 0.000 | 0.000 | -5.000 | 4.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS | \\(X_{3}\\)
|---|---|---|---|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | -4.000 | 6.000 | 1.000 |
| \\(X_{15}\\) | 1.000 | 0.000 | 0.000 | -2.000 | 0.000 | 0.000 | 1.000 | 2.000 | 2.000 |
| \\(X_{9}\\) | 0.000 | 1.000 | 0.000 | -1.000 | 0.000 | 0.000 | 3.000 | 3.000 | 1.000 |
| \\(X_{16}\\) | 0.000 | 0.000 | 1.000 | 1.000 | 0.000 | 0.000 | -5.000 | 4.000 | -1.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -1.000 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |


\\(Ĉ_J = \ \\)array([ 0., -1., -1.,  1., -1.,  1.,  0., -4., -1., -1.])


\\(J = {[0, 11, 2, 3, 4, 5, 6, 14, 8, 10]}\\)

### Pivot Operations

Variable to enter: \\(X_{3}\\)
Variable to leave: \\(X_{15}\\)
#### Row Operations:

1. \\(R_{X_{15}} \leftarrow \frac{R_{X_{15}}}{2.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 1.000R_{X_{15}}\\)
2. \\(R_{X_{9}} \leftarrow R_{X_{9}} - 1.000R_{X_{15}}\\)
3. \\(R_{X_{16}} \leftarrow R_{X_{16}} + 1.000R_{X_{15}}\\)
4. \\(R_{X_{1}} \leftarrow R_{X_{1}} - 0.000R_{X_{15}}\\)
5. \\(R_{X_{12}} \leftarrow R_{X_{12}} + 1.000R_{X_{15}}\\)
6. \\(R_{X_{13}} \leftarrow R_{X_{13}} - 0.000R_{X_{15}}\\)
7. \\(R_{X_{7}} \leftarrow R_{X_{7}} - 0.000R_{X_{15}}\\)


Computed Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | 0.500 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -4.500 | 5.000 |
| \\(X_{3}\\) | 0.500 | 0.000 | 0.000 | -1.000 | 0.000 | 0.000 | 0.500 | 1.000 |
| \\(X_{9}\\) | -0.500 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.500 | 2.000 |
| \\(X_{16}\\) | 0.500 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -4.500 | 5.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.500 | 0.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.500 | 1.000 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Iteration 3

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | 0.500 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -4.500 | 5.000 |
| \\(X_{3}\\) | 0.500 | 0.000 | 0.000 | -1.000 | 0.000 | 0.000 | 0.500 | 1.000 |
| \\(X_{9}\\) | -0.500 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.500 | 2.000 |
| \\(X_{16}\\) | 0.500 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -4.500 | 5.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.500 | 0.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.500 | 1.000 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS | \\(X_{5}\\)
|---|---|---|---|---|---|---|---|---|---|
| Z | 0.500 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -4.500 | 5.000 | 0.500 |
| \\(X_{3}\\) | 0.500 | 0.000 | 0.000 | -1.000 | 0.000 | 0.000 | 0.500 | 1.000 | 0.500 |
| \\(X_{9}\\) | -0.500 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.500 | 2.000 | -1.500 |
| \\(X_{16}\\) | 0.500 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -4.500 | 5.000 | 0.500 |
| \\(X_{1}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.500 | 0.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.500 | 1.000 | 0.500 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | -1.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |


\\(Ĉ_J = \ \\)array([ 0. ,  0. ,  0. , -0.5, -0.5,  0.5,  0. , -4.5, -0.5, -1. ])


\\(J = {[0, 11, 2, 15, 4, 5, 6, 14, 8, 10]}\\)

### Pivot Operations

Variable to enter: \\(X_{5}\\)
Variable to leave: \\(X_{3}\\)
#### Row Operations:

1. \\(R_{X_{3}} \leftarrow \frac{R_{X_{3}}}{0.500}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 0.500R_{X_{3}}\\)
2. \\(R_{X_{9}} \leftarrow R_{X_{9}} + 1.500R_{X_{3}}\\)
3. \\(R_{X_{16}} \leftarrow R_{X_{16}} - 0.500R_{X_{3}}\\)
4. \\(R_{X_{1}} \leftarrow R_{X_{1}} - 0.000R_{X_{3}}\\)
5. \\(R_{X_{12}} \leftarrow R_{X_{12}} - 0.500R_{X_{3}}\\)
6. \\(R_{X_{13}} \leftarrow R_{X_{13}} + 1.000R_{X_{3}}\\)
7. \\(R_{X_{7}} \leftarrow R_{X_{7}} - 0.000R_{X_{3}}\\)


Computed Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 1.000 | 1.000 | 0.000 | 0.000 | -5.000 | 4.000 |
| \\(X_{5}\\) | 1.000 | 0.000 | 0.000 | -2.000 | 0.000 | 0.000 | 1.000 | 2.000 |
| \\(X_{9}\\) | 1.000 | 1.000 | 0.000 | -3.000 | 0.000 | 0.000 | 4.000 | 5.000 |
| \\(X_{16}\\) | 0.000 | 0.000 | 1.000 | 1.000 | 0.000 | 0.000 | -5.000 | 4.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 1.000 | 0.000 | 0.000 | -2.000 | 0.000 | 1.000 | 1.000 | 2.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Iteration 4

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 1.000 | 1.000 | 0.000 | 0.000 | -5.000 | 4.000 |
| \\(X_{5}\\) | 1.000 | 0.000 | 0.000 | -2.000 | 0.000 | 0.000 | 1.000 | 2.000 |
| \\(X_{9}\\) | 1.000 | 1.000 | 0.000 | -3.000 | 0.000 | 0.000 | 4.000 | 5.000 |
| \\(X_{16}\\) | 0.000 | 0.000 | 1.000 | 1.000 | 0.000 | 0.000 | -5.000 | 4.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 1.000 | 0.000 | 0.000 | -2.000 | 0.000 | 1.000 | 1.000 | 2.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS | \\(X_{11}\\)
|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 1.000 | 1.000 | 0.000 | 0.000 | -5.000 | 4.000 | 1.000 |
| \\(X_{5}\\) | 1.000 | 0.000 | 0.000 | -2.000 | 0.000 | 0.000 | 1.000 | 2.000 | -2.000 |
| \\(X_{9}\\) | 1.000 | 1.000 | 0.000 | -3.000 | 0.000 | 0.000 | 4.000 | 5.000 | -3.000 |
| \\(X_{16}\\) | 0.000 | 0.000 | 1.000 | 1.000 | 0.000 | 0.000 | -5.000 | 4.000 | 1.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 1.000 | 0.000 | 0.000 | -2.000 | 0.000 | 1.000 | 1.000 | 2.000 | -2.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |


\\(Ĉ_J = \ \\)array([ 0.,  1.,  1., -1.,  0., -1.,  0., -5.,  0., -1.])


\\(J = {[0, 11, 2, 15, 4, 3, 6, 14, 8, 10]}\\)

### Pivot Operations

Variable to enter: \\(X_{11}\\)
Variable to leave: \\(X_{1}\\)
#### Row Operations:

1. \\(R_{X_{1}} \leftarrow \frac{R_{X_{1}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 1.000R_{X_{1}}\\)
2. \\(R_{X_{5}} \leftarrow R_{X_{5}} + 2.000R_{X_{1}}\\)
3. \\(R_{X_{9}} \leftarrow R_{X_{9}} + 3.000R_{X_{1}}\\)
4. \\(R_{X_{16}} \leftarrow R_{X_{16}} - 1.000R_{X_{1}}\\)
5. \\(R_{X_{12}} \leftarrow R_{X_{12}} - 0.000R_{X_{1}}\\)
6. \\(R_{X_{13}} \leftarrow R_{X_{13}} + 2.000R_{X_{1}}\\)
7. \\(R_{X_{7}} \leftarrow R_{X_{7}} - 0.000R_{X_{1}}\\)


Computed Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -5.000 | 4.000 |
| \\(X_{5}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 |
| \\(X_{9}\\) | 1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 4.000 | 5.000 |
| \\(X_{16}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -5.000 | 4.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 1.000 | 2.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Iteration 5

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -5.000 | 4.000 |
| \\(X_{5}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 |
| \\(X_{9}\\) | 1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 4.000 | 5.000 |
| \\(X_{16}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -5.000 | 4.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 1.000 | 2.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS | \\(X_{0}\\)
|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -5.000 | 4.000 | 1.000 |
| \\(X_{5}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 | -2.000 |
| \\(X_{9}\\) | 1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 4.000 | 5.000 | -3.000 |
| \\(X_{16}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -5.000 | 4.000 | 1.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | -1.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 1.000 | 2.000 | -2.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |


\\(Ĉ_J = \ \\)array([ 1., -1.,  1., -1.,  0., -1.,  0., -5.,  0., -1.])


\\(J = {[0, 1, 2, 15, 4, 3, 6, 14, 8, 10]}\\)

### Pivot Operations

Variable to enter: \\(X_{0}\\)
Variable to leave: \\(X_{16}\\)
#### Row Operations:

1. \\(R_{X_{16}} \leftarrow \frac{R_{X_{16}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 1.000R_{X_{16}}\\)
2. \\(R_{X_{5}} \leftarrow R_{X_{5}} + 2.000R_{X_{16}}\\)
3. \\(R_{X_{9}} \leftarrow R_{X_{9}} + 3.000R_{X_{16}}\\)
4. \\(R_{X_{11}} \leftarrow R_{X_{11}} + 1.000R_{X_{16}}\\)
5. \\(R_{X_{12}} \leftarrow R_{X_{12}} - 0.000R_{X_{16}}\\)
6. \\(R_{X_{13}} \leftarrow R_{X_{13}} + 2.000R_{X_{16}}\\)
7. \\(R_{X_{7}} \leftarrow R_{X_{7}} - 0.000R_{X_{16}}\\)


Computed Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | 1.000 | 0.000 | 2.000 | 0.000 | 0.000 | 0.000 | -9.000 | 10.000 |
| \\(X_{9}\\) | 1.000 | 1.000 | 3.000 | 0.000 | 0.000 | 0.000 | -11.000 | 17.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -5.000 | 4.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 1.000 | 1.000 | 0.000 | 0.000 | -5.000 | 4.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 1.000 | 0.000 | 2.000 | 0.000 | 0.000 | 1.000 | -9.000 | 10.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Iteration 6

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | 1.000 | 0.000 | 2.000 | 0.000 | 0.000 | 0.000 | -9.000 | 10.000 |
| \\(X_{9}\\) | 1.000 | 1.000 | 3.000 | 0.000 | 0.000 | 0.000 | -11.000 | 17.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -5.000 | 4.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 1.000 | 1.000 | 0.000 | 0.000 | -5.000 | 4.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 1.000 | 0.000 | 2.000 | 0.000 | 0.000 | 1.000 | -9.000 | 10.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
-2.000 & 2.000 & -2.000 & 2.000 & -1.000 & 1.000 & 1.000 & -1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -0.000 \\
-1.000 & 1.000 & -1.000 & 1.000 & 1.000 & -1.000 & 3.000 & -3.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & -0.000 & -0.000 \\
1.000 & -1.000 & 1.000 & -1.000 & 0.000 & 0.000 & -5.000 & 5.000 & 0.000 & 0.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & -0.000 & 1.000 \\
-1.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & -0.000 & -0.000 \\
0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & -0.000 & -0.000 \\
0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & -0.000 & -0.000 \\
0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & -1.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -0.000 & -0.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
2.000\\
3.000\\
4.000\\
0.000\\
0.000\\
0.000\\
0.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 1.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}\\
X_{7}\\
X_{8}\\
X_{9}\\
X_{10}\\
X_{11}\\
X_{12}\\
X_{13}\\
X_{14}\\
X_{15}\\
X_{16}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal multiple solutions

Optimal Solution: \\(X^{*} = ['4.000', '0.000', '0.000', '0.000', '0.000', '10.000', '0.000', '0.000', '0.000', '17.000', '0.000', '4.000', '0.000', '10.000', '0.000', '0.000', '0.000']\\)

Optimal Value: \\(Z^{*} = 0.000\\)

Optimal Basis: \\([5, 9, 0, 11, 12, 13, 7]\\)

Final non-basic variables set J: \\([16, 1, 2, 15, 4, 3, 6, 14, 8, 10]\\)

\\(\hat{C}_{J}\\): \\([-1.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]\\)

Number of Iterations: 7



**Segunda Fase:**

In [114]:
# resolvendo problema original com o simplex tableau revisado

I = [5, 9, 0, 11, 12, 13, 7]
solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | -12.000 | 0.000 | -22.000 | 0.000 | 0.000 | 0.000 | 89.000 | -112.000 |
| \\(X_{5}\\) | 1.000 | 0.000 | 2.000 | 0.000 | 0.000 | 0.000 | -9.000 | 10.000 |
| \\(X_{9}\\) | 1.000 | 1.000 | 3.000 | 0.000 | 0.000 | 0.000 | -11.000 | 17.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -5.000 | 4.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 1.000 | 1.000 | 0.000 | 0.000 | -5.000 | 4.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 1.000 | 0.000 | 2.000 | 0.000 | 0.000 | 1.000 | -9.000 | 10.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS | \\(X_{14}\\)
|---|---|---|---|---|---|---|---|---|---|
| Z | -12.000 | 0.000 | -22.000 | 0.000 | 0.000 | 0.000 | 89.000 | -112.000 | 89.000 |
| \\(X_{5}\\) | 1.000 | 0.000 | 2.000 | 0.000 | 0.000 | 0.000 | -9.000 | 10.000 | -9.000 |
| \\(X_{9}\\) | 1.000 | 1.000 | 3.000 | 0.000 | 0.000 | 0.000 | -11.000 | 17.000 | -11.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -5.000 | 4.000 | -5.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 1.000 | 1.000 | 0.000 | 0.000 | -5.000 | 4.000 | -5.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 1.000 | 0.000 | 2.000 | 0.000 | 0.000 | 1.000 | -9.000 | 10.000 | -9.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 1.000 |


\\(Ĉ_J = \ \\)array([ 0.,  0.,  0.,  0.,  0., 12., 22., 89.])


\\(J = {[1, 2, 3, 4, 6, 8, 10, 14]}\\)

### Pivot Operations

Variable to enter: \\(X_{14}\\)
Variable to leave: \\(X_{7}\\)
#### Row Operations:

1. \\(R_{X_{7}} \leftarrow \frac{R_{X_{7}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 89.000R_{X_{7}}\\)
2. \\(R_{X_{5}} \leftarrow R_{X_{5}} + 9.000R_{X_{7}}\\)
3. \\(R_{X_{9}} \leftarrow R_{X_{9}} + 11.000R_{X_{7}}\\)
4. \\(R_{X_{0}} \leftarrow R_{X_{0}} + 5.000R_{X_{7}}\\)
5. \\(R_{X_{11}} \leftarrow R_{X_{11}} + 5.000R_{X_{7}}\\)
6. \\(R_{X_{12}} \leftarrow R_{X_{12}} - 0.000R_{X_{7}}\\)
7. \\(R_{X_{13}} \leftarrow R_{X_{13}} + 9.000R_{X_{7}}\\)


Computed Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | -12.000 | 0.000 | -22.000 | 0.000 | 0.000 | 0.000 | 0.000 | -112.000 |
| \\(X_{5}\\) | 1.000 | 0.000 | 2.000 | 0.000 | 0.000 | 0.000 | 0.000 | 10.000 |
| \\(X_{9}\\) | 1.000 | 1.000 | 3.000 | 0.000 | 0.000 | 0.000 | 0.000 | 17.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 4.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 4.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 1.000 | 0.000 | 2.000 | 0.000 | 0.000 | 1.000 | 0.000 | 10.000 |
| \\(X_{14}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Iteration 1

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | -12.000 | 0.000 | -22.000 | 0.000 | 0.000 | 0.000 | 0.000 | -112.000 |
| \\(X_{5}\\) | 1.000 | 0.000 | 2.000 | 0.000 | 0.000 | 0.000 | 0.000 | 10.000 |
| \\(X_{9}\\) | 1.000 | 1.000 | 3.000 | 0.000 | 0.000 | 0.000 | 0.000 | 17.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 4.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 4.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{13}\\) | 1.000 | 0.000 | 2.000 | 0.000 | 0.000 | 1.000 | 0.000 | 10.000 |
| \\(X_{14}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
-2.000 & 2.000 & -2.000 & 2.000 & -1.000 & 1.000 & 1.000 & -1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
-1.000 & 1.000 & -1.000 & 1.000 & 1.000 & -1.000 & 3.000 & -3.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
1.000 & -1.000 & 1.000 & -1.000 & 0.000 & 0.000 & -5.000 & 5.000 & 0.000 & 0.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
-1.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 \\
0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 \\
0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 \\
0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & -1.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
2.000\\
3.000\\
4.000\\
0.000\\
0.000\\
0.000\\
0.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
2.000 & -2.000 & 2.000 & -2.000 & 12.000 & -12.000 & 9.000 & -9.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}\\
X_{7}\\
X_{8}\\
X_{9}\\
X_{10}\\
X_{11}\\
X_{12}\\
X_{13}\\
X_{14}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Unbounded solution

Optimal Solution: \\(X^{*} = ['4.000', '0.000', '0.000', '0.000', '0.000', '10.000', '0.000', '0.000', '0.000', '17.000', '0.000', '4.000', '0.000', '10.000', '0.000']\\)

Optimal Value: \\(Z^{*} = -112.000\\)

Optimal Basis: \\([5, 9, 0, 11, 12, 13, 14]\\)

Final non-basic variables set J: \\([1, 2, 3, 4, 6, 8, 10, 7]\\)

\\(\hat{C}_{J}\\): \\([0.0, 0.0, 0.0, 0.0, 89.0, 12.0, 22.0, -89.0]\\)

Number of Iterations: 2



**Resolvendo o primal:**

Primeira etapa: artificial

In [115]:
basis = np.eye(4)
basis[0, 0] *= -1
basis[3, 3] *= -1

A = np.array([
    [2, -1, -1],
    [2, -1, -1],
    [1,  1,  0],
    [-1, 3,  5]
])

A = np.hstack((A, basis))

B = np.array([
    2,
    2,
    12,
    9
])
C = -np.array(
    [-2, 3, -4] + [0] * 4
)

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = np.arange(3, 7)
I[0] = 7
I[3] = 8
I = I.tolist()

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  |   | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | 0.000 | 1.000 | 11.000 |
| \\(X_{7}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{4}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 12.000 |
| \\(X_{8}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 9.000 |


Extended Tableau

|   |  |   | \\(A_I^{-1}\\) |   | RHS | \\(X_{2}\\)
|---|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | 0.000 | 1.000 | 11.000 | 4.000 |
| \\(X_{7}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 2.000 | -1.000 |
| \\(X_{4}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 2.000 | -1.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 12.000 | 0.000 |
| \\(X_{8}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 9.000 | 5.000 |


\\(Ĉ_J = \ \\)array([ 1.,  2.,  4., -1., -1.])


\\(J = {[0, 1, 2, 3, 6]}\\)

### Pivot Operations

Variable to enter: \\(X_{2}\\)
Variable to leave: \\(X_{8}\\)
#### Row Operations:

1. \\(R_{X_{8}} \leftarrow \frac{R_{X_{8}}}{5.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 4.000R_{X_{8}}\\)
2. \\(R_{X_{7}} \leftarrow R_{X_{7}} + 1.000R_{X_{8}}\\)
3. \\(R_{X_{4}} \leftarrow R_{X_{4}} + 1.000R_{X_{8}}\\)
4. \\(R_{X_{5}} \leftarrow R_{X_{5}} - 0.000R_{X_{8}}\\)


Computed Tableau:

|   |  |   | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | 0.000 | 0.200 | 3.800 |
| \\(X_{7}\\) | 1.000 | 0.000 | 0.000 | 0.200 | 3.800 |
| \\(X_{4}\\) | 0.000 | 1.000 | 0.000 | 0.200 | 3.800 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 12.000 |
| \\(X_{2}\\) | 0.000 | 0.000 | 0.000 | 0.200 | 1.800 |


## Iteration 1

Starting Tableau:

|   |  |   | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | 0.000 | 0.200 | 3.800 |
| \\(X_{7}\\) | 1.000 | 0.000 | 0.000 | 0.200 | 3.800 |
| \\(X_{4}\\) | 0.000 | 1.000 | 0.000 | 0.200 | 3.800 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 12.000 |
| \\(X_{2}\\) | 0.000 | 0.000 | 0.000 | 0.200 | 1.800 |


Extended Tableau

|   |  |   | \\(A_I^{-1}\\) |   | RHS | \\(X_{0}\\)
|---|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | 0.000 | 0.200 | 3.800 | 1.800 |
| \\(X_{7}\\) | 1.000 | 0.000 | 0.000 | 0.200 | 3.800 | 1.800 |
| \\(X_{4}\\) | 0.000 | 1.000 | 0.000 | 0.200 | 3.800 | 1.800 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 12.000 | 1.000 |
| \\(X_{2}\\) | 0.000 | 0.000 | 0.000 | 0.200 | 1.800 | -0.200 |


\\(Ĉ_J = \ \\)array([ 1.8, -0.4, -0.8, -1. , -0.2])


\\(J = {[0, 1, 8, 3, 6]}\\)

### Pivot Operations

Variable to enter: \\(X_{0}\\)
Variable to leave: \\(X_{7}\\)
#### Row Operations:

1. \\(R_{X_{7}} \leftarrow \frac{R_{X_{7}}}{1.800}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 1.800R_{X_{7}}\\)
2. \\(R_{X_{4}} \leftarrow R_{X_{4}} - 1.800R_{X_{7}}\\)
3. \\(R_{X_{5}} \leftarrow R_{X_{5}} - 1.000R_{X_{7}}\\)
4. \\(R_{X_{2}} \leftarrow R_{X_{2}} + 0.200R_{X_{7}}\\)


Computed Tableau:

|   |  |   | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | -0.000 | -0.000 |
| \\(X_{0}\\) | 0.556 | 0.000 | 0.000 | 0.111 | 2.111 |
| \\(X_{4}\\) | -1.000 | 1.000 | 0.000 | 0.000 | -0.000 |
| \\(X_{5}\\) | -0.556 | 0.000 | 1.000 | -0.111 | 9.889 |
| \\(X_{2}\\) | 0.111 | 0.000 | 0.000 | 0.222 | 2.222 |


## Iteration 2

Starting Tableau:

|   |  |   | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | -0.000 | -0.000 |
| \\(X_{0}\\) | 0.556 | 0.000 | 0.000 | 0.111 | 2.111 |
| \\(X_{4}\\) | -1.000 | 1.000 | 0.000 | 0.000 | -0.000 |
| \\(X_{5}\\) | -0.556 | 0.000 | 1.000 | -0.111 | 9.889 |
| \\(X_{2}\\) | 0.111 | 0.000 | 0.000 | 0.222 | 2.222 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
2.000 & -1.000 & -1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 1.000 & -0.000 \\
2.000 & -1.000 & -1.000 & 0.000 & 1.000 & 0.000 & 0.000 & -0.000 & -0.000 \\
1.000 & 1.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & -0.000 & -0.000 \\
-1.000 & 3.000 & 5.000 & 0.000 & 0.000 & 0.000 & -1.000 & -0.000 & 1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
2.000\\
2.000\\
12.000\\
9.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 1.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}\\
X_{7}\\
X_{8}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal multiple solutions

Optimal Solution: \\(X^{*} = ['2.111', '0.000', '2.222', '0.000', '-0.000', '9.889', '0.000', '0.000', '0.000']\\)

Optimal Value: \\(Z^{*} = -0.000\\)

Optimal Basis: \\([0, 4, 5, 2]\\)

Final non-basic variables set J: \\([7, 1, 8, 3, 6]\\)

\\(\hat{C}_{J}\\): \\([-1.0, -0.0, -1.0, 0.0, 0.0]\\)

Number of Iterations: 3



In [116]:
iterations = list(solution.keys()).pop()
I = solution[iterations]['I']
# resolvendo problema original com o simplex tableau revisado com BIF

solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  |   | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|---|
| Z | 1.556 | 0.000 | 0.000 | 1.111 | 13.111 |
| \\(X_{0}\\) | 0.556 | 0.000 | 0.000 | 0.111 | 2.111 |
| \\(X_{4}\\) | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | -0.556 | 0.000 | 1.000 | -0.111 | 9.889 |
| \\(X_{2}\\) | 0.111 | 0.000 | 0.000 | 0.222 | 2.222 |


Extended Tableau

|   |  |   | \\(A_I^{-1}\\) |   | RHS | \\(X_{1}\\)
|---|---|---|---|---|---|---|
| Z | 1.556 | 0.000 | 0.000 | 1.111 | 13.111 | 4.778 |
| \\(X_{0}\\) | 0.556 | 0.000 | 0.000 | 0.111 | 2.111 | -0.222 |
| \\(X_{4}\\) | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | -0.556 | 0.000 | 1.000 | -0.111 | 9.889 | 1.222 |
| \\(X_{2}\\) | 0.111 | 0.000 | 0.000 | 0.222 | 2.222 | 0.556 |


\\(Ĉ_J = \ \\)array([ 4.777778, -1.555556, -1.111111])


\\(J = {[1, 3, 6]}\\)

### Pivot Operations

Variable to enter: \\(X_{1}\\)
Variable to leave: \\(X_{2}\\)
#### Row Operations:

1. \\(R_{X_{2}} \leftarrow \frac{R_{X_{2}}}{0.556}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 4.778R_{X_{2}}\\)
2. \\(R_{X_{0}} \leftarrow R_{X_{0}} + 0.222R_{X_{2}}\\)
3. \\(R_{X_{4}} \leftarrow R_{X_{4}} - 0.000R_{X_{2}}\\)
4. \\(R_{X_{5}} \leftarrow R_{X_{5}} - 1.222R_{X_{2}}\\)


Computed Tableau:

|   |  |   | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|---|
| Z | 0.600 | 0.000 | 0.000 | -0.800 | -6.000 |
| \\(X_{0}\\) | 0.600 | 0.000 | 0.000 | 0.200 | 3.000 |
| \\(X_{4}\\) | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | -0.800 | 0.000 | 1.000 | -0.600 | 5.000 |
| \\(X_{1}\\) | 0.200 | 0.000 | 0.000 | 0.400 | 4.000 |


## Iteration 1

Starting Tableau:

|   |  |   | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|---|
| Z | 0.600 | 0.000 | 0.000 | -0.800 | -6.000 |
| \\(X_{0}\\) | 0.600 | 0.000 | 0.000 | 0.200 | 3.000 |
| \\(X_{4}\\) | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | -0.800 | 0.000 | 1.000 | -0.600 | 5.000 |
| \\(X_{1}\\) | 0.200 | 0.000 | 0.000 | 0.400 | 4.000 |


Extended Tableau

|   |  |   | \\(A_I^{-1}\\) |   | RHS | \\(X_{6}\\)
|---|---|---|---|---|---|---|
| Z | 0.600 | 0.000 | 0.000 | -0.800 | -6.000 | 0.800 |
| \\(X_{0}\\) | 0.600 | 0.000 | 0.000 | 0.200 | 3.000 | -0.200 |
| \\(X_{4}\\) | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | -0.800 | 0.000 | 1.000 | -0.600 | 5.000 | 0.600 |
| \\(X_{1}\\) | 0.200 | 0.000 | 0.000 | 0.400 | 4.000 | -0.400 |


\\(Ĉ_J = \ \\)array([-8.6, -0.6,  0.8])


\\(J = {[2, 3, 6]}\\)

### Pivot Operations

Variable to enter: \\(X_{6}\\)
Variable to leave: \\(X_{5}\\)
#### Row Operations:

1. \\(R_{X_{5}} \leftarrow \frac{R_{X_{5}}}{0.600}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 0.800R_{X_{5}}\\)
2. \\(R_{X_{0}} \leftarrow R_{X_{0}} + 0.200R_{X_{5}}\\)
3. \\(R_{X_{4}} \leftarrow R_{X_{4}} - 0.000R_{X_{5}}\\)
4. \\(R_{X_{1}} \leftarrow R_{X_{1}} + 0.400R_{X_{5}}\\)


Computed Tableau:

|   |  |   | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|---|
| Z | 1.667 | 0.000 | -1.333 | -0.000 | -12.667 |
| \\(X_{0}\\) | 0.333 | 0.000 | 0.333 | 0.000 | 4.667 |
| \\(X_{4}\\) | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{6}\\) | -1.333 | 0.000 | 1.667 | -1.000 | 8.333 |
| \\(X_{1}\\) | -0.333 | 0.000 | 0.667 | 0.000 | 7.333 |


## Iteration 2

Starting Tableau:

|   |  |   | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|---|
| Z | 1.667 | 0.000 | -1.333 | -0.000 | -12.667 |
| \\(X_{0}\\) | 0.333 | 0.000 | 0.333 | 0.000 | 4.667 |
| \\(X_{4}\\) | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{6}\\) | -1.333 | 0.000 | 1.667 | -1.000 | 8.333 |
| \\(X_{1}\\) | -0.333 | 0.000 | 0.667 | 0.000 | 7.333 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
2.000 & -1.000 & -1.000 & -1.000 & 0.000 & 0.000 & 0.000 \\
2.000 & -1.000 & -1.000 & 0.000 & 1.000 & 0.000 & 0.000 \\
1.000 & 1.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 \\
-1.000 & 3.000 & 5.000 & 0.000 & 0.000 & 0.000 & -1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
2.000\\
2.000\\
12.000\\
9.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
2.000 & -3.000 & 4.000 & 0.000 & 0.000 & 0.000 & 0.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal unique solution

Optimal Solution: \\(X^{*} = ['4.667', '7.333', '0.000', '0.000', '0.000', '0.000', '8.333']\\)

Optimal Value: \\(Z^{*} = -12.667\\)

Optimal Basis: \\([0, 4, 6, 1]\\)

Final non-basic variables set J: \\([2, 3, 5]\\)

\\(\hat{C}_{J}\\): \\([-5.666667, -1.666667, -1.333333]\\)

Number of Iterations: 3



In [117]:
result = linprog(C, A_eq = A, b_eq = B, method='highs')
display(result)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -12.666666666666666
              x: [ 4.667e+00  7.333e+00  0.000e+00  0.000e+00 -0.000e+00
                   0.000e+00  8.333e+00]
            nit: 4
          lower:  residual: [ 4.667e+00  7.333e+00  0.000e+00  0.000e+00
                             -0.000e+00  0.000e+00  8.333e+00]
                 marginals: [ 0.000e+00  0.000e+00  5.667e+00  1.667e+00
                              0.000e+00  1.333e+00  0.000e+00]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00  0.000e+00]
          eqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00]
                 marginals: [ 1.667e+00 -0.000e+00 -1.333e+00 -0.000e+00]
        ineqlin:

## letra d)

In [118]:
# dual

C = - np.array([
    3, 1, 4, -4, 0, 0, 0, 0, 0, 0, 0
])

A = np.array([
    [ 1,  1,  1, -1,  1, -1],
    [-1, -1, -1,  1, -1,  1],
    [-1,  1, -2,  2,  0,  0],
    [ 0,  0,  1, -1,  0,  0],
    [ 0,  0,  0,  0,  1, -1]
])

excesses_slacks = np.eye(5)
excesses_slacks[0, 0] = -1
excesses_slacks[2, 2] = -1
A = np.hstack((A, excesses_slacks))

B = np.array([
    1,
    1,
    3,
    0,
    0
])

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = np.arange(6, 11)
I[0] = 11
I[2] = 12
I = I.tolist()

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 4.000 |
| \\(X_{11}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{7}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 3.000 |
| \\(X_{9}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{10}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS | \\(X_{1}\\)
|---|---|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 4.000 | 2.000 |
| \\(X_{11}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 1.000 |
| \\(X_{7}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 1.000 | -1.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 3.000 | 1.000 |
| \\(X_{9}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{10}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |


\\(Ĉ_J = \ \\)array([ 0.,  2., -1.,  1.,  1., -1., -1., -1.])


\\(J = {[0, 1, 2, 3, 4, 5, 6, 8]}\\)

### Pivot Operations

Variable to enter: \\(X_{1}\\)
Variable to leave: \\(X_{11}\\)
#### Row Operations:

1. \\(R_{X_{11}} \leftarrow \frac{R_{X_{11}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 2.000R_{X_{11}}\\)
2. \\(R_{X_{7}} \leftarrow R_{X_{7}} + 1.000R_{X_{11}}\\)
3. \\(R_{X_{12}} \leftarrow R_{X_{12}} - 1.000R_{X_{11}}\\)
4. \\(R_{X_{9}} \leftarrow R_{X_{9}} - 0.000R_{X_{11}}\\)
5. \\(R_{X_{10}} \leftarrow R_{X_{10}} - 0.000R_{X_{11}}\\)


Computed Tableau:

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|
| Z | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{1}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{7}\\) | 1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{12}\\) | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{9}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{10}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Iteration 1

Starting Tableau:

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|
| Z | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{1}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{7}\\) | 1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{12}\\) | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{9}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{10}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS | \\(X_{3}\\)
|---|---|---|---|---|---|---|---|
| Z | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 2.000 | 3.000 |
| \\(X_{1}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | -1.000 |
| \\(X_{7}\\) | 1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 2.000 | 0.000 |
| \\(X_{12}\\) | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 2.000 | 3.000 |
| \\(X_{9}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | -1.000 |
| \\(X_{10}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |


\\(Ĉ_J = \ \\)array([-2., -2., -3.,  3., -1.,  1.,  1., -1.])


\\(J = {[0, 11, 2, 3, 4, 5, 6, 8]}\\)

### Pivot Operations

Variable to enter: \\(X_{3}\\)
Variable to leave: \\(X_{12}\\)
#### Row Operations:

1. \\(R_{X_{12}} \leftarrow \frac{R_{X_{12}}}{3.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 3.000R_{X_{12}}\\)
2. \\(R_{X_{1}} \leftarrow R_{X_{1}} + 1.000R_{X_{12}}\\)
3. \\(R_{X_{7}} \leftarrow R_{X_{7}} - 0.000R_{X_{12}}\\)
4. \\(R_{X_{9}} \leftarrow R_{X_{9}} + 1.000R_{X_{12}}\\)
5. \\(R_{X_{10}} \leftarrow R_{X_{10}} - 0.000R_{X_{12}}\\)


Computed Tableau:

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{1}\\) | 0.667 | 0.000 | 0.333 | 0.000 | 0.000 | 1.667 |
| \\(X_{7}\\) | 1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{3}\\) | -0.333 | 0.000 | 0.333 | 0.000 | 0.000 | 0.667 |
| \\(X_{9}\\) | -0.333 | 0.000 | 0.333 | 1.000 | 0.000 | 0.667 |
| \\(X_{10}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Iteration 2

Starting Tableau:

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{1}\\) | 0.667 | 0.000 | 0.333 | 0.000 | 0.000 | 1.667 |
| \\(X_{7}\\) | 1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{3}\\) | -0.333 | 0.000 | 0.333 | 0.000 | 0.000 | 0.667 |
| \\(X_{9}\\) | -0.333 | 0.000 | 0.333 | 1.000 | 0.000 | 0.667 |
| \\(X_{10}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
1.000 & 1.000 & 1.000 & -1.000 & 1.000 & -1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -0.000 \\
-1.000 & -1.000 & -1.000 & 1.000 & -1.000 & 1.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & -0.000 & -0.000 \\
-1.000 & 1.000 & -2.000 & 2.000 & 0.000 & 0.000 & 0.000 & 0.000 & -1.000 & 0.000 & 0.000 & -0.000 & 1.000 \\
0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & -0.000 & -0.000 \\
0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -0.000 & -0.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
1.000\\
1.000\\
3.000\\
0.000\\
0.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 1.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}\\
X_{7}\\
X_{8}\\
X_{9}\\
X_{10}\\
X_{11}\\
X_{12}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal multiple solutions

Optimal Solution: \\(X^{*} = ['0.000', '1.667', '0.000', '0.667', '0.000', '0.000', '0.000', '2.000', '0.000', '0.667', '0.000', '0.000', '0.000']\\)

Optimal Value: \\(Z^{*} = 0.000\\)

Optimal Basis: \\([1, 7, 3, 9, 10]\\)

Final non-basic variables set J: \\([0, 11, 2, 12, 4, 5, 6, 8]\\)

\\(\hat{C}_{J}\\): \\([0.0, -1.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0]\\)

Number of Iterations: 3



In [119]:
iterations = list(solution.keys()).pop()
I = solution[iterations]['I']
# resolvendo problema original com o simplex tableau revisado com BIF

solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|
| Z | -2.000 | 0.000 | 1.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{1}\\) | 0.667 | 0.000 | 0.333 | 0.000 | 0.000 | 1.667 |
| \\(X_{7}\\) | 1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{3}\\) | -0.333 | 0.000 | 0.333 | 0.000 | 0.000 | 0.667 |
| \\(X_{9}\\) | -0.333 | 0.000 | 0.333 | 1.000 | 0.000 | 0.667 |
| \\(X_{10}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS | \\(X_{5}\\)
|---|---|---|---|---|---|---|---|
| Z | -2.000 | 0.000 | 1.000 | 0.000 | 0.000 | 1.000 | 2.000 |
| \\(X_{1}\\) | 0.667 | 0.000 | 0.333 | 0.000 | 0.000 | 1.667 | -0.667 |
| \\(X_{7}\\) | 1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 2.000 | 0.000 |
| \\(X_{3}\\) | -0.333 | 0.000 | 0.333 | 0.000 | 0.000 | 0.667 | 0.333 |
| \\(X_{9}\\) | -0.333 | 0.000 | 0.333 | 1.000 | 0.000 | 0.667 | 0.333 |
| \\(X_{10}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | -1.000 |


\\(Ĉ_J = \ \\)array([ 0.,  0., -2.,  2.,  2., -1.])


\\(J = {[0, 2, 4, 5, 6, 8]}\\)

### Pivot Operations

Variable to enter: \\(X_{5}\\)
Variable to leave: \\(X_{3}\\)
#### Row Operations:

1. \\(R_{X_{3}} \leftarrow \frac{R_{X_{3}}}{0.333}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 2.000R_{X_{3}}\\)
2. \\(R_{X_{1}} \leftarrow R_{X_{1}} + 0.667R_{X_{3}}\\)
3. \\(R_{X_{7}} \leftarrow R_{X_{7}} - 0.000R_{X_{3}}\\)
4. \\(R_{X_{9}} \leftarrow R_{X_{9}} - 0.333R_{X_{3}}\\)
5. \\(R_{X_{10}} \leftarrow R_{X_{10}} + 1.000R_{X_{3}}\\)


Computed Tableau:

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | -1.000 | 0.000 | 0.000 | -3.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 3.000 |
| \\(X_{7}\\) | 1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{5}\\) | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{9}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{10}\\) | -1.000 | 0.000 | 1.000 | 0.000 | 1.000 | 2.000 |


## Iteration 1

Starting Tableau:

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | -1.000 | 0.000 | 0.000 | -3.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 3.000 |
| \\(X_{7}\\) | 1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{5}\\) | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{9}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{10}\\) | -1.000 | 0.000 | 1.000 | 0.000 | 1.000 | 2.000 |


Extended Tableau

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS | \\(X_{2}\\)
|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | -1.000 | 0.000 | 0.000 | -3.000 | 6.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 3.000 | -2.000 |
| \\(X_{7}\\) | 1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 2.000 | 0.000 |
| \\(X_{5}\\) | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 2.000 | -3.000 |
| \\(X_{9}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{10}\\) | -1.000 | 0.000 | 1.000 | 0.000 | 1.000 | 2.000 | -3.000 |


\\(Ĉ_J = \ \\)array([ 4.,  6.,  0., -6.,  0.,  1.])


\\(J = {[0, 2, 4, 3, 6, 8]}\\)

### Pivot Operations

Variable to enter: \\(X_{2}\\)
Variable to leave: \\(X_{9}\\)
#### Row Operations:

1. \\(R_{X_{9}} \leftarrow \frac{R_{X_{9}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 6.000R_{X_{9}}\\)
2. \\(R_{X_{1}} \leftarrow R_{X_{1}} + 2.000R_{X_{9}}\\)
3. \\(R_{X_{7}} \leftarrow R_{X_{7}} - 0.000R_{X_{9}}\\)
4. \\(R_{X_{5}} \leftarrow R_{X_{5}} + 3.000R_{X_{9}}\\)
5. \\(R_{X_{10}} \leftarrow R_{X_{10}} + 3.000R_{X_{9}}\\)


Computed Tableau:

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | -1.000 | -6.000 | 0.000 | -3.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 1.000 | 2.000 | 0.000 | 3.000 |
| \\(X_{7}\\) | 1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{5}\\) | -1.000 | 0.000 | 1.000 | 3.000 | 0.000 | 2.000 |
| \\(X_{2}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{10}\\) | -1.000 | 0.000 | 1.000 | 3.000 | 1.000 | 2.000 |


## Iteration 2

Starting Tableau:

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | -1.000 | -6.000 | 0.000 | -3.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 1.000 | 2.000 | 0.000 | 3.000 |
| \\(X_{7}\\) | 1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{5}\\) | -1.000 | 0.000 | 1.000 | 3.000 | 0.000 | 2.000 |
| \\(X_{2}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{10}\\) | -1.000 | 0.000 | 1.000 | 3.000 | 1.000 | 2.000 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
1.000 & 1.000 & 1.000 & -1.000 & 1.000 & -1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
-1.000 & -1.000 & -1.000 & 1.000 & -1.000 & 1.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 \\
-1.000 & 1.000 & -2.000 & 2.000 & 0.000 & 0.000 & 0.000 & 0.000 & -1.000 & 0.000 & 0.000 \\
0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 \\
0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
1.000\\
1.000\\
3.000\\
0.000\\
0.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
-3.000 & -1.000 & -4.000 & 4.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}\\
X_{7}\\
X_{8}\\
X_{9}\\
X_{10}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Unbounded solution

Optimal Solution: \\(X^{*} = ['0.000', '3.000', '0.000', '0.000', '0.000', '2.000', '0.000', '2.000', '0.000', '0.000', '2.000']\\)

Optimal Value: \\(Z^{*} = -3.000\\)

Optimal Basis: \\([1, 7, 5, 2, 10]\\)

Final non-basic variables set J: \\([0, 9, 4, 3, 6, 8]\\)

\\(\hat{C}_{J}\\): \\([4.0, -6.0, 0.0, 0.0, 0.0, 1.0]\\)

Number of Iterations: 3



In [120]:
result = linprog(C, A_eq = A, b_eq = B, method='highs')
display(result)

       message: The problem is unbounded. (HiGHS Status 10: model_status is Unbounded; primal_status is At upper bound)
       success: False
        status: 3
           fun: None
             x: None
           nit: 6
         lower:  residual: None
                marginals: None
         upper:  residual: None
                marginals: None
         eqlin:  residual: None
                marginals: None
       ineqlin:  residual: None
                marginals: None

In [121]:
# problema primal

C = np.array([
    -1, 1, -3, 0, 0, 0, 0
])
A = np.array([
    [ 1, -1,  1],
    [ 1, -1, -1],
    [ 1, -1,  2],
    [ 1,  1,  0]
])
excesses_slacks = np.eye(4)
excesses_slacks[0, 0] = -1
excesses_slacks[1, 1] = -1
A = np.hstack((A, excesses_slacks))
B = np.array([
    3,
    1,
    4,
    0
])

# primeira fase, variáveis artificiais

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = np.arange(3, 7)
I[0] = 7
I[1] = 8
I = I.tolist()

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  |   | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | 0.000 | 0.000 | 4.000 |
| \\(X_{7}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 3.000 |
| \\(X_{8}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 4.000 |
| \\(X_{6}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   | \\(A_I^{-1}\\) |   | RHS | \\(X_{0}\\)
|---|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | 0.000 | 0.000 | 4.000 | 2.000 |
| \\(X_{7}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 3.000 | 1.000 |
| \\(X_{8}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 1.000 | 1.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 4.000 | 1.000 |
| \\(X_{6}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 1.000 |


\\(Ĉ_J = \ \\)array([ 2., -2.,  0., -1., -1.])


\\(J = {[0, 1, 2, 3, 4]}\\)

### Pivot Operations

Variable to enter: \\(X_{0}\\)
Variable to leave: \\(X_{6}\\)
#### Row Operations:

1. \\(R_{X_{6}} \leftarrow \frac{R_{X_{6}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 2.000R_{X_{6}}\\)
2. \\(R_{X_{7}} \leftarrow R_{X_{7}} - 1.000R_{X_{6}}\\)
3. \\(R_{X_{8}} \leftarrow R_{X_{8}} - 1.000R_{X_{6}}\\)
4. \\(R_{X_{5}} \leftarrow R_{X_{5}} - 1.000R_{X_{6}}\\)


Computed Tableau:

|   |  |   | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | 0.000 | -2.000 | 4.000 |
| \\(X_{7}\\) | 1.000 | 0.000 | 0.000 | -1.000 | 3.000 |
| \\(X_{8}\\) | 0.000 | 1.000 | 0.000 | -1.000 | 1.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | -1.000 | 4.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Iteration 1

Starting Tableau:

|   |  |   | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | 0.000 | -2.000 | 4.000 |
| \\(X_{7}\\) | 1.000 | 0.000 | 0.000 | -1.000 | 3.000 |
| \\(X_{8}\\) | 0.000 | 1.000 | 0.000 | -1.000 | 1.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | -1.000 | 4.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
1.000 & -1.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 1.000 & -0.000 \\
1.000 & -1.000 & -1.000 & 0.000 & -1.000 & 0.000 & 0.000 & -0.000 & 1.000 \\
1.000 & -1.000 & 2.000 & 0.000 & 0.000 & 1.000 & 0.000 & -0.000 & -0.000 \\
1.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -0.000 & -0.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
3.000\\
1.000\\
4.000\\
0.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 1.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}\\
X_{7}\\
X_{8}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal multiple solutions

Optimal Solution: \\(X^{*} = ['0.000', '0.000', '0.000', '0.000', '0.000', '4.000', '0.000', '3.000', '1.000']\\)

Optimal Value: \\(Z^{*} = 4.000\\)

Optimal Basis: \\([7, 8, 5, 0]\\)

Final non-basic variables set J: \\([6, 1, 2, 3, 4]\\)

\\(\hat{C}_{J}\\): \\([-2.0, -4.0, 0.0, -1.0, -1.0]\\)

Number of Iterations: 2



## letra e)

In [122]:
C = np.array([
    -1, 2, 5, 16, 12, 21, 10, 0, 0
])
A = np.array([
    [-1,  1,  1,  2,  1,  1,  0, -1,  0],
    [ 1, -1,  0,  1,  1,  2,  1,  0, -1]
])
B = np.array([
    3,
    2
])

# primeira fase, variáveis artificiais

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = [9, 10]

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | 1.000 | 1.000 | 5.000 |
| \\(X_{9}\\) | 1.000 | 0.000 | 3.000 |
| \\(X_{10}\\) | 0.000 | 1.000 | 2.000 |


Extended Tableau

|   |  | \\(A_I^{-1}\\) | RHS | \\(X_{3}\\)
|---|---|---|---|---|
| Z | 1.000 | 1.000 | 5.000 | 3.000 |
| \\(X_{9}\\) | 1.000 | 0.000 | 3.000 | 2.000 |
| \\(X_{10}\\) | 0.000 | 1.000 | 2.000 | 1.000 |


\\(Ĉ_J = \ \\)array([ 0.,  0.,  1.,  3.,  2.,  3.,  1., -1., -1.])


\\(J = {[0, 1, 2, 3, 4, 5, 6, 7, 8]}\\)

### Pivot Operations

Variable to enter: \\(X_{3}\\)
Variable to leave: \\(X_{9}\\)
#### Row Operations:

1. \\(R_{X_{9}} \leftarrow \frac{R_{X_{9}}}{2.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 3.000R_{X_{9}}\\)
2. \\(R_{X_{10}} \leftarrow R_{X_{10}} - 1.000R_{X_{9}}\\)


Computed Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | -0.500 | 1.000 | 0.500 |
| \\(X_{3}\\) | 0.500 | 0.000 | 1.500 |
| \\(X_{10}\\) | -0.500 | 1.000 | 0.500 |


## Iteration 1

Starting Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | -0.500 | 1.000 | 0.500 |
| \\(X_{3}\\) | 0.500 | 0.000 | 1.500 |
| \\(X_{10}\\) | -0.500 | 1.000 | 0.500 |


Extended Tableau

|   |  | \\(A_I^{-1}\\) | RHS | \\(X_{0}\\)
|---|---|---|---|---|
| Z | -0.500 | 1.000 | 0.500 | 1.500 |
| \\(X_{3}\\) | 0.500 | 0.000 | 1.500 | -0.500 |
| \\(X_{10}\\) | -0.500 | 1.000 | 0.500 | 1.500 |


\\(Ĉ_J = \ \\)array([ 1.5, -1.5, -0.5, -1.5,  0.5,  1.5,  1. ,  0.5, -1. ])


\\(J = {[0, 1, 2, 9, 4, 5, 6, 7, 8]}\\)

### Pivot Operations

Variable to enter: \\(X_{0}\\)
Variable to leave: \\(X_{10}\\)
#### Row Operations:

1. \\(R_{X_{10}} \leftarrow \frac{R_{X_{10}}}{1.500}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 1.500R_{X_{10}}\\)
2. \\(R_{X_{3}} \leftarrow R_{X_{3}} + 0.500R_{X_{10}}\\)


Computed Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 |
| \\(X_{3}\\) | 0.333 | 0.333 | 1.667 |
| \\(X_{0}\\) | -0.333 | 0.667 | 0.333 |


## Iteration 2

Starting Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 |
| \\(X_{3}\\) | 0.333 | 0.333 | 1.667 |
| \\(X_{0}\\) | -0.333 | 0.667 | 0.333 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
-1.000 & 1.000 & 1.000 & 2.000 & 1.000 & 1.000 & 0.000 & -1.000 & 0.000 & 1.000 & 0.000 \\
1.000 & -1.000 & 0.000 & 1.000 & 1.000 & 2.000 & 1.000 & 0.000 & -1.000 & 0.000 & 1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
3.000\\
2.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 1.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}\\
X_{7}\\
X_{8}\\
X_{9}\\
X_{10}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal multiple solutions

Optimal Solution: \\(X^{*} = ['0.333', '0.000', '0.000', '1.667', '0.000', '0.000', '0.000', '0.000', '0.000', '0.000', '0.000']\\)

Optimal Value: \\(Z^{*} = 0.000\\)

Optimal Basis: \\([3, 0]\\)

Final non-basic variables set J: \\([10, 1, 2, 9, 4, 5, 6, 7, 8]\\)

\\(\hat{C}_{J}\\): \\([-1.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0]\\)

Number of Iterations: 3



In [123]:
# segunda fase
iterations = list(solution.keys()).pop()
I = solution[iterations]['I']
# resolvendo problema original com o simplex tableau revisado com BIF

solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | 5.667 | 4.667 | 26.333 |
| \\(X_{3}\\) | 0.333 | 0.333 | 1.667 |
| \\(X_{0}\\) | -0.333 | 0.667 | 0.333 |


Extended Tableau

|   |  | \\(A_I^{-1}\\) | RHS | \\(X_{2}\\)
|---|---|---|---|---|
| Z | 5.667 | 4.667 | 26.333 | 0.667 |
| \\(X_{3}\\) | 0.333 | 0.333 | 1.667 | 0.333 |
| \\(X_{0}\\) | -0.333 | 0.667 | 0.333 | -0.333 |


\\(Ĉ_J = \ \\)array([-1.      ,  0.666667, -1.666667, -6.      , -5.333333, -5.666667,
       -4.666667])


\\(J = {[1, 2, 4, 5, 6, 7, 8]}\\)

### Pivot Operations

Variable to enter: \\(X_{2}\\)
Variable to leave: \\(X_{3}\\)
#### Row Operations:

1. \\(R_{X_{3}} \leftarrow \frac{R_{X_{3}}}{0.333}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 0.667R_{X_{3}}\\)
2. \\(R_{X_{0}} \leftarrow R_{X_{0}} + 0.333R_{X_{3}}\\)


Computed Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | 5.000 | 4.000 | 23.000 |
| \\(X_{2}\\) | 1.000 | 1.000 | 5.000 |
| \\(X_{0}\\) | 0.000 | 1.000 | 2.000 |


## Iteration 1

Starting Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | 5.000 | 4.000 | 23.000 |
| \\(X_{2}\\) | 1.000 | 1.000 | 5.000 |
| \\(X_{0}\\) | 0.000 | 1.000 | 2.000 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
-1.000 & 1.000 & 1.000 & 2.000 & 1.000 & 1.000 & 0.000 & -1.000 & 0.000 \\
1.000 & -1.000 & 0.000 & 1.000 & 1.000 & 2.000 & 1.000 & 0.000 & -1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
3.000\\
2.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
-1.000 & 2.000 & 5.000 & 16.000 & 12.000 & 21.000 & 10.000 & 0.000 & 0.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}\\
X_{7}\\
X_{8}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal unique solution

Optimal Solution: \\(X^{*} = ['2.000', '0.000', '5.000', '0.000', '0.000', '0.000', '0.000', '0.000', '0.000']\\)

Optimal Value: \\(Z^{*} = 23.000\\)

Optimal Basis: \\([2, 0]\\)

Final non-basic variables set J: \\([1, 3, 4, 5, 6, 7, 8]\\)

\\(\hat{C}_{J}\\): \\([-1.0, -2.000001, -3.000001, -8.000001, -6.0, -5.0, -4.0]\\)

Number of Iterations: 2



## letra f)

In [161]:
C = -np.array([1, -1, 2, -2, 0, 0, 0, 0] + [0] * 12)
slacks_and_excesses = np.eye(12)
excesses = [1, 3, 5]
for i in excesses:
    slacks_and_excesses[i , i] = -1
A = np.array([
    [5, -5, 1, -1, 1, -1, 0, 0],
    [5, -5, 1, -1, 1, -1, 0, 0],
    [1, -1, 1, -1, 0,  0, 1,-1],
    [1, -1, 1, -1, 0,  0, 1,-1],
    [1, -1, 1, -1, 0,  0, 0, 0],
    [1, -1, 1, -1, 0,  0, 0, 0],
    [3, -3, -2, 2, 0,  0, 0, 0],
    [-2, 2, 4, -4, 0,  0, 0, 0],
    [-1, 1, 0,  0, 0,  0, 0, 0],
    [ 0, 0, -1, 1, 0,  0, 0, 0],
    [ 0, 0, 0, 0,  1, -1, 0, 0],
    [ 0, 0, 0, 0,  0,  0, 1,-1]
])
A = np.hstack((A, slacks_and_excesses))
B = np.array([
    5,
    5,
    1,
    1,
    3,
    3,
    6,
    8,
    0,
    0,
    0,
    0
])

# primeira fase, variáveis artificiais

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = [8, 20, 10, 21, 12, 22, 14, 15, 16, 17, 18, 19]

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I, cycle_proof=True)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 1.000 | 0.000 | 1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 9.000 |
| \\(X_{8}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 5.000 |
| \\(X_{20}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 5.000 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{21}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.000 |
| \\(X_{22}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.000 |
| \\(X_{14}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 6.000 |
| \\(X_{15}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 8.000 |
| \\(X_{16}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{17}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{18}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{19}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS | \\(X_{0}\\)
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 1.000 | 0.000 | 1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 9.000 | 7.000 |
| \\(X_{8}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 5.000 | 5.000 |
| \\(X_{20}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 5.000 | 5.000 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 1.000 |
| \\(X_{21}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 1.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.000 | 1.000 |
| \\(X_{22}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.000 | 1.000 |
| \\(X_{14}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 6.000 | 3.000 |
| \\(X_{15}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 8.000 | -2.000 |
| \\(X_{16}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | -1.000 |
| \\(X_{17}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{18}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{19}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |


\\(Ĉ_J = \ \\)array([ 7., -7.,  3., -3.,  1., -1.,  1., -1., -1., -1., -1.])


\\(J = {[0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 13]}\\)

### Pivot Operations

Variable to enter: \\(X_{0}\\)
Variable to leave: \\(X_{14}\\)
#### Row Operations:

1. \\(R_{X_{14}} \leftarrow \frac{R_{X_{14}}}{3.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 7.000R_{X_{14}}\\)
2. \\(R_{X_{8}} \leftarrow R_{X_{8}} - 5.000R_{X_{14}}\\)
3. \\(R_{X_{20}} \leftarrow R_{X_{20}} - 5.000R_{X_{14}}\\)
4. \\(R_{X_{10}} \leftarrow R_{X_{10}} - 1.000R_{X_{14}}\\)
5. \\(R_{X_{21}} \leftarrow R_{X_{21}} - 1.000R_{X_{14}}\\)
6. \\(R_{X_{12}} \leftarrow R_{X_{12}} - 1.000R_{X_{14}}\\)
7. \\(R_{X_{22}} \leftarrow R_{X_{22}} - 1.000R_{X_{14}}\\)
8. \\(R_{X_{15}} \leftarrow R_{X_{15}} + 2.000R_{X_{14}}\\)
9. \\(R_{X_{16}} \leftarrow R_{X_{16}} + 1.000R_{X_{14}}\\)
10. \\(R_{X_{17}} \leftarrow R_{X_{17}} - 0.000R_{X_{14}}\\)
11. \\(R_{X_{18}} \leftarrow R_{X_{18}} - 0.000R_{X_{14}}\\)
12. \\(R_{X_{19}} \leftarrow R_{X_{19}} - 0.000R_{X_{14}}\\)


Computed Tableau:

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 1.000 | 0.000 | 1.000 | 0.000 | 1.000 | -2.333 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -5.000 |
| \\(X_{8}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -1.667 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -5.000 |
| \\(X_{20}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | -1.667 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -5.000 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -0.333 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -1.000 |
| \\(X_{21}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | -0.333 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -1.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | -0.333 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{22}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | -0.333 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.333 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{15}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.667 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 12.000 |
| \\(X_{16}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.333 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{17}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{18}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{19}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Iteration 1

Starting Tableau:

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 1.000 | 0.000 | 1.000 | 0.000 | 1.000 | -2.333 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -5.000 |
| \\(X_{8}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -1.667 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -5.000 |
| \\(X_{20}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | -1.667 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -5.000 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -0.333 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -1.000 |
| \\(X_{21}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | -0.333 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -1.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | -0.333 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{22}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | -0.333 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.333 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{15}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.667 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 12.000 |
| \\(X_{16}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.333 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{17}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{18}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{19}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS | \\(X_{2}\\)
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 1.000 | 0.000 | 1.000 | 0.000 | 1.000 | -2.333 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -5.000 | 7.667 |
| \\(X_{8}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -1.667 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -5.000 | 4.333 |
| \\(X_{20}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | -1.667 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -5.000 | 4.333 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -0.333 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -1.000 | 1.667 |
| \\(X_{21}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | -0.333 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -1.000 | 1.667 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | -0.333 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 1.667 |
| \\(X_{22}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | -0.333 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 1.667 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.333 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 | -0.667 |
| \\(X_{15}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.667 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 12.000 | 2.667 |
| \\(X_{16}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.333 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 2.000 | -0.667 |
| \\(X_{17}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -1.000 |
| \\(X_{18}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{19}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |


\\(Ĉ_J = \ \\)array([-2.333333, -0.      ,  7.666667, -7.666667,  1.      , -1.      ,
        1.      , -1.      , -1.      , -1.      , -1.      ])


\\(J = {[14, 1, 2, 3, 4, 5, 6, 7, 9, 11, 13]}\\)

### Pivot Operations

Variable to enter: \\(X_{2}\\)
Variable to leave: \\(X_{20}\\)
#### Row Operations:

1. \\(R_{X_{20}} \leftarrow \frac{R_{X_{20}}}{4.333}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 7.667R_{X_{20}}\\)
2. \\(R_{X_{8}} \leftarrow R_{X_{8}} - 4.333R_{X_{20}}\\)
3. \\(R_{X_{10}} \leftarrow R_{X_{10}} - 1.667R_{X_{20}}\\)
4. \\(R_{X_{21}} \leftarrow R_{X_{21}} - 1.667R_{X_{20}}\\)
5. \\(R_{X_{12}} \leftarrow R_{X_{12}} - 1.667R_{X_{20}}\\)
6. \\(R_{X_{22}} \leftarrow R_{X_{22}} - 1.667R_{X_{20}}\\)
7. \\(R_{X_{0}} \leftarrow R_{X_{0}} + 0.667R_{X_{20}}\\)
8. \\(R_{X_{15}} \leftarrow R_{X_{15}} - 2.667R_{X_{20}}\\)
9. \\(R_{X_{16}} \leftarrow R_{X_{16}} + 0.667R_{X_{20}}\\)
10. \\(R_{X_{17}} \leftarrow R_{X_{17}} + 1.000R_{X_{20}}\\)
11. \\(R_{X_{18}} \leftarrow R_{X_{18}} - 0.000R_{X_{20}}\\)
12. \\(R_{X_{19}} \leftarrow R_{X_{19}} - 0.000R_{X_{20}}\\)


Computed Tableau:

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | -0.769 | 0.000 | 1.000 | 0.000 | 1.000 | 0.615 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.846 |
| \\(X_{8}\\) | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{2}\\) | 0.000 | 0.231 | 0.000 | 0.000 | 0.000 | 0.000 | -0.385 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -1.154 |
| \\(X_{10}\\) | 0.000 | -0.385 | 1.000 | 0.000 | 0.000 | 0.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.923 |
| \\(X_{21}\\) | 0.000 | -0.385 | 0.000 | 1.000 | 0.000 | 0.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.923 |
| \\(X_{12}\\) | 0.000 | -0.385 | 0.000 | 0.000 | 1.000 | 0.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.923 |
| \\(X_{22}\\) | 0.000 | -0.385 | 0.000 | 0.000 | 0.000 | 1.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.923 |
| \\(X_{0}\\) | 0.000 | 0.154 | 0.000 | 0.000 | 0.000 | 0.000 | 0.077 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.231 |
| \\(X_{15}\\) | 0.000 | -0.615 | 0.000 | 0.000 | 0.000 | 0.000 | 1.692 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 15.077 |
| \\(X_{16}\\) | 0.000 | 0.154 | 0.000 | 0.000 | 0.000 | 0.000 | 0.077 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 1.231 |
| \\(X_{17}\\) | 0.000 | 0.231 | 0.000 | 0.000 | 0.000 | 0.000 | -0.385 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | -1.154 |
| \\(X_{18}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{19}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Iteration 2

Starting Tableau:

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | -0.769 | 0.000 | 1.000 | 0.000 | 1.000 | 0.615 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.846 |
| \\(X_{8}\\) | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{2}\\) | 0.000 | 0.231 | 0.000 | 0.000 | 0.000 | 0.000 | -0.385 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -1.154 |
| \\(X_{10}\\) | 0.000 | -0.385 | 1.000 | 0.000 | 0.000 | 0.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.923 |
| \\(X_{21}\\) | 0.000 | -0.385 | 0.000 | 1.000 | 0.000 | 0.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.923 |
| \\(X_{12}\\) | 0.000 | -0.385 | 0.000 | 0.000 | 1.000 | 0.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.923 |
| \\(X_{22}\\) | 0.000 | -0.385 | 0.000 | 0.000 | 0.000 | 1.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.923 |
| \\(X_{0}\\) | 0.000 | 0.154 | 0.000 | 0.000 | 0.000 | 0.000 | 0.077 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.231 |
| \\(X_{15}\\) | 0.000 | -0.615 | 0.000 | 0.000 | 0.000 | 0.000 | 1.692 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 15.077 |
| \\(X_{16}\\) | 0.000 | 0.154 | 0.000 | 0.000 | 0.000 | 0.000 | 0.077 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 1.231 |
| \\(X_{17}\\) | 0.000 | 0.231 | 0.000 | 0.000 | 0.000 | 0.000 | -0.385 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | -1.154 |
| \\(X_{18}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{19}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS | \\(X_{6}\\)
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | -0.769 | 0.000 | 1.000 | 0.000 | 1.000 | 0.615 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.846 | 1.000 |
| \\(X_{8}\\) | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{2}\\) | 0.000 | 0.231 | 0.000 | 0.000 | 0.000 | 0.000 | -0.385 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -1.154 | 0.000 |
| \\(X_{10}\\) | 0.000 | -0.385 | 1.000 | 0.000 | 0.000 | 0.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.923 | 1.000 |
| \\(X_{21}\\) | 0.000 | -0.385 | 0.000 | 1.000 | 0.000 | 0.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.923 | 1.000 |
| \\(X_{12}\\) | 0.000 | -0.385 | 0.000 | 0.000 | 1.000 | 0.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.923 | 0.000 |
| \\(X_{22}\\) | 0.000 | -0.385 | 0.000 | 0.000 | 0.000 | 1.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.923 | 0.000 |
| \\(X_{0}\\) | 0.000 | 0.154 | 0.000 | 0.000 | 0.000 | 0.000 | 0.077 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.231 | 0.000 |
| \\(X_{15}\\) | 0.000 | -0.615 | 0.000 | 0.000 | 0.000 | 0.000 | 1.692 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 15.077 | 0.000 |
| \\(X_{16}\\) | 0.000 | 0.154 | 0.000 | 0.000 | 0.000 | 0.000 | 0.077 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 1.231 | 0.000 |
| \\(X_{17}\\) | 0.000 | 0.231 | 0.000 | 0.000 | 0.000 | 0.000 | -0.385 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | -1.154 | 0.000 |
| \\(X_{18}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{19}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 1.000 |


\\(Ĉ_J = \ \\)array([ 0.615385,  0.      , -1.769231,  0.      , -0.769231,  0.769231,
        1.      , -1.      ,  0.769231, -1.      , -1.      ])


\\(J = {[14, 1, 20, 3, 4, 5, 6, 7, 9, 11, 13]}\\)

### Pivot Operations

Variable to enter: \\(X_{6}\\)
Variable to leave: \\(X_{19}\\)
#### Row Operations:

1. \\(R_{X_{19}} \leftarrow \frac{R_{X_{19}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 1.000R_{X_{19}}\\)
2. \\(R_{X_{8}} \leftarrow R_{X_{8}} - 0.000R_{X_{19}}\\)
3. \\(R_{X_{2}} \leftarrow R_{X_{2}} - 0.000R_{X_{19}}\\)
4. \\(R_{X_{10}} \leftarrow R_{X_{10}} - 1.000R_{X_{19}}\\)
5. \\(R_{X_{21}} \leftarrow R_{X_{21}} - 1.000R_{X_{19}}\\)
6. \\(R_{X_{12}} \leftarrow R_{X_{12}} - 0.000R_{X_{19}}\\)
7. \\(R_{X_{22}} \leftarrow R_{X_{22}} - 0.000R_{X_{19}}\\)
8. \\(R_{X_{0}} \leftarrow R_{X_{0}} - 0.000R_{X_{19}}\\)
9. \\(R_{X_{15}} \leftarrow R_{X_{15}} - 0.000R_{X_{19}}\\)
10. \\(R_{X_{16}} \leftarrow R_{X_{16}} - 0.000R_{X_{19}}\\)
11. \\(R_{X_{17}} \leftarrow R_{X_{17}} - 0.000R_{X_{19}}\\)
12. \\(R_{X_{18}} \leftarrow R_{X_{18}} - 0.000R_{X_{19}}\\)


Computed Tableau:

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | -0.769 | 0.000 | 1.000 | 0.000 | 1.000 | 0.615 | 0.000 | 0.000 | 0.000 | 0.000 | -1.000 | 3.846 |
| \\(X_{8}\\) | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{2}\\) | 0.000 | 0.231 | 0.000 | 0.000 | 0.000 | 0.000 | -0.385 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -1.154 |
| \\(X_{10}\\) | 0.000 | -0.385 | 1.000 | 0.000 | 0.000 | 0.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | -1.000 | 0.923 |
| \\(X_{21}\\) | 0.000 | -0.385 | 0.000 | 1.000 | 0.000 | 0.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | -1.000 | 0.923 |
| \\(X_{12}\\) | 0.000 | -0.385 | 0.000 | 0.000 | 1.000 | 0.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.923 |
| \\(X_{22}\\) | 0.000 | -0.385 | 0.000 | 0.000 | 0.000 | 1.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.923 |
| \\(X_{0}\\) | 0.000 | 0.154 | 0.000 | 0.000 | 0.000 | 0.000 | 0.077 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.231 |
| \\(X_{15}\\) | 0.000 | -0.615 | 0.000 | 0.000 | 0.000 | 0.000 | 1.692 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 15.077 |
| \\(X_{16}\\) | 0.000 | 0.154 | 0.000 | 0.000 | 0.000 | 0.000 | 0.077 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 1.231 |
| \\(X_{17}\\) | 0.000 | 0.231 | 0.000 | 0.000 | 0.000 | 0.000 | -0.385 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | -1.154 |
| \\(X_{18}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{6}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Iteration 3

Starting Tableau:

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | -0.769 | 0.000 | 1.000 | 0.000 | 1.000 | 0.615 | 0.000 | 0.000 | 0.000 | 0.000 | -1.000 | 3.846 |
| \\(X_{8}\\) | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{2}\\) | 0.000 | 0.231 | 0.000 | 0.000 | 0.000 | 0.000 | -0.385 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -1.154 |
| \\(X_{10}\\) | 0.000 | -0.385 | 1.000 | 0.000 | 0.000 | 0.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | -1.000 | 0.923 |
| \\(X_{21}\\) | 0.000 | -0.385 | 0.000 | 1.000 | 0.000 | 0.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | -1.000 | 0.923 |
| \\(X_{12}\\) | 0.000 | -0.385 | 0.000 | 0.000 | 1.000 | 0.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.923 |
| \\(X_{22}\\) | 0.000 | -0.385 | 0.000 | 0.000 | 0.000 | 1.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.923 |
| \\(X_{0}\\) | 0.000 | 0.154 | 0.000 | 0.000 | 0.000 | 0.000 | 0.077 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.231 |
| \\(X_{15}\\) | 0.000 | -0.615 | 0.000 | 0.000 | 0.000 | 0.000 | 1.692 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 15.077 |
| \\(X_{16}\\) | 0.000 | 0.154 | 0.000 | 0.000 | 0.000 | 0.000 | 0.077 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 1.231 |
| \\(X_{17}\\) | 0.000 | 0.231 | 0.000 | 0.000 | 0.000 | 0.000 | -0.385 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | -1.154 |
| \\(X_{18}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{6}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS | \\(X_{5}\\)
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | -0.769 | 0.000 | 1.000 | 0.000 | 1.000 | 0.615 | 0.000 | 0.000 | 0.000 | 0.000 | -1.000 | 3.846 | 0.769 |
| \\(X_{8}\\) | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{2}\\) | 0.000 | 0.231 | 0.000 | 0.000 | 0.000 | 0.000 | -0.385 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -1.154 | -0.231 |
| \\(X_{10}\\) | 0.000 | -0.385 | 1.000 | 0.000 | 0.000 | 0.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | -1.000 | 0.923 | 0.385 |
| \\(X_{21}\\) | 0.000 | -0.385 | 0.000 | 1.000 | 0.000 | 0.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | -1.000 | 0.923 | 0.385 |
| \\(X_{12}\\) | 0.000 | -0.385 | 0.000 | 0.000 | 1.000 | 0.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.923 | 0.385 |
| \\(X_{22}\\) | 0.000 | -0.385 | 0.000 | 0.000 | 0.000 | 1.000 | 0.308 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.923 | 0.385 |
| \\(X_{0}\\) | 0.000 | 0.154 | 0.000 | 0.000 | 0.000 | 0.000 | 0.077 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.231 | -0.154 |
| \\(X_{15}\\) | 0.000 | -0.615 | 0.000 | 0.000 | 0.000 | 0.000 | 1.692 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 15.077 | 0.615 |
| \\(X_{16}\\) | 0.000 | 0.154 | 0.000 | 0.000 | 0.000 | 0.000 | 0.077 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 1.231 | -0.154 |
| \\(X_{17}\\) | 0.000 | 0.231 | 0.000 | 0.000 | 0.000 | 0.000 | -0.385 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | -1.154 | -0.231 |
| \\(X_{18}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | -1.000 |
| \\(X_{6}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |


\\(Ĉ_J = \ \\)array([ 0.615385,  0.      , -1.769231,  0.      , -0.769231,  0.769231,
       -1.      ,  0.      ,  0.769231, -1.      , -1.      ])


\\(J = {[14, 1, 20, 3, 4, 5, 19, 7, 9, 11, 13]}\\)

### Pivot Operations

Variable to enter: \\(X_{5}\\)
Variable to leave: \\(X_{21}\\)
#### Row Operations:

1. \\(R_{X_{21}} \leftarrow \frac{R_{X_{21}}}{0.385}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 0.769R_{X_{21}}\\)
2. \\(R_{X_{8}} \leftarrow R_{X_{8}} - 0.000R_{X_{21}}\\)
3. \\(R_{X_{2}} \leftarrow R_{X_{2}} + 0.231R_{X_{21}}\\)
4. \\(R_{X_{10}} \leftarrow R_{X_{10}} - 0.385R_{X_{21}}\\)
5. \\(R_{X_{12}} \leftarrow R_{X_{12}} - 0.385R_{X_{21}}\\)
6. \\(R_{X_{22}} \leftarrow R_{X_{22}} - 0.385R_{X_{21}}\\)
7. \\(R_{X_{0}} \leftarrow R_{X_{0}} + 0.154R_{X_{21}}\\)
8. \\(R_{X_{15}} \leftarrow R_{X_{15}} - 0.615R_{X_{21}}\\)
9. \\(R_{X_{16}} \leftarrow R_{X_{16}} + 0.154R_{X_{21}}\\)
10. \\(R_{X_{17}} \leftarrow R_{X_{17}} + 0.231R_{X_{21}}\\)
11. \\(R_{X_{18}} \leftarrow R_{X_{18}} + 1.000R_{X_{21}}\\)
12. \\(R_{X_{6}} \leftarrow R_{X_{6}} - 0.000R_{X_{21}}\\)


Computed Tableau:

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 1.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 |
| \\(X_{8}\\) | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{2}\\) | 0.000 | 0.000 | 0.000 | 0.600 | 0.000 | 0.000 | -0.200 | 0.000 | 0.000 | 0.000 | 0.000 | -0.600 | -0.600 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | 0.000 | -1.000 | 0.000 | 2.600 | 0.000 | 0.000 | 0.800 | 0.000 | 0.000 | 0.000 | 0.000 | -2.600 | 2.400 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 |
| \\(X_{22}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 0.400 | 0.000 | 0.000 | 0.200 | 0.000 | 0.000 | 0.000 | 0.000 | -0.400 | 1.600 |
| \\(X_{15}\\) | 0.000 | 0.000 | 0.000 | -1.600 | 0.000 | 0.000 | 1.200 | 1.000 | 0.000 | 0.000 | 0.000 | 1.600 | 13.600 |
| \\(X_{16}\\) | 0.000 | 0.000 | 0.000 | 0.400 | 0.000 | 0.000 | 0.200 | 0.000 | 1.000 | 0.000 | 0.000 | -0.400 | 1.600 |
| \\(X_{17}\\) | 0.000 | 0.000 | 0.000 | 0.600 | 0.000 | 0.000 | -0.200 | 0.000 | 0.000 | 1.000 | 0.000 | -0.600 | -0.600 |
| \\(X_{18}\\) | 0.000 | -1.000 | 0.000 | 2.600 | 0.000 | 0.000 | 0.800 | 0.000 | 0.000 | 0.000 | 1.000 | -2.600 | 2.400 |
| \\(X_{6}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Iteration 4

Starting Tableau:

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 1.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 |
| \\(X_{8}\\) | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{2}\\) | 0.000 | 0.000 | 0.000 | 0.600 | 0.000 | 0.000 | -0.200 | 0.000 | 0.000 | 0.000 | 0.000 | -0.600 | -0.600 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | 0.000 | -1.000 | 0.000 | 2.600 | 0.000 | 0.000 | 0.800 | 0.000 | 0.000 | 0.000 | 0.000 | -2.600 | 2.400 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 |
| \\(X_{22}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 0.400 | 0.000 | 0.000 | 0.200 | 0.000 | 0.000 | 0.000 | 0.000 | -0.400 | 1.600 |
| \\(X_{15}\\) | 0.000 | 0.000 | 0.000 | -1.600 | 0.000 | 0.000 | 1.200 | 1.000 | 0.000 | 0.000 | 0.000 | 1.600 | 13.600 |
| \\(X_{16}\\) | 0.000 | 0.000 | 0.000 | 0.400 | 0.000 | 0.000 | 0.200 | 0.000 | 1.000 | 0.000 | 0.000 | -0.400 | 1.600 |
| \\(X_{17}\\) | 0.000 | 0.000 | 0.000 | 0.600 | 0.000 | 0.000 | -0.200 | 0.000 | 0.000 | 1.000 | 0.000 | -0.600 | -0.600 |
| \\(X_{18}\\) | 0.000 | -1.000 | 0.000 | 2.600 | 0.000 | 0.000 | 0.800 | 0.000 | 0.000 | 0.000 | 1.000 | -2.600 | 2.400 |
| \\(X_{6}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS | \\(X_{19}\\)
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 1.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 | 1.000 |
| \\(X_{8}\\) | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{2}\\) | 0.000 | 0.000 | 0.000 | 0.600 | 0.000 | 0.000 | -0.200 | 0.000 | 0.000 | 0.000 | 0.000 | -0.600 | -0.600 | -0.600 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | 0.000 | -1.000 | 0.000 | 2.600 | 0.000 | 0.000 | 0.800 | 0.000 | 0.000 | 0.000 | 0.000 | -2.600 | 2.400 | -2.600 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 | 1.000 |
| \\(X_{22}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 | 1.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 0.400 | 0.000 | 0.000 | 0.200 | 0.000 | 0.000 | 0.000 | 0.000 | -0.400 | 1.600 | -0.400 |
| \\(X_{15}\\) | 0.000 | 0.000 | 0.000 | -1.600 | 0.000 | 0.000 | 1.200 | 1.000 | 0.000 | 0.000 | 0.000 | 1.600 | 13.600 | 1.600 |
| \\(X_{16}\\) | 0.000 | 0.000 | 0.000 | 0.400 | 0.000 | 0.000 | 0.200 | 0.000 | 1.000 | 0.000 | 0.000 | -0.400 | 1.600 | -0.400 |
| \\(X_{17}\\) | 0.000 | 0.000 | 0.000 | 0.600 | 0.000 | 0.000 | -0.200 | 0.000 | 0.000 | 1.000 | 0.000 | -0.600 | -0.600 | -0.600 |
| \\(X_{18}\\) | 0.000 | -1.000 | 0.000 | 2.600 | 0.000 | 0.000 | 0.800 | 0.000 | 0.000 | 0.000 | 1.000 | -2.600 | 2.400 | -2.600 |
| \\(X_{6}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 1.000 |


\\(Ĉ_J = \ \\)array([-0.      ,  0.      , -1.      ,  0.      ,  0.      , -2.000001,
        1.000001,  0.      , -0.      ,  1.000001, -1.      ])


\\(J = {[14, 1, 20, 3, 4, 21, 19, 7, 9, 11, 13]}\\)

### Pivot Operations

Variable to enter: \\(X_{19}\\)
Variable to leave: \\(X_{6}\\)
#### Row Operations:

1. \\(R_{X_{6}} \leftarrow \frac{R_{X_{6}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 1.000R_{X_{6}}\\)
2. \\(R_{X_{8}} \leftarrow R_{X_{8}} - 0.000R_{X_{6}}\\)
3. \\(R_{X_{2}} \leftarrow R_{X_{2}} + 0.600R_{X_{6}}\\)
4. \\(R_{X_{10}} \leftarrow R_{X_{10}} - 0.000R_{X_{6}}\\)
5. \\(R_{X_{5}} \leftarrow R_{X_{5}} + 2.600R_{X_{6}}\\)
6. \\(R_{X_{12}} \leftarrow R_{X_{12}} - 1.000R_{X_{6}}\\)
7. \\(R_{X_{22}} \leftarrow R_{X_{22}} - 1.000R_{X_{6}}\\)
8. \\(R_{X_{0}} \leftarrow R_{X_{0}} + 0.400R_{X_{6}}\\)
9. \\(R_{X_{15}} \leftarrow R_{X_{15}} - 1.600R_{X_{6}}\\)
10. \\(R_{X_{16}} \leftarrow R_{X_{16}} + 0.400R_{X_{6}}\\)
11. \\(R_{X_{17}} \leftarrow R_{X_{17}} + 0.600R_{X_{6}}\\)
12. \\(R_{X_{18}} \leftarrow R_{X_{18}} + 2.600R_{X_{6}}\\)


Computed Tableau:

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 1.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | 2.000 |
| \\(X_{8}\\) | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{2}\\) | 0.000 | 0.000 | 0.000 | 0.600 | 0.000 | 0.000 | -0.200 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.600 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | 0.000 | -1.000 | 0.000 | 2.600 | 0.000 | 0.000 | 0.800 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.400 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{22}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 0.400 | 0.000 | 0.000 | 0.200 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.600 |
| \\(X_{15}\\) | 0.000 | 0.000 | 0.000 | -1.600 | 0.000 | 0.000 | 1.200 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 13.600 |
| \\(X_{16}\\) | 0.000 | 0.000 | 0.000 | 0.400 | 0.000 | 0.000 | 0.200 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 1.600 |
| \\(X_{17}\\) | 0.000 | 0.000 | 0.000 | 0.600 | 0.000 | 0.000 | -0.200 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | -0.600 |
| \\(X_{18}\\) | 0.000 | -1.000 | 0.000 | 2.600 | 0.000 | 0.000 | 0.800 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 2.400 |
| \\(X_{19}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Iteration 5

Starting Tableau:

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 1.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | 2.000 |
| \\(X_{8}\\) | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{2}\\) | 0.000 | 0.000 | 0.000 | 0.600 | 0.000 | 0.000 | -0.200 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.600 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | 0.000 | -1.000 | 0.000 | 2.600 | 0.000 | 0.000 | 0.800 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.400 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{22}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 0.400 | 0.000 | 0.000 | 0.200 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.600 |
| \\(X_{15}\\) | 0.000 | 0.000 | 0.000 | -1.600 | 0.000 | 0.000 | 1.200 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 13.600 |
| \\(X_{16}\\) | 0.000 | 0.000 | 0.000 | 0.400 | 0.000 | 0.000 | 0.200 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 1.600 |
| \\(X_{17}\\) | 0.000 | 0.000 | 0.000 | 0.600 | 0.000 | 0.000 | -0.200 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | -0.600 |
| \\(X_{18}\\) | 0.000 | -1.000 | 0.000 | 2.600 | 0.000 | 0.000 | 0.800 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 2.400 |
| \\(X_{19}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS | \\(X_{7}\\)
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 1.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | 2.000 | 1.000 |
| \\(X_{8}\\) | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{2}\\) | 0.000 | 0.000 | 0.000 | 0.600 | 0.000 | 0.000 | -0.200 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.600 | -0.600 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | 0.000 | -1.000 | 0.000 | 2.600 | 0.000 | 0.000 | 0.800 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.400 | -2.600 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 | 1.000 |
| \\(X_{22}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 | 1.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 0.400 | 0.000 | 0.000 | 0.200 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.600 | -0.400 |
| \\(X_{15}\\) | 0.000 | 0.000 | 0.000 | -1.600 | 0.000 | 0.000 | 1.200 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 13.600 | 1.600 |
| \\(X_{16}\\) | 0.000 | 0.000 | 0.000 | 0.400 | 0.000 | 0.000 | 0.200 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 1.600 | -0.400 |
| \\(X_{17}\\) | 0.000 | 0.000 | 0.000 | 0.600 | 0.000 | 0.000 | -0.200 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | -0.600 | -0.600 |
| \\(X_{18}\\) | 0.000 | -1.000 | 0.000 | 2.600 | 0.000 | 0.000 | 0.800 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 2.400 | -2.600 |
| \\(X_{19}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | -1.000 |


\\(Ĉ_J = \ \\)array([-0.      ,  0.      , -1.      ,  0.      ,  0.      , -2.000001,
       -1.000001,  1.000001, -0.      ,  1.000001, -1.      ])


\\(J = {[14, 1, 20, 3, 4, 21, 6, 7, 9, 11, 13]}\\)

### Pivot Operations

Variable to enter: \\(X_{7}\\)
Variable to leave: \\(X_{22}\\)
#### Row Operations:

1. \\(R_{X_{22}} \leftarrow \frac{R_{X_{22}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 1.000R_{X_{22}}\\)
2. \\(R_{X_{8}} \leftarrow R_{X_{8}} - 0.000R_{X_{22}}\\)
3. \\(R_{X_{2}} \leftarrow R_{X_{2}} + 0.600R_{X_{22}}\\)
4. \\(R_{X_{10}} \leftarrow R_{X_{10}} - 0.000R_{X_{22}}\\)
5. \\(R_{X_{5}} \leftarrow R_{X_{5}} + 2.600R_{X_{22}}\\)
6. \\(R_{X_{12}} \leftarrow R_{X_{12}} - 1.000R_{X_{22}}\\)
7. \\(R_{X_{0}} \leftarrow R_{X_{0}} + 0.400R_{X_{22}}\\)
8. \\(R_{X_{15}} \leftarrow R_{X_{15}} - 1.600R_{X_{22}}\\)
9. \\(R_{X_{16}} \leftarrow R_{X_{16}} + 0.400R_{X_{22}}\\)
10. \\(R_{X_{17}} \leftarrow R_{X_{17}} + 0.600R_{X_{22}}\\)
11. \\(R_{X_{18}} \leftarrow R_{X_{18}} + 2.600R_{X_{22}}\\)
12. \\(R_{X_{19}} \leftarrow R_{X_{19}} + 1.000R_{X_{22}}\\)


Computed Tableau:

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | -0.000 |
| \\(X_{8}\\) | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{2}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.600 | -0.200 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.600 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | 0.000 | -1.000 | 0.000 | 0.000 | 0.000 | 2.600 | 0.800 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 7.600 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.400 | 0.200 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.400 |
| \\(X_{15}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -1.600 | 1.200 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 10.400 |
| \\(X_{16}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.400 | 0.200 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 2.400 |
| \\(X_{17}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.600 | -0.200 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.600 |
| \\(X_{18}\\) | 0.000 | -1.000 | 0.000 | 0.000 | 0.000 | 2.600 | 0.800 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 7.600 |
| \\(X_{19}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 |


## Iteration 6

Starting Tableau:

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | -0.000 |
| \\(X_{8}\\) | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{2}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.600 | -0.200 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.600 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | 0.000 | -1.000 | 0.000 | 0.000 | 0.000 | 2.600 | 0.800 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 7.600 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.400 | 0.200 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.400 |
| \\(X_{15}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -1.600 | 1.200 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 10.400 |
| \\(X_{16}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.400 | 0.200 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 2.400 |
| \\(X_{17}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.600 | -0.200 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.600 |
| \\(X_{18}\\) | 0.000 | -1.000 | 0.000 | 0.000 | 0.000 | 2.600 | 0.800 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 7.600 |
| \\(X_{19}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 |


Extended Tableau

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS | \\(X_{13}\\)
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | -0.000 | 0.000 |
| \\(X_{8}\\) | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{2}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.600 | -0.200 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.600 | -0.600 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | 0.000 | -1.000 | 0.000 | 0.000 | 0.000 | 2.600 | 0.800 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 7.600 | -2.600 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 | -1.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.400 | 0.200 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.400 | -0.400 |
| \\(X_{15}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -1.600 | 1.200 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 10.400 | 1.600 |
| \\(X_{16}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.400 | 0.200 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 2.400 | -0.400 |
| \\(X_{17}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.600 | -0.200 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.600 | -0.600 |
| \\(X_{18}\\) | 0.000 | -1.000 | 0.000 | 0.000 | 0.000 | 2.600 | 0.800 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 7.600 | -2.600 |
| \\(X_{19}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 | -1.000 |


\\(Ĉ_J = \ \\)array([-0.000000e+00,  0.000000e+00, -1.000000e+00,  0.000000e+00,
        0.000000e+00, -1.000000e+00,  0.000000e+00, -1.000001e+00,
       -0.000000e+00, -0.000000e+00,  1.000000e-06])


\\(J = {[14, 1, 20, 3, 4, 21, 6, 22, 9, 11, 13]}\\)

### Pivot Operations

Variable to enter: \\(X_{13}\\)
Variable to leave: \\(X_{12}\\)
#### Row Operations:

1. \\(R_{X_{12}} \leftarrow \frac{R_{X_{12}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 0.000R_{X_{12}}\\)
2. \\(R_{X_{8}} \leftarrow R_{X_{8}} - 0.000R_{X_{12}}\\)
3. \\(R_{X_{2}} \leftarrow R_{X_{2}} + 0.600R_{X_{12}}\\)
4. \\(R_{X_{10}} \leftarrow R_{X_{10}} - 0.000R_{X_{12}}\\)
5. \\(R_{X_{5}} \leftarrow R_{X_{5}} + 2.600R_{X_{12}}\\)
6. \\(R_{X_{7}} \leftarrow R_{X_{7}} + 1.000R_{X_{12}}\\)
7. \\(R_{X_{0}} \leftarrow R_{X_{0}} + 0.400R_{X_{12}}\\)
8. \\(R_{X_{15}} \leftarrow R_{X_{15}} - 1.600R_{X_{12}}\\)
9. \\(R_{X_{16}} \leftarrow R_{X_{16}} + 0.400R_{X_{12}}\\)
10. \\(R_{X_{17}} \leftarrow R_{X_{17}} + 0.600R_{X_{12}}\\)
11. \\(R_{X_{18}} \leftarrow R_{X_{18}} + 2.600R_{X_{12}}\\)
12. \\(R_{X_{19}} \leftarrow R_{X_{19}} + 1.000R_{X_{12}}\\)


Computed Tableau:

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | 0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | -0.000 |
| \\(X_{8}\\) | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{2}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.600 | 0.000 | -0.200 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.600 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | 0.000 | -1.000 | 0.000 | 0.000 | 2.600 | 0.000 | 0.800 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 7.600 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.400 | 0.000 | 0.200 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.400 |
| \\(X_{15}\\) | 0.000 | 0.000 | 0.000 | 0.000 | -1.600 | 0.000 | 1.200 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 10.400 |
| \\(X_{16}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.400 | 0.000 | 0.200 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 2.400 |
| \\(X_{17}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.600 | 0.000 | -0.200 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.600 |
| \\(X_{18}\\) | 0.000 | -1.000 | 0.000 | 0.000 | 2.600 | 0.000 | 0.800 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 7.600 |
| \\(X_{19}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 |


## Iteration 7

Starting Tableau:

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | 0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | -0.000 |
| \\(X_{8}\\) | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{2}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.600 | 0.000 | -0.200 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.600 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | 0.000 | -1.000 | 0.000 | 0.000 | 2.600 | 0.000 | 0.800 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 7.600 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.400 | 0.000 | 0.200 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.400 |
| \\(X_{15}\\) | 0.000 | 0.000 | 0.000 | 0.000 | -1.600 | 0.000 | 1.200 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 10.400 |
| \\(X_{16}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.400 | 0.000 | 0.200 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 2.400 |
| \\(X_{17}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.600 | 0.000 | -0.200 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.600 |
| \\(X_{18}\\) | 0.000 | -1.000 | 0.000 | 0.000 | 2.600 | 0.000 | 0.800 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 7.600 |
| \\(X_{19}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
5.000 & -5.000 & 1.000 & -1.000 & 1.000 & -1.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & -0.000 & -0.000 & -0.000 \\
5.000 & -5.000 & 1.000 & -1.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -0.000 & -0.000 \\
1.000 & -1.000 & 1.000 & -1.000 & 0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & -0.000 & -0.000 & -0.000 \\
1.000 & -1.000 & 1.000 & -1.000 & 0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & -0.000 & 1.000 & -0.000 \\
1.000 & -1.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & -0.000 & -0.000 & -0.000 \\
1.000 & -1.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & -0.000 & -0.000 & 1.000 \\
3.000 & -3.000 & -2.000 & 2.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & -0.000 & -0.000 & -0.000 \\
-2.000 & 2.000 & 4.000 & -4.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & -0.000 & -0.000 & -0.000 \\
-1.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & -0.000 & -0.000 & -0.000 \\
0.000 & 0.000 & -1.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & -0.000 & -0.000 & -0.000 \\
0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & -0.000 & -0.000 & -0.000 \\
0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -0.000 & -0.000 & -0.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
5.000\\
5.000\\
1.000\\
1.000\\
3.000\\
3.000\\
6.000\\
8.000\\
0.000\\
0.000\\
0.000\\
0.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 1.000 & 1.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}\\
X_{7}\\
X_{8}\\
X_{9}\\
X_{10}\\
X_{11}\\
X_{12}\\
X_{13}\\
X_{14}\\
X_{15}\\
X_{16}\\
X_{17}\\
X_{18}\\
X_{19}\\
X_{20}\\
X_{21}\\
X_{22}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal multiple solutions

Optimal Solution: \\(X^{*} = ['2.400', '0.000', '0.600', '0.000', '0.000', '7.600', '0.000', '2.000', '0.000', '0.000', '0.000', '0.000', '0.000', '0.000', '0.000', '10.400', '2.400', '0.600', '7.600', '2.000', '0.000', '0.000', '0.000']\\)

Optimal Value: \\(Z^{*} = -0.000\\)

Optimal Basis: \\([8, 2, 10, 5, 13, 7, 0, 15, 16, 17, 18, 19]\\)

Final non-basic variables set J: \\([14, 1, 20, 3, 4, 21, 6, 22, 9, 11, 12]\\)

\\(\hat{C}_{J}\\): \\([-0.0, 0.0, -1.0, 0.0, 0.0, -1.0, 0.0, -1.0, -0.0, -0.0, -1e-06]\\)

Number of Iterations: 8



In [141]:
# segunda fase
iterations = list(solution.keys()).pop()
I = solution[iterations]['I']
# resolvendo problema original com o simplex tableau revisado com BIF

solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 | -1.600 | 0.000 | 0.200 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -3.600 |
| \\(X_{8}\\) | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{2}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.600 | 0.000 | -0.200 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.600 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | 0.000 | -1.000 | 0.000 | 0.000 | 2.600 | 0.000 | 0.800 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 7.600 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.400 | 0.000 | 0.200 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.400 |
| \\(X_{15}\\) | 0.000 | 0.000 | 0.000 | 0.000 | -1.600 | 0.000 | 1.200 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 10.400 |
| \\(X_{16}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.400 | 0.000 | 0.200 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 2.400 |
| \\(X_{17}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.600 | 0.000 | -0.200 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.600 |
| \\(X_{18}\\) | 0.000 | -1.000 | 0.000 | 0.000 | 2.600 | 0.000 | 0.800 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 7.600 |
| \\(X_{19}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 |


Extended Tableau

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS | \\(X_{14}\\)
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 | -1.600 | 0.000 | 0.200 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -3.600 | 0.200 |
| \\(X_{8}\\) | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{2}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.600 | 0.000 | -0.200 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.600 | -0.200 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | 0.000 | -1.000 | 0.000 | 0.000 | 2.600 | 0.000 | 0.800 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 7.600 | 0.800 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 | 0.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.400 | 0.000 | 0.200 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.400 | 0.200 |
| \\(X_{15}\\) | 0.000 | 0.000 | 0.000 | 0.000 | -1.600 | 0.000 | 1.200 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 10.400 | 1.200 |
| \\(X_{16}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.400 | 0.000 | 0.200 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 2.400 | 0.200 |
| \\(X_{17}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.600 | 0.000 | -0.200 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.600 | -0.200 |
| \\(X_{18}\\) | 0.000 | -1.000 | 0.000 | 0.000 | 2.600 | 0.000 | 0.800 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 7.600 | 0.800 |
| \\(X_{19}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 | 0.000 |


\\(Ĉ_J = \ \\)array([ 0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1.6,  0.2])


\\(J = {[1, 3, 4, 6, 9, 11, 12, 14]}\\)

### Pivot Operations

Variable to enter: \\(X_{14}\\)
Variable to leave: \\(X_{8}\\)
#### Row Operations:

1. \\(R_{X_{8}} \leftarrow \frac{R_{X_{8}}}{0.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 0.200R_{X_{8}}\\)
2. \\(R_{X_{2}} \leftarrow R_{X_{2}} + 0.200R_{X_{8}}\\)
3. \\(R_{X_{10}} \leftarrow R_{X_{10}} - 0.000R_{X_{8}}\\)
4. \\(R_{X_{5}} \leftarrow R_{X_{5}} - 0.800R_{X_{8}}\\)
5. \\(R_{X_{13}} \leftarrow R_{X_{13}} - 0.000R_{X_{8}}\\)
6. \\(R_{X_{7}} \leftarrow R_{X_{7}} - 0.000R_{X_{8}}\\)
7. \\(R_{X_{0}} \leftarrow R_{X_{0}} - 0.200R_{X_{8}}\\)
8. \\(R_{X_{15}} \leftarrow R_{X_{15}} - 1.200R_{X_{8}}\\)
9. \\(R_{X_{16}} \leftarrow R_{X_{16}} - 0.200R_{X_{8}}\\)
10. \\(R_{X_{17}} \leftarrow R_{X_{17}} + 0.200R_{X_{8}}\\)
11. \\(R_{X_{18}} \leftarrow R_{X_{18}} - 0.800R_{X_{8}}\\)
12. \\(R_{X_{19}} \leftarrow R_{X_{19}} - 0.000R_{X_{8}}\\)


Computed Tableau:

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | -2401919801264264.500 | 2401919801264264.500 | 0.000 | 0.000 | -1.600 | 0.000 | -0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -4.800 |
| \\(X_{14}\\) | 12009599006321322.000 | -12009599006321322.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 6.000 |
| \\(X_{2}\\) | 2401919801264263.500 | -2401919801264263.500 | 0.000 | 0.000 | 0.600 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.800 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | -9607679205057060.000 | 9607679205057060.000 | 0.000 | 0.000 | 2.600 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.800 |
| \\(X_{13}\\) | -1.333 | 1.333 | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{7}\\) | -1.333 | 1.333 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{0}\\) | -2401919801264264.500 | 2401919801264264.500 | 0.000 | 0.000 | 0.400 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.200 |
| \\(X_{15}\\) | -14411518807585584.000 | 14411518807585584.000 | 0.000 | 0.000 | -1.600 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.200 |
| \\(X_{16}\\) | -2401919801264265.000 | 2401919801264265.000 | 0.000 | 0.000 | 0.400 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 1.200 |
| \\(X_{17}\\) | 2401919801264264.000 | -2401919801264264.000 | 0.000 | 0.000 | 0.600 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 1.800 |
| \\(X_{18}\\) | -9607679205057060.000 | 9607679205057060.000 | 0.000 | 0.000 | 2.600 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 2.800 |
| \\(X_{19}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 |


## Iteration 1

Starting Tableau:

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | -2401919801264264.500 | 2401919801264264.500 | 0.000 | 0.000 | -1.600 | 0.000 | -0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -4.800 |
| \\(X_{14}\\) | 12009599006321322.000 | -12009599006321322.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 6.000 |
| \\(X_{2}\\) | 2401919801264263.500 | -2401919801264263.500 | 0.000 | 0.000 | 0.600 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.800 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | -9607679205057060.000 | 9607679205057060.000 | 0.000 | 0.000 | 2.600 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.800 |
| \\(X_{13}\\) | -1.333 | 1.333 | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{7}\\) | -1.333 | 1.333 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{0}\\) | -2401919801264264.500 | 2401919801264264.500 | 0.000 | 0.000 | 0.400 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.200 |
| \\(X_{15}\\) | -14411518807585584.000 | 14411518807585584.000 | 0.000 | 0.000 | -1.600 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.200 |
| \\(X_{16}\\) | -2401919801264265.000 | 2401919801264265.000 | 0.000 | 0.000 | 0.400 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 1.200 |
| \\(X_{17}\\) | 2401919801264264.000 | -2401919801264264.000 | 0.000 | 0.000 | 0.600 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 1.800 |
| \\(X_{18}\\) | -9607679205057060.000 | 9607679205057060.000 | 0.000 | 0.000 | 2.600 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 2.800 |
| \\(X_{19}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 |


Extended Tableau

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS | \\(X_{1}\\)
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | -2401919801264264.500 | 2401919801264264.500 | 0.000 | 0.000 | -1.600 | 0.000 | -0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -4.800 | 0.600 |
| \\(X_{14}\\) | 12009599006321322.000 | -12009599006321322.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 6.000 | -3.000 |
| \\(X_{2}\\) | 2401919801264263.500 | -2401919801264263.500 | 0.000 | 0.000 | 0.600 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.800 | -0.600 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | -9607679205057060.000 | 9607679205057060.000 | 0.000 | 0.000 | 2.600 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.800 | -2.600 |
| \\(X_{13}\\) | -1.333 | 1.333 | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{7}\\) | -1.333 | 1.333 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 | 0.000 |
| \\(X_{0}\\) | -2401919801264264.500 | 2401919801264264.500 | 0.000 | 0.000 | 0.400 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.200 | -0.400 |
| \\(X_{15}\\) | -14411518807585584.000 | 14411518807585584.000 | 0.000 | 0.000 | -1.600 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.200 | 3.600 |
| \\(X_{16}\\) | -2401919801264265.000 | 2401919801264265.000 | 0.000 | 0.000 | 0.400 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 1.200 | 0.600 |
| \\(X_{17}\\) | 2401919801264264.000 | -2401919801264264.000 | 0.000 | 0.000 | 0.600 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 1.800 | -0.600 |
| \\(X_{18}\\) | -9607679205057060.000 | 9607679205057060.000 | 0.000 | 0.000 | 2.600 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 2.800 | -2.600 |
| \\(X_{19}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 | 0.000 |


\\(Ĉ_J = \ \\)array([ 6.0000000e-01, -4.0000000e-01,  0.0000000e+00,  0.0000000e+00,
       -2.4019198e+15,  0.0000000e+00, -1.6000000e+00, -2.4019198e+15])


\\(J = {[1, 3, 4, 6, 9, 11, 12, 8]}\\)

### Pivot Operations

Variable to enter: \\(X_{1}\\)
Variable to leave: \\(X_{15}\\)
#### Row Operations:

1. \\(R_{X_{15}} \leftarrow \frac{R_{X_{15}}}{3.600}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 0.600R_{X_{15}}\\)
2. \\(R_{X_{14}} \leftarrow R_{X_{14}} + 3.000R_{X_{15}}\\)
3. \\(R_{X_{2}} \leftarrow R_{X_{2}} + 0.600R_{X_{15}}\\)
4. \\(R_{X_{10}} \leftarrow R_{X_{10}} - 0.000R_{X_{15}}\\)
5. \\(R_{X_{5}} \leftarrow R_{X_{5}} + 2.600R_{X_{15}}\\)
6. \\(R_{X_{13}} \leftarrow R_{X_{13}} - 0.000R_{X_{15}}\\)
7. \\(R_{X_{7}} \leftarrow R_{X_{7}} - 0.000R_{X_{15}}\\)
8. \\(R_{X_{0}} \leftarrow R_{X_{0}} + 0.400R_{X_{15}}\\)
9. \\(R_{X_{16}} \leftarrow R_{X_{16}} - 0.600R_{X_{15}}\\)
10. \\(R_{X_{17}} \leftarrow R_{X_{17}} + 0.600R_{X_{15}}\\)
11. \\(R_{X_{18}} \leftarrow R_{X_{18}} + 2.600R_{X_{15}}\\)
12. \\(R_{X_{19}} \leftarrow R_{X_{19}} - 0.000R_{X_{15}}\\)


Computed Tableau:

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | -0.500 | 0.500 | 0.000 | 0.000 | -1.333 | 0.000 | -0.000 | -0.167 | 0.000 | 0.000 | 0.000 | 0.000 | -5.333 |
| \\(X_{14}\\) | 2.000 | -2.000 | 0.000 | 0.000 | -1.333 | 0.000 | 1.000 | 0.833 | 0.000 | 0.000 | 0.000 | 0.000 | 8.667 |
| \\(X_{2}\\) | -1.000 | 1.000 | 0.000 | 0.000 | 0.333 | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 0.000 | 2.333 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | -20015998343868872.000 | 20015998343868872.000 | 0.000 | 0.000 | 1.444 | 0.000 | 0.000 | 0.722 | 0.000 | 0.000 | 0.000 | 0.000 | 5.111 |
| \\(X_{13}\\) | -1.152 | 1.152 | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{7}\\) | -1.152 | 1.152 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{0}\\) | -4003199668773774.000 | 4003199668773774.000 | 0.000 | 0.000 | 0.222 | 0.000 | 0.000 | 0.111 | 0.000 | 0.000 | 0.000 | 0.000 | 1.556 |
| \\(X_{1}\\) | -4003199668773773.500 | 4003199668773773.500 | 0.000 | 0.000 | -0.444 | 0.000 | 0.000 | 0.278 | 0.000 | 0.000 | 0.000 | 0.000 | 0.889 |
| \\(X_{16}\\) | -1.000 | 1.000 | 0.000 | 0.000 | 0.667 | 0.000 | 0.000 | -0.167 | 1.000 | 0.000 | 0.000 | 0.000 | 0.667 |
| \\(X_{17}\\) | 0.500 | -0.500 | 0.000 | 0.000 | 0.333 | 0.000 | 0.000 | 0.167 | 0.000 | 1.000 | 0.000 | 0.000 | 2.333 |
| \\(X_{18}\\) | -20015998343868872.000 | 20015998343868872.000 | 0.000 | 0.000 | 1.444 | 0.000 | 0.000 | 0.722 | 0.000 | 0.000 | 1.000 | 0.000 | 5.111 |
| \\(X_{19}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 |


## Iteration 2

Starting Tableau:

|   |  |   |   |   |   |   | \\(A_I^{-1}\\) |   |   |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Z | -0.500 | 0.500 | 0.000 | 0.000 | -1.333 | 0.000 | -0.000 | -0.167 | 0.000 | 0.000 | 0.000 | 0.000 | -5.333 |
| \\(X_{14}\\) | 2.000 | -2.000 | 0.000 | 0.000 | -1.333 | 0.000 | 1.000 | 0.833 | 0.000 | 0.000 | 0.000 | 0.000 | 8.667 |
| \\(X_{2}\\) | -1.000 | 1.000 | 0.000 | 0.000 | 0.333 | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 0.000 | 2.333 |
| \\(X_{10}\\) | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{5}\\) | -20015998343868872.000 | 20015998343868872.000 | 0.000 | 0.000 | 1.444 | 0.000 | 0.000 | 0.722 | 0.000 | 0.000 | 0.000 | 0.000 | 5.111 |
| \\(X_{13}\\) | -1.152 | 1.152 | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{7}\\) | -1.152 | 1.152 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{0}\\) | -4003199668773774.000 | 4003199668773774.000 | 0.000 | 0.000 | 0.222 | 0.000 | 0.000 | 0.111 | 0.000 | 0.000 | 0.000 | 0.000 | 1.556 |
| \\(X_{1}\\) | -4003199668773773.500 | 4003199668773773.500 | 0.000 | 0.000 | -0.444 | 0.000 | 0.000 | 0.278 | 0.000 | 0.000 | 0.000 | 0.000 | 0.889 |
| \\(X_{16}\\) | -1.000 | 1.000 | 0.000 | 0.000 | 0.667 | 0.000 | 0.000 | -0.167 | 1.000 | 0.000 | 0.000 | 0.000 | 0.667 |
| \\(X_{17}\\) | 0.500 | -0.500 | 0.000 | 0.000 | 0.333 | 0.000 | 0.000 | 0.167 | 0.000 | 1.000 | 0.000 | 0.000 | 2.333 |
| \\(X_{18}\\) | -20015998343868872.000 | 20015998343868872.000 | 0.000 | 0.000 | 1.444 | 0.000 | 0.000 | 0.722 | 0.000 | 0.000 | 1.000 | 0.000 | 5.111 |
| \\(X_{19}\\) | 0.000 | 0.000 | 0.000 | -1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 2.000 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
5.000 & -5.000 & 1.000 & -1.000 & 1.000 & -1.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
5.000 & -5.000 & 1.000 & -1.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
1.000 & -1.000 & 1.000 & -1.000 & 0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
1.000 & -1.000 & 1.000 & -1.000 & 0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
1.000 & -1.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
1.000 & -1.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
3.000 & -3.000 & -2.000 & 2.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
-2.000 & 2.000 & 4.000 & -4.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
-1.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 \\
0.000 & 0.000 & -1.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 \\
0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 \\
0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
5.000\\
5.000\\
1.000\\
1.000\\
3.000\\
3.000\\
6.000\\
8.000\\
0.000\\
0.000\\
0.000\\
0.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
-1.000 & 1.000 & -2.000 & 2.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}\\
X_{7}\\
X_{8}\\
X_{9}\\
X_{10}\\
X_{11}\\
X_{12}\\
X_{13}\\
X_{14}\\
X_{15}\\
X_{16}\\
X_{17}\\
X_{18}\\
X_{19}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal multiple solutions

Optimal Solution: \\(X^{*} = ['1.556', '0.889', '2.333', '0.000', '0.000', '5.111', '0.000', '2.000', '0.000', '0.000', '0.000', '0.000', '0.000', '0.000', '8.667', '0.000', '0.667', '2.333', '5.111', '2.000']\\)

Optimal Value: \\(Z^{*} = -5.333\\)

Optimal Basis: \\([14, 2, 10, 5, 13, 7, 0, 1, 16, 17, 18, 19]\\)

Final non-basic variables set J: \\([15, 3, 4, 6, 9, 11, 12, 8]\\)

\\(\hat{C}_{J}\\): \\([-0.166667, 0.0, 0.0, 0.0, -0.5, 0.0, -1.333333, -0.5]\\)

Number of Iterations: 3



## letra g)

In [142]:
C = np.array([-3, 3, -4, 4, 0, 0, 0, 0, 0, 0])
B = np.array([
    2,
    3,
    6,
    8,
    0,
    0
])
A = np.array([
    [-1, 1, -2, 2, -1, 0, 0, 0, 0, 0],
    [-2, 2, 1, -1, 0, -1, 0, 0, 0, 0],
    [-3, 3, -1, 1, 0, 0, -1, 0, 0, 0],
    [-1, 1, 3, -3, 0, 0, 0, -1, 0, 0],
    [-1, 1, 0,  0, 0, 0, 0,  0, 1, 0],
    [ 0, 0, -1, 1, 0, 0, 0,  0, 0, 1]
])

# primeira fase, variáveis artificiais

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = [10, 11, 12, 13, 8, 9]

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I, cycle_proof=True)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | 1.000 | 1.000 | 0.000 | 0.000 | 19.000 |
| \\(X_{10}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{11}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 6.000 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 8.000 |
| \\(X_{8}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{9}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS | \\(X_{1}\\)
|---|---|---|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | 1.000 | 1.000 | 0.000 | 0.000 | 19.000 | 7.000 |
| \\(X_{10}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 | 1.000 |
| \\(X_{11}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.000 | 2.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 6.000 | 3.000 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 8.000 | 1.000 |
| \\(X_{8}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{9}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |


\\(Ĉ_J = \ \\)array([-7.,  7.,  1., -1., -1., -1., -1., -1.])


\\(J = {[0, 1, 2, 3, 4, 5, 6, 7]}\\)

### Pivot Operations

Variable to enter: \\(X_{1}\\)
Variable to leave: \\(X_{8}\\)
#### Row Operations:

1. \\(R_{X_{8}} \leftarrow \frac{R_{X_{8}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 7.000R_{X_{8}}\\)
2. \\(R_{X_{10}} \leftarrow R_{X_{10}} - 1.000R_{X_{8}}\\)
3. \\(R_{X_{11}} \leftarrow R_{X_{11}} - 2.000R_{X_{8}}\\)
4. \\(R_{X_{12}} \leftarrow R_{X_{12}} - 3.000R_{X_{8}}\\)
5. \\(R_{X_{13}} \leftarrow R_{X_{13}} - 1.000R_{X_{8}}\\)
6. \\(R_{X_{9}} \leftarrow R_{X_{9}} - 0.000R_{X_{8}}\\)


Computed Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | 1.000 | 1.000 | -7.000 | 0.000 | 19.000 |
| \\(X_{10}\\) | 1.000 | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 2.000 |
| \\(X_{11}\\) | 0.000 | 1.000 | 0.000 | 0.000 | -2.000 | 0.000 | 3.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 1.000 | 0.000 | -3.000 | 0.000 | 6.000 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 8.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{9}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Iteration 1

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | 1.000 | 1.000 | -7.000 | 0.000 | 19.000 |
| \\(X_{10}\\) | 1.000 | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 2.000 |
| \\(X_{11}\\) | 0.000 | 1.000 | 0.000 | 0.000 | -2.000 | 0.000 | 3.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 1.000 | 0.000 | -3.000 | 0.000 | 6.000 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 8.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{9}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS | \\(X_{2}\\)
|---|---|---|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | 1.000 | 1.000 | -7.000 | 0.000 | 19.000 | 1.000 |
| \\(X_{10}\\) | 1.000 | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 2.000 | -2.000 |
| \\(X_{11}\\) | 0.000 | 1.000 | 0.000 | 0.000 | -2.000 | 0.000 | 3.000 | 1.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 1.000 | 0.000 | -3.000 | 0.000 | 6.000 | -1.000 |
| \\(X_{13}\\) | 0.000 | 0.000 | 0.000 | 1.000 | -1.000 | 0.000 | 8.000 | 3.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{9}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | -1.000 |


\\(Ĉ_J = \ \\)array([ 0., -7.,  1., -1., -1., -1., -1., -1.])


\\(J = {[0, 8, 2, 3, 4, 5, 6, 7]}\\)

### Pivot Operations

Variable to enter: \\(X_{2}\\)
Variable to leave: \\(X_{13}\\)
#### Row Operations:

1. \\(R_{X_{13}} \leftarrow \frac{R_{X_{13}}}{3.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 1.000R_{X_{13}}\\)
2. \\(R_{X_{10}} \leftarrow R_{X_{10}} + 2.000R_{X_{13}}\\)
3. \\(R_{X_{11}} \leftarrow R_{X_{11}} - 1.000R_{X_{13}}\\)
4. \\(R_{X_{12}} \leftarrow R_{X_{12}} + 1.000R_{X_{13}}\\)
5. \\(R_{X_{1}} \leftarrow R_{X_{1}} - 0.000R_{X_{13}}\\)
6. \\(R_{X_{9}} \leftarrow R_{X_{9}} + 1.000R_{X_{13}}\\)


Computed Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | 1.000 | 0.667 | -6.667 | 0.000 | 16.333 |
| \\(X_{10}\\) | 1.000 | 0.000 | 0.000 | 0.667 | -1.667 | 0.000 | 7.333 |
| \\(X_{11}\\) | 0.000 | 1.000 | 0.000 | -0.333 | -1.667 | 0.000 | 0.333 |
| \\(X_{12}\\) | 0.000 | 0.000 | 1.000 | 0.333 | -3.333 | 0.000 | 8.667 |
| \\(X_{2}\\) | 0.000 | 0.000 | 0.000 | 0.333 | -0.333 | 0.000 | 2.667 |
| \\(X_{1}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{9}\\) | 0.000 | 0.000 | 0.000 | 0.333 | -0.333 | 1.000 | 2.667 |


## Iteration 2

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | 1.000 | 0.667 | -6.667 | 0.000 | 16.333 |
| \\(X_{10}\\) | 1.000 | 0.000 | 0.000 | 0.667 | -1.667 | 0.000 | 7.333 |
| \\(X_{11}\\) | 0.000 | 1.000 | 0.000 | -0.333 | -1.667 | 0.000 | 0.333 |
| \\(X_{12}\\) | 0.000 | 0.000 | 1.000 | 0.333 | -3.333 | 0.000 | 8.667 |
| \\(X_{2}\\) | 0.000 | 0.000 | 0.000 | 0.333 | -0.333 | 0.000 | 2.667 |
| \\(X_{1}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{9}\\) | 0.000 | 0.000 | 0.000 | 0.333 | -0.333 | 1.000 | 2.667 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
-1.000 & 1.000 & -2.000 & 2.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 \\
-2.000 & 2.000 & 1.000 & -1.000 & 0.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 \\
-3.000 & 3.000 & -1.000 & 1.000 & 0.000 & 0.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 \\
-1.000 & 1.000 & 3.000 & -3.000 & 0.000 & 0.000 & 0.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 \\
-1.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
0.000 & 0.000 & -1.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
2.000\\
3.000\\
6.000\\
8.000\\
0.000\\
0.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 1.000 & 1.000 & 1.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}\\
X_{7}\\
X_{8}\\
X_{9}\\
X_{10}\\
X_{11}\\
X_{12}\\
X_{13}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal multiple solutions

Optimal Solution: \\(X^{*} = ['0.000', '0.000', '2.667', '0.000', '0.000', '0.000', '0.000', '0.000', '0.000', '2.667', '7.333', '0.333', '8.667', '0.000']\\)

Optimal Value: \\(Z^{*} = 16.333\\)

Optimal Basis: \\([10, 11, 12, 2, 1, 9]\\)

Final non-basic variables set J: \\([0, 8, 13, 3, 4, 5, 6, 7]\\)

\\(\hat{C}_{J}\\): \\([0.0, -6.666667, -0.333333, 0.0, -1.0, -1.0, -1.0, -0.666667]\\)

Number of Iterations: 3



In [143]:
# problema primal

C = -np.array([2, 3, 6, 8, 0, 0])
B = np.array([
    3,
    4
])
A = np.array([
    [1, 2, 3, 1, -1, 0],
    [2,-1, 1,-3, 0, -1]
])

# primeira fase, variáveis artificiais

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = [6, 7]

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I, cycle_proof=True)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | 1.000 | 1.000 | 7.000 |
| \\(X_{6}\\) | 1.000 | 0.000 | 3.000 |
| \\(X_{7}\\) | 0.000 | 1.000 | 4.000 |


Extended Tableau

|   |  | \\(A_I^{-1}\\) | RHS | \\(X_{2}\\)
|---|---|---|---|---|
| Z | 1.000 | 1.000 | 7.000 | 4.000 |
| \\(X_{6}\\) | 1.000 | 0.000 | 3.000 | 3.000 |
| \\(X_{7}\\) | 0.000 | 1.000 | 4.000 | 1.000 |


\\(Ĉ_J = \ \\)array([ 3.,  1.,  4., -2., -1., -1.])


\\(J = {[0, 1, 2, 3, 4, 5]}\\)

### Pivot Operations

Variable to enter: \\(X_{2}\\)
Variable to leave: \\(X_{6}\\)
#### Row Operations:

1. \\(R_{X_{6}} \leftarrow \frac{R_{X_{6}}}{3.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 4.000R_{X_{6}}\\)
2. \\(R_{X_{7}} \leftarrow R_{X_{7}} - 1.000R_{X_{6}}\\)


Computed Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | -0.333 | 1.000 | 3.000 |
| \\(X_{2}\\) | 0.333 | 0.000 | 1.000 |
| \\(X_{7}\\) | -0.333 | 1.000 | 3.000 |


## Iteration 1

Starting Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | -0.333 | 1.000 | 3.000 |
| \\(X_{2}\\) | 0.333 | 0.000 | 1.000 |
| \\(X_{7}\\) | -0.333 | 1.000 | 3.000 |


Extended Tableau

|   |  | \\(A_I^{-1}\\) | RHS | \\(X_{0}\\)
|---|---|---|---|---|
| Z | -0.333 | 1.000 | 3.000 | 1.667 |
| \\(X_{2}\\) | 0.333 | 0.000 | 1.000 | 0.333 |
| \\(X_{7}\\) | -0.333 | 1.000 | 3.000 | 1.667 |


\\(Ĉ_J = \ \\)array([ 1.666667, -1.666667, -1.333333, -3.333333,  0.333333, -1.      ])


\\(J = {[0, 1, 6, 3, 4, 5]}\\)

### Pivot Operations

Variable to enter: \\(X_{0}\\)
Variable to leave: \\(X_{7}\\)
#### Row Operations:

1. \\(R_{X_{7}} \leftarrow \frac{R_{X_{7}}}{1.667}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 1.667R_{X_{7}}\\)
2. \\(R_{X_{2}} \leftarrow R_{X_{2}} - 0.333R_{X_{7}}\\)


Computed Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | 0.000 | -0.000 | -0.000 |
| \\(X_{2}\\) | 0.400 | -0.200 | 0.400 |
| \\(X_{0}\\) | -0.200 | 0.600 | 1.800 |


## Iteration 2

Starting Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | 0.000 | -0.000 | -0.000 |
| \\(X_{2}\\) | 0.400 | -0.200 | 0.400 |
| \\(X_{0}\\) | -0.200 | 0.600 | 1.800 |


Extended Tableau

|   |  | \\(A_I^{-1}\\) | RHS | \\(X_{3}\\)
|---|---|---|---|---|
| Z | 0.000 | -0.000 | -0.000 | 0.000 |
| \\(X_{2}\\) | 0.400 | -0.200 | 0.400 | 1.000 |
| \\(X_{0}\\) | -0.200 | 0.600 | 1.800 | -2.000 |


\\(Ĉ_J = \ \\)array([-1.e+00,  0.e+00, -1.e+00,  1.e-06, -0.e+00,  0.e+00])


\\(J = {[7, 1, 6, 3, 4, 5]}\\)

### Pivot Operations

Variable to enter: \\(X_{3}\\)
Variable to leave: \\(X_{2}\\)
#### Row Operations:

1. \\(R_{X_{2}} \leftarrow \frac{R_{X_{2}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 0.000R_{X_{2}}\\)
2. \\(R_{X_{0}} \leftarrow R_{X_{0}} + 2.000R_{X_{2}}\\)


Computed Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | -0.000 | 0.000 | -0.000 |
| \\(X_{3}\\) | 0.400 | -0.200 | 0.400 |
| \\(X_{0}\\) | 0.600 | 0.200 | 2.600 |


## Iteration 3

Starting Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | -0.000 | 0.000 | -0.000 |
| \\(X_{3}\\) | 0.400 | -0.200 | 0.400 |
| \\(X_{0}\\) | 0.600 | 0.200 | 2.600 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
1.000 & 2.000 & 3.000 & 1.000 & -1.000 & 0.000 & 1.000 & 0.000 \\
2.000 & -1.000 & 1.000 & -3.000 & 0.000 & -1.000 & 0.000 & 1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
3.000\\
4.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 1.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}\\
X_{7}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal multiple solutions

Optimal Solution: \\(X^{*} = ['2.600', '0.000', '0.000', '0.400', '0.000', '0.000', '0.000', '0.000']\\)

Optimal Value: \\(Z^{*} = -0.000\\)

Optimal Basis: \\([3, 0]\\)

Final non-basic variables set J: \\([7, 1, 6, 2, 4, 5]\\)

\\(\hat{C}_{J}\\): \\([-1.0, -1e-06, -1.0, -1e-06, 0.0, -0.0]\\)

Number of Iterations: 4



In [144]:
# segunda fase
iterations = list(solution.keys()).pop()
I = solution[iterations]['I']
# resolvendo problema original com o simplex tableau revisado com BIF

solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | -4.400 | 1.200 | -8.400 |
| \\(X_{3}\\) | 0.400 | -0.200 | 0.400 |
| \\(X_{0}\\) | 0.600 | 0.200 | 2.600 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
1.000 & 2.000 & 3.000 & 1.000 & -1.000 & 0.000 \\
2.000 & -1.000 & 1.000 & -3.000 & 0.000 & -1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
3.000\\
4.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
-2.000 & -3.000 & -6.000 & -8.000 & 0.000 & 0.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Unbounded solution

Optimal Solution: \\(X^{*} = ['2.600', '0.000', '0.000', '0.400', '0.000', '0.000']\\)

Optimal Value: \\(Z^{*} = -8.400\\)

Optimal Basis: \\([3, 0]\\)

Final non-basic variables set J: \\([1, 2, 4, 5]\\)

\\(\hat{C}_{J}\\): \\([-7.0, -6.0, 4.4, -1.2]\\)

Number of Iterations: 1



## letra h)

In [145]:
C = - np.array([
    5, -5, 2, -2, 0, 0, 0, 0, 0
])
B = np.array([
    3,
    4,
    9,
    0,
    0
])
A = np.array([
    [ 1, -1,  0,  0,  1,  0,  0,  0,  0],
    [ 0,  0,  1, -1,  0,  1,  0,  0,  0],
    [ 1, -1,  2, -2,  0,  0,  1,  0,  0],
    [-1,  1,  0,  0,  0,  0,  0,  1,  0],
    [ 0,  0, -1,  1,  0,  0,  0,  0,  1]
])

# Não será necessário adicionar variáveis artificiais, pois, não existem
# variáveis de excesso no problema dual

I = list(range(4, 9))
solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{4}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.000 |
| \\(X_{5}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 4.000 |
| \\(X_{6}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 9.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{8}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS | \\(X_{0}\\)
|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 5.000 |
| \\(X_{4}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.000 | 1.000 |
| \\(X_{5}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 4.000 | 0.000 |
| \\(X_{6}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 9.000 | 1.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | -1.000 |
| \\(X_{8}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |


\\(Ĉ_J = \ \\)array([ 5., -5.,  2., -2.])


\\(J = {[0, 1, 2, 3]}\\)

### Pivot Operations

Variable to enter: \\(X_{0}\\)
Variable to leave: \\(X_{4}\\)
#### Row Operations:

1. \\(R_{X_{4}} \leftarrow \frac{R_{X_{4}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 5.000R_{X_{4}}\\)
2. \\(R_{X_{5}} \leftarrow R_{X_{5}} - 0.000R_{X_{4}}\\)
3. \\(R_{X_{6}} \leftarrow R_{X_{6}} - 1.000R_{X_{4}}\\)
4. \\(R_{X_{7}} \leftarrow R_{X_{7}} + 1.000R_{X_{4}}\\)
5. \\(R_{X_{8}} \leftarrow R_{X_{8}} - 0.000R_{X_{4}}\\)


Computed Tableau:

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|
| Z | -5.000 | 0.000 | 0.000 | 0.000 | 0.000 | -15.000 |
| \\(X_{0}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.000 |
| \\(X_{5}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 4.000 |
| \\(X_{6}\\) | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 6.000 |
| \\(X_{7}\\) | 1.000 | 0.000 | 0.000 | 1.000 | 0.000 | 3.000 |
| \\(X_{8}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Iteration 1

Starting Tableau:

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|
| Z | -5.000 | 0.000 | 0.000 | 0.000 | 0.000 | -15.000 |
| \\(X_{0}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.000 |
| \\(X_{5}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 4.000 |
| \\(X_{6}\\) | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 6.000 |
| \\(X_{7}\\) | 1.000 | 0.000 | 0.000 | 1.000 | 0.000 | 3.000 |
| \\(X_{8}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS | \\(X_{2}\\)
|---|---|---|---|---|---|---|---|
| Z | -5.000 | 0.000 | 0.000 | 0.000 | 0.000 | -15.000 | 2.000 |
| \\(X_{0}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.000 | 0.000 |
| \\(X_{5}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 4.000 | 1.000 |
| \\(X_{6}\\) | -1.000 | 0.000 | 1.000 | 0.000 | 0.000 | 6.000 | 2.000 |
| \\(X_{7}\\) | 1.000 | 0.000 | 0.000 | 1.000 | 0.000 | 3.000 | 0.000 |
| \\(X_{8}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | -1.000 |


\\(Ĉ_J = \ \\)array([-5.,  0.,  2., -2.])


\\(J = {[4, 1, 2, 3]}\\)

### Pivot Operations

Variable to enter: \\(X_{2}\\)
Variable to leave: \\(X_{6}\\)
#### Row Operations:

1. \\(R_{X_{6}} \leftarrow \frac{R_{X_{6}}}{2.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 2.000R_{X_{6}}\\)
2. \\(R_{X_{0}} \leftarrow R_{X_{0}} - 0.000R_{X_{6}}\\)
3. \\(R_{X_{5}} \leftarrow R_{X_{5}} - 1.000R_{X_{6}}\\)
4. \\(R_{X_{7}} \leftarrow R_{X_{7}} - 0.000R_{X_{6}}\\)
5. \\(R_{X_{8}} \leftarrow R_{X_{8}} + 1.000R_{X_{6}}\\)


Computed Tableau:

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|
| Z | -4.000 | 0.000 | -1.000 | 0.000 | 0.000 | -21.000 |
| \\(X_{0}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.000 |
| \\(X_{5}\\) | 0.500 | 1.000 | -0.500 | 0.000 | 0.000 | 1.000 |
| \\(X_{2}\\) | -0.500 | 0.000 | 0.500 | 0.000 | 0.000 | 3.000 |
| \\(X_{7}\\) | 1.000 | 0.000 | 0.000 | 1.000 | 0.000 | 3.000 |
| \\(X_{8}\\) | -0.500 | 0.000 | 0.500 | 0.000 | 1.000 | 3.000 |


## Iteration 2

Starting Tableau:

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|
| Z | -4.000 | 0.000 | -1.000 | 0.000 | 0.000 | -21.000 |
| \\(X_{0}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.000 |
| \\(X_{5}\\) | 0.500 | 1.000 | -0.500 | 0.000 | 0.000 | 1.000 |
| \\(X_{2}\\) | -0.500 | 0.000 | 0.500 | 0.000 | 0.000 | 3.000 |
| \\(X_{7}\\) | 1.000 | 0.000 | 0.000 | 1.000 | 0.000 | 3.000 |
| \\(X_{8}\\) | -0.500 | 0.000 | 0.500 | 0.000 | 1.000 | 3.000 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
1.000 & -1.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 \\
1.000 & -1.000 & 2.000 & -2.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 \\
-1.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 \\
0.000 & 0.000 & -1.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
3.000\\
4.000\\
9.000\\
0.000\\
0.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
-5.000 & 5.000 & -2.000 & 2.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}\\
X_{7}\\
X_{8}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal multiple solutions

Optimal Solution: \\(X^{*} = ['3.000', '0.000', '3.000', '0.000', '0.000', '1.000', '0.000', '3.000', '3.000']\\)

Optimal Value: \\(Z^{*} = -21.000\\)

Optimal Basis: \\([0, 5, 2, 7, 8]\\)

Final non-basic variables set J: \\([4, 1, 6, 3]\\)

\\(\hat{C}_{J}\\): \\([-4.0, 0.0, -1.0, 0.0]\\)

Number of Iterations: 3



### solver scipy linprog

In [146]:
result = linprog(C, A_eq = A, b_eq = B, method='highs')
display(result)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -21.0
              x: [ 3.000e+00  0.000e+00  3.000e+00  0.000e+00  0.000e+00
                   1.000e+00  0.000e+00  3.000e+00  3.000e+00]
            nit: 1
          lower:  residual: [ 3.000e+00  0.000e+00  3.000e+00  0.000e+00
                              0.000e+00  1.000e+00  0.000e+00  3.000e+00
                              3.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              4.000e+00  0.000e+00  1.000e+00  0.000e+00
                              0.000e+00]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf        inf        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00  0.000e+00  0.0

## Número 2 - Dual

In [153]:
C = -np.array(
    [10, -10, 2, -2, 8, -8] + [0] * 7
)

slacks_and_excesses = np.eye(7)
excesses = [0, 1, 2, 3]
for i in excesses:
    slacks_and_excesses[i , i] = -1

A = np.array([
    [-2, 2, -2, 2, -2,  2],
    [-1, 1, -1, 1,  0,  0],
    [ 0, 0,  0, 0, -6,  6],
    [-3, 3,  0, 0, -1,  1],
    [1, -1,  0, 0,  0,  0],
    [0,  0,  1,-1,  0,  0],
    [0,  0,  0, 0,  1, -1]
])
A = np.hstack((A, slacks_and_excesses))
B = np.array([
    3,
    2,
    2,
    1,
    0,
    0,
    0
])

# primeira fase, variáveis artificiais

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = [13, 14, 15, 16, 10, 11, 12]

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I, cycle_proof=True)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | 1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 8.000 |
| \\(X_{13}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.000 |
| \\(X_{14}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{15}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{16}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{10}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS | \\(X_{5}\\)
|---|---|---|---|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | 1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 8.000 | 9.000 |
| \\(X_{13}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 3.000 | 2.000 |
| \\(X_{14}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 | 0.000 |
| \\(X_{15}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 | 6.000 |
| \\(X_{16}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 1.000 | 1.000 |
| \\(X_{10}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | -1.000 |


\\(Ĉ_J = \ \\)array([-6.,  6., -3.,  3., -9.,  9., -1., -1., -1., -1.])


\\(J = {[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}\\)

### Pivot Operations

Variable to enter: \\(X_{5}\\)
Variable to leave: \\(X_{15}\\)
#### Row Operations:

1. \\(R_{X_{15}} \leftarrow \frac{R_{X_{15}}}{6.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 9.000R_{X_{15}}\\)
2. \\(R_{X_{13}} \leftarrow R_{X_{13}} - 2.000R_{X_{15}}\\)
3. \\(R_{X_{14}} \leftarrow R_{X_{14}} - 0.000R_{X_{15}}\\)
4. \\(R_{X_{16}} \leftarrow R_{X_{16}} - 1.000R_{X_{15}}\\)
5. \\(R_{X_{10}} \leftarrow R_{X_{10}} - 0.000R_{X_{15}}\\)
6. \\(R_{X_{11}} \leftarrow R_{X_{11}} - 0.000R_{X_{15}}\\)
7. \\(R_{X_{12}} \leftarrow R_{X_{12}} + 1.000R_{X_{15}}\\)


Computed Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | -0.500 | 1.000 | 0.000 | 0.000 | 0.000 | 5.000 |
| \\(X_{13}\\) | 1.000 | 0.000 | -0.333 | 0.000 | 0.000 | 0.000 | 0.000 | 2.333 |
| \\(X_{14}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 0.000 | 0.333 |
| \\(X_{16}\\) | 0.000 | 0.000 | -0.167 | 1.000 | 0.000 | 0.000 | 0.000 | 0.667 |
| \\(X_{10}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 1.000 | 0.333 |


## Iteration 1

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | -0.500 | 1.000 | 0.000 | 0.000 | 0.000 | 5.000 |
| \\(X_{13}\\) | 1.000 | 0.000 | -0.333 | 0.000 | 0.000 | 0.000 | 0.000 | 2.333 |
| \\(X_{14}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 0.000 | 0.333 |
| \\(X_{16}\\) | 0.000 | 0.000 | -0.167 | 1.000 | 0.000 | 0.000 | 0.000 | 0.667 |
| \\(X_{10}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 1.000 | 0.333 |


Extended Tableau

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS | \\(X_{1}\\)
|---|---|---|---|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | -0.500 | 1.000 | 0.000 | 0.000 | 0.000 | 5.000 | 6.000 |
| \\(X_{13}\\) | 1.000 | 0.000 | -0.333 | 0.000 | 0.000 | 0.000 | 0.000 | 2.333 | 2.000 |
| \\(X_{14}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 2.000 | 1.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 0.000 | 0.333 | 0.000 |
| \\(X_{16}\\) | 0.000 | 0.000 | -0.167 | 1.000 | 0.000 | 0.000 | 0.000 | 0.667 | 3.000 |
| \\(X_{10}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | -1.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 1.000 | 0.333 | 0.000 |


\\(Ĉ_J = \ \\)array([-6. ,  6. , -3. ,  3. ,  0. , -1.5, -1. , -1. ,  0.5, -1. ])


\\(J = {[0, 1, 2, 3, 4, 15, 6, 7, 8, 9]}\\)

### Pivot Operations

Variable to enter: \\(X_{1}\\)
Variable to leave: \\(X_{16}\\)
#### Row Operations:

1. \\(R_{X_{16}} \leftarrow \frac{R_{X_{16}}}{3.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 6.000R_{X_{16}}\\)
2. \\(R_{X_{13}} \leftarrow R_{X_{13}} - 2.000R_{X_{16}}\\)
3. \\(R_{X_{14}} \leftarrow R_{X_{14}} - 1.000R_{X_{16}}\\)
4. \\(R_{X_{5}} \leftarrow R_{X_{5}} - 0.000R_{X_{16}}\\)
5. \\(R_{X_{10}} \leftarrow R_{X_{10}} + 1.000R_{X_{16}}\\)
6. \\(R_{X_{11}} \leftarrow R_{X_{11}} - 0.000R_{X_{16}}\\)
7. \\(R_{X_{12}} \leftarrow R_{X_{12}} - 0.000R_{X_{16}}\\)


Computed Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | -0.167 | -1.000 | 0.000 | 0.000 | 0.000 | 3.667 |
| \\(X_{13}\\) | 1.000 | 0.000 | -0.222 | -0.667 | 0.000 | 0.000 | 0.000 | 1.889 |
| \\(X_{14}\\) | 0.000 | 1.000 | 0.056 | -0.333 | 0.000 | 0.000 | 0.000 | 1.778 |
| \\(X_{5}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 0.000 | 0.333 |
| \\(X_{1}\\) | 0.000 | 0.000 | -0.056 | 0.333 | 0.000 | 0.000 | 0.000 | 0.222 |
| \\(X_{10}\\) | 0.000 | 0.000 | -0.056 | 0.333 | 1.000 | 0.000 | 0.000 | 0.222 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 1.000 | 0.333 |


## Iteration 2

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | -0.167 | -1.000 | 0.000 | 0.000 | 0.000 | 3.667 |
| \\(X_{13}\\) | 1.000 | 0.000 | -0.222 | -0.667 | 0.000 | 0.000 | 0.000 | 1.889 |
| \\(X_{14}\\) | 0.000 | 1.000 | 0.056 | -0.333 | 0.000 | 0.000 | 0.000 | 1.778 |
| \\(X_{5}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 0.000 | 0.333 |
| \\(X_{1}\\) | 0.000 | 0.000 | -0.056 | 0.333 | 0.000 | 0.000 | 0.000 | 0.222 |
| \\(X_{10}\\) | 0.000 | 0.000 | -0.056 | 0.333 | 1.000 | 0.000 | 0.000 | 0.222 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 1.000 | 0.333 |


Extended Tableau

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS | \\(X_{3}\\)
|---|---|---|---|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | -0.167 | -1.000 | 0.000 | 0.000 | 0.000 | 3.667 | 3.000 |
| \\(X_{13}\\) | 1.000 | 0.000 | -0.222 | -0.667 | 0.000 | 0.000 | 0.000 | 1.889 | 2.000 |
| \\(X_{14}\\) | 0.000 | 1.000 | 0.056 | -0.333 | 0.000 | 0.000 | 0.000 | 1.778 | 1.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 0.000 | 0.333 | 0.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | -0.056 | 0.333 | 0.000 | 0.000 | 0.000 | 0.222 | 0.000 |
| \\(X_{10}\\) | 0.000 | 0.000 | -0.056 | 0.333 | 1.000 | 0.000 | 0.000 | 0.222 | 0.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | -1.000 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 1.000 | 0.333 | 0.000 |


\\(Ĉ_J = \ \\)array([ 0.      , -2.      , -3.      ,  3.      ,  0.      , -1.166667,
       -1.      , -1.      ,  0.166667,  1.      ])


\\(J = {[0, 16, 2, 3, 4, 15, 6, 7, 8, 9]}\\)

### Pivot Operations

Variable to enter: \\(X_{3}\\)
Variable to leave: \\(X_{13}\\)
#### Row Operations:

1. \\(R_{X_{13}} \leftarrow \frac{R_{X_{13}}}{2.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 3.000R_{X_{13}}\\)
2. \\(R_{X_{14}} \leftarrow R_{X_{14}} - 1.000R_{X_{13}}\\)
3. \\(R_{X_{5}} \leftarrow R_{X_{5}} - 0.000R_{X_{13}}\\)
4. \\(R_{X_{1}} \leftarrow R_{X_{1}} - 0.000R_{X_{13}}\\)
5. \\(R_{X_{10}} \leftarrow R_{X_{10}} - 0.000R_{X_{13}}\\)
6. \\(R_{X_{11}} \leftarrow R_{X_{11}} + 1.000R_{X_{13}}\\)
7. \\(R_{X_{12}} \leftarrow R_{X_{12}} - 0.000R_{X_{13}}\\)


Computed Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | -0.500 | 1.000 | 0.167 | 0.000 | 0.000 | 0.000 | 0.000 | 0.833 |
| \\(X_{3}\\) | 0.500 | 0.000 | -0.111 | -0.333 | 0.000 | 0.000 | 0.000 | 0.944 |
| \\(X_{14}\\) | -0.500 | 1.000 | 0.167 | 0.000 | 0.000 | 0.000 | 0.000 | 0.833 |
| \\(X_{5}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 0.000 | 0.333 |
| \\(X_{1}\\) | 0.000 | 0.000 | -0.056 | 0.333 | 0.000 | 0.000 | 0.000 | 0.222 |
| \\(X_{10}\\) | 0.000 | 0.000 | -0.056 | 0.333 | 1.000 | 0.000 | 0.000 | 0.222 |
| \\(X_{11}\\) | 0.500 | 0.000 | -0.111 | -0.333 | 0.000 | 1.000 | 0.000 | 0.944 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 1.000 | 0.333 |


## Iteration 3

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | -0.500 | 1.000 | 0.167 | 0.000 | 0.000 | 0.000 | 0.000 | 0.833 |
| \\(X_{3}\\) | 0.500 | 0.000 | -0.111 | -0.333 | 0.000 | 0.000 | 0.000 | 0.944 |
| \\(X_{14}\\) | -0.500 | 1.000 | 0.167 | 0.000 | 0.000 | 0.000 | 0.000 | 0.833 |
| \\(X_{5}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 0.000 | 0.333 |
| \\(X_{1}\\) | 0.000 | 0.000 | -0.056 | 0.333 | 0.000 | 0.000 | 0.000 | 0.222 |
| \\(X_{10}\\) | 0.000 | 0.000 | -0.056 | 0.333 | 1.000 | 0.000 | 0.000 | 0.222 |
| \\(X_{11}\\) | 0.500 | 0.000 | -0.111 | -0.333 | 0.000 | 1.000 | 0.000 | 0.944 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 1.000 | 0.333 |


Extended Tableau

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS | \\(X_{6}\\)
|---|---|---|---|---|---|---|---|---|---|
| Z | -0.500 | 1.000 | 0.167 | 0.000 | 0.000 | 0.000 | 0.000 | 0.833 | 0.500 |
| \\(X_{3}\\) | 0.500 | 0.000 | -0.111 | -0.333 | 0.000 | 0.000 | 0.000 | 0.944 | -0.500 |
| \\(X_{14}\\) | -0.500 | 1.000 | 0.167 | 0.000 | 0.000 | 0.000 | 0.000 | 0.833 | 0.500 |
| \\(X_{5}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 0.000 | 0.333 | 0.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | -0.056 | 0.333 | 0.000 | 0.000 | 0.000 | 0.222 | 0.000 |
| \\(X_{10}\\) | 0.000 | 0.000 | -0.056 | 0.333 | 1.000 | 0.000 | 0.000 | 0.222 | 0.000 |
| \\(X_{11}\\) | 0.500 | 0.000 | -0.111 | -0.333 | 0.000 | 1.000 | 0.000 | 0.944 | -0.500 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 1.000 | 0.333 | 0.000 |


\\(Ĉ_J = \ \\)array([ 0.      , -1.      ,  0.      , -1.5     ,  0.      , -0.833333,
        0.5     , -1.      , -0.166667,  0.      ])


\\(J = {[0, 16, 2, 13, 4, 15, 6, 7, 8, 9]}\\)

### Pivot Operations

Variable to enter: \\(X_{6}\\)
Variable to leave: \\(X_{14}\\)
#### Row Operations:

1. \\(R_{X_{14}} \leftarrow \frac{R_{X_{14}}}{0.500}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 0.500R_{X_{14}}\\)
2. \\(R_{X_{3}} \leftarrow R_{X_{3}} + 0.500R_{X_{14}}\\)
3. \\(R_{X_{5}} \leftarrow R_{X_{5}} - 0.000R_{X_{14}}\\)
4. \\(R_{X_{1}} \leftarrow R_{X_{1}} - 0.000R_{X_{14}}\\)
5. \\(R_{X_{10}} \leftarrow R_{X_{10}} - 0.000R_{X_{14}}\\)
6. \\(R_{X_{11}} \leftarrow R_{X_{11}} + 0.500R_{X_{14}}\\)
7. \\(R_{X_{12}} \leftarrow R_{X_{12}} - 0.000R_{X_{14}}\\)


Computed Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 |
| \\(X_{3}\\) | 0.000 | 1.000 | 0.056 | -0.333 | 0.000 | 0.000 | 0.000 | 1.778 |
| \\(X_{6}\\) | -1.000 | 2.000 | 0.333 | 0.000 | 0.000 | 0.000 | 0.000 | 1.667 |
| \\(X_{5}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 0.000 | 0.333 |
| \\(X_{1}\\) | 0.000 | 0.000 | -0.056 | 0.333 | 0.000 | 0.000 | 0.000 | 0.222 |
| \\(X_{10}\\) | 0.000 | 0.000 | -0.056 | 0.333 | 1.000 | 0.000 | 0.000 | 0.222 |
| \\(X_{11}\\) | 0.000 | 1.000 | 0.056 | -0.333 | 0.000 | 1.000 | 0.000 | 1.778 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 1.000 | 0.333 |


## Iteration 4

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | -0.000 | 0.000 | 0.000 | 0.000 | 0.000 | -0.000 |
| \\(X_{3}\\) | 0.000 | 1.000 | 0.056 | -0.333 | 0.000 | 0.000 | 0.000 | 1.778 |
| \\(X_{6}\\) | -1.000 | 2.000 | 0.333 | 0.000 | 0.000 | 0.000 | 0.000 | 1.667 |
| \\(X_{5}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 0.000 | 0.333 |
| \\(X_{1}\\) | 0.000 | 0.000 | -0.056 | 0.333 | 0.000 | 0.000 | 0.000 | 0.222 |
| \\(X_{10}\\) | 0.000 | 0.000 | -0.056 | 0.333 | 1.000 | 0.000 | 0.000 | 0.222 |
| \\(X_{11}\\) | 0.000 | 1.000 | 0.056 | -0.333 | 0.000 | 1.000 | 0.000 | 1.778 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 1.000 | 0.333 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
-2.000 & 2.000 & -2.000 & 2.000 & -2.000 & 2.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -0.000 & -0.000 & -0.000 \\
-1.000 & 1.000 & -1.000 & 1.000 & 0.000 & 0.000 & 0.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & -0.000 & 1.000 & -0.000 & -0.000 \\
0.000 & 0.000 & 0.000 & 0.000 & -6.000 & 6.000 & 0.000 & 0.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & -0.000 & -0.000 & 1.000 & -0.000 \\
-3.000 & 3.000 & 0.000 & 0.000 & -1.000 & 1.000 & 0.000 & 0.000 & 0.000 & -1.000 & 0.000 & 0.000 & 0.000 & -0.000 & -0.000 & -0.000 & 1.000 \\
1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & -0.000 & -0.000 & -0.000 & -0.000 \\
0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & -0.000 & -0.000 & -0.000 & -0.000 \\
0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -0.000 & -0.000 & -0.000 & -0.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
3.000\\
2.000\\
2.000\\
1.000\\
0.000\\
0.000\\
0.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 1.000 & 1.000 & 1.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}\\
X_{7}\\
X_{8}\\
X_{9}\\
X_{10}\\
X_{11}\\
X_{12}\\
X_{13}\\
X_{14}\\
X_{15}\\
X_{16}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal multiple solutions

Optimal Solution: \\(X^{*} = ['0.000', '0.222', '0.000', '1.778', '0.000', '0.333', '1.667', '0.000', '0.000', '0.000', '0.222', '1.778', '0.333', '0.000', '0.000', '0.000', '0.000']\\)

Optimal Value: \\(Z^{*} = -0.000\\)

Optimal Basis: \\([3, 6, 5, 1, 10, 11, 12]\\)

Final non-basic variables set J: \\([0, 16, 2, 13, 4, 15, 14, 7, 8, 9]\\)

\\(\hat{C}_{J}\\): \\([0.0, -1.0, 0.0, -1.0, 0.0, -1.0, -1.0, 0.0, 0.0, 0.0]\\)

Number of Iterations: 5



In [154]:
# segunda fase
iterations = list(solution.keys()).pop()
I = solution[iterations]['I']
# resolvendo problema original com o simplex tableau revisado com BIF

solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   |   | RHS |
|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 2.000 | 0.889 | 2.667 | 0.000 | 0.000 | 0.000 | 8.444 |
| \\(X_{3}\\) | 0.000 | 1.000 | 0.056 | -0.333 | 0.000 | 0.000 | 0.000 | 1.778 |
| \\(X_{6}\\) | -1.000 | 2.000 | 0.333 | 0.000 | 0.000 | 0.000 | 0.000 | 1.667 |
| \\(X_{5}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 0.000 | 0.333 |
| \\(X_{1}\\) | 0.000 | 0.000 | -0.056 | 0.333 | 0.000 | 0.000 | 0.000 | 0.222 |
| \\(X_{10}\\) | 0.000 | 0.000 | -0.056 | 0.333 | 1.000 | 0.000 | 0.000 | 0.222 |
| \\(X_{11}\\) | 0.000 | 1.000 | 0.056 | -0.333 | 0.000 | 1.000 | 0.000 | 1.778 |
| \\(X_{12}\\) | 0.000 | 0.000 | 0.167 | 0.000 | 0.000 | 0.000 | 1.000 | 0.333 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
-2.000 & 2.000 & -2.000 & 2.000 & -2.000 & 2.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
-1.000 & 1.000 & -1.000 & 1.000 & 0.000 & 0.000 & 0.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
0.000 & 0.000 & 0.000 & 0.000 & -6.000 & 6.000 & 0.000 & 0.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
-3.000 & 3.000 & 0.000 & 0.000 & -1.000 & 1.000 & 0.000 & 0.000 & 0.000 & -1.000 & 0.000 & 0.000 & 0.000 \\
1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 \\
0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 \\
0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
3.000\\
2.000\\
2.000\\
1.000\\
0.000\\
0.000\\
0.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
-10.000 & 10.000 & -2.000 & 2.000 & -8.000 & 8.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}\\
X_{7}\\
X_{8}\\
X_{9}\\
X_{10}\\
X_{11}\\
X_{12}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal multiple solutions

Optimal Solution: \\(X^{*} = ['0.000', '0.222', '0.000', '1.778', '0.000', '0.333', '1.667', '0.000', '0.000', '0.000', '0.222', '1.778', '0.333']\\)

Optimal Value: \\(Z^{*} = 8.444\\)

Optimal Basis: \\([3, 6, 5, 1, 10, 11, 12]\\)

Final non-basic variables set J: \\([0, 2, 4, 7, 8, 9]\\)

\\(\hat{C}_{J}\\): \\([0.0, 0.0, 0.0, -2.0, -0.888889, -2.666667]\\)

Number of Iterations: 1



In [171]:
c_p = np.array([3, 2, 2, 1, 0, 0, 0])
x = np.array([1, 0, 8/9, 8/3, 0, 0, 0])
a_p = np.array([
    [2, 1, 0, 3, 1, 0, 0],
    [2, 1, 0, 0, 0, 1, 0],
    [0, 2, 6, 1, 0, 0, 1]
])

display(a_p @ x)
display(c_p @ x)

array([10.,  2.,  8.])

7.444444444444445

In [176]:
x_d = np.array([-2/9, -16/9, -3/9, 15/9, 0, 0, 0, 2/9, 16/9, 3/9])
c_d = -np.array([10, 2, 8] + [0] * 7)
a_d = np.array([
    [-2, -2, -2,-1,  0,  0,  0, 0, 0, 0],
    [-1, -1,  0, 0, -1,  0,  0, 0, 0, 0],
    [ 0,  0, -6, 0,  0, -1,  0, 0, 0, 0],
    [-3,  0, -1, 0,  0,  0, -1, 0, 0, 0],
    [ 1,  0,  0, 0,  0,  0,  0, 1, 0, 0],
    [ 0,  1,  0, 0,  0,  0,  0, 0, 1, 0],
    [ 0,  0,  1, 0,  0,  0,  0, 0, 0, 1]
])
a_d @ x_d
c_d @ x_d

8.444444444444445

## Número 3)

In [179]:
C = np.array(
    [1, -2, 2] + [0] * 5
)

slacks_and_excesses = np.eye(5)
excesses = [0, 1]
for i in excesses:
    slacks_and_excesses[i , i] = -1

A = np.array([
    [ 5, -3,  3],
    [-1, -1,  1],
    [ 3, -4,  4],
    [ 4, -1,  1],
    [ 0,  1, -1]
])
A = np.hstack((A, slacks_and_excesses))
B = np.array([
    15,
    1,
    24,
    8,
    0
])

# primeira fase, variáveis artificiais

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = [8, 9, 5, 6, 7]

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I, cycle_proof=True)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 16.000 |
| \\(X_{8}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 15.000 |
| \\(X_{9}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 24.000 |
| \\(X_{6}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 8.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS | \\(X_{0}\\)
|---|---|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | 0.000 | 0.000 | 0.000 | 16.000 | 4.000 |
| \\(X_{8}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 15.000 | 5.000 |
| \\(X_{9}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 1.000 | -1.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 24.000 | 3.000 |
| \\(X_{6}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 8.000 | 4.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 |


\\(Ĉ_J = \ \\)array([ 4., -4.,  4., -1., -1.])


\\(J = {[0, 1, 2, 3, 4]}\\)

### Pivot Operations

Variable to enter: \\(X_{0}\\)
Variable to leave: \\(X_{6}\\)
#### Row Operations:

1. \\(R_{X_{6}} \leftarrow \frac{R_{X_{6}}}{4.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 4.000R_{X_{6}}\\)
2. \\(R_{X_{8}} \leftarrow R_{X_{8}} - 5.000R_{X_{6}}\\)
3. \\(R_{X_{9}} \leftarrow R_{X_{9}} + 1.000R_{X_{6}}\\)
4. \\(R_{X_{5}} \leftarrow R_{X_{5}} - 3.000R_{X_{6}}\\)
5. \\(R_{X_{7}} \leftarrow R_{X_{7}} - 0.000R_{X_{6}}\\)


Computed Tableau:

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | 0.000 | -1.000 | 0.000 | 8.000 |
| \\(X_{8}\\) | 1.000 | 0.000 | 0.000 | -1.250 | 0.000 | 5.000 |
| \\(X_{9}\\) | 0.000 | 1.000 | 0.000 | 0.250 | 0.000 | 3.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | -0.750 | 0.000 | 18.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 0.250 | 0.000 | 2.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


## Iteration 1

Starting Tableau:

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | 0.000 | -1.000 | 0.000 | 8.000 |
| \\(X_{8}\\) | 1.000 | 0.000 | 0.000 | -1.250 | 0.000 | 5.000 |
| \\(X_{9}\\) | 0.000 | 1.000 | 0.000 | 0.250 | 0.000 | 3.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | -0.750 | 0.000 | 18.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 0.250 | 0.000 | 2.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 |


Extended Tableau

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS | \\(X_{2}\\)
|---|---|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | 0.000 | -1.000 | 0.000 | 8.000 | 3.000 |
| \\(X_{8}\\) | 1.000 | 0.000 | 0.000 | -1.250 | 0.000 | 5.000 | 1.750 |
| \\(X_{9}\\) | 0.000 | 1.000 | 0.000 | 0.250 | 0.000 | 3.000 | 1.250 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | -0.750 | 0.000 | 18.000 | 3.250 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 0.250 | 0.000 | 2.000 | 0.250 |
| \\(X_{7}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | -1.000 |


\\(Ĉ_J = \ \\)array([-1., -3.,  3., -1., -1.])


\\(J = {[6, 1, 2, 3, 4]}\\)

### Pivot Operations

Variable to enter: \\(X_{2}\\)
Variable to leave: \\(X_{9}\\)
#### Row Operations:

1. \\(R_{X_{9}} \leftarrow \frac{R_{X_{9}}}{1.250}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 3.000R_{X_{9}}\\)
2. \\(R_{X_{8}} \leftarrow R_{X_{8}} - 1.750R_{X_{9}}\\)
3. \\(R_{X_{5}} \leftarrow R_{X_{5}} - 3.250R_{X_{9}}\\)
4. \\(R_{X_{0}} \leftarrow R_{X_{0}} - 0.250R_{X_{9}}\\)
5. \\(R_{X_{7}} \leftarrow R_{X_{7}} + 1.000R_{X_{9}}\\)


Computed Tableau:

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|
| Z | 1.000 | -1.400 | 0.000 | -1.600 | 0.000 | 0.800 |
| \\(X_{8}\\) | 1.000 | -1.400 | 0.000 | -1.600 | 0.000 | 0.800 |
| \\(X_{2}\\) | 0.000 | 0.800 | 0.000 | 0.200 | 0.000 | 2.400 |
| \\(X_{5}\\) | 0.000 | -2.600 | 1.000 | -1.400 | 0.000 | 10.200 |
| \\(X_{0}\\) | 0.000 | -0.200 | 0.000 | 0.200 | 0.000 | 1.400 |
| \\(X_{7}\\) | 0.000 | 0.800 | 0.000 | 0.200 | 1.000 | 2.400 |


## Iteration 2

Starting Tableau:

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|
| Z | 1.000 | -1.400 | 0.000 | -1.600 | 0.000 | 0.800 |
| \\(X_{8}\\) | 1.000 | -1.400 | 0.000 | -1.600 | 0.000 | 0.800 |
| \\(X_{2}\\) | 0.000 | 0.800 | 0.000 | 0.200 | 0.000 | 2.400 |
| \\(X_{5}\\) | 0.000 | -2.600 | 1.000 | -1.400 | 0.000 | 10.200 |
| \\(X_{0}\\) | 0.000 | -0.200 | 0.000 | 0.200 | 0.000 | 1.400 |
| \\(X_{7}\\) | 0.000 | 0.800 | 0.000 | 0.200 | 1.000 | 2.400 |


Extended Tableau

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS | \\(X_{4}\\)
|---|---|---|---|---|---|---|---|
| Z | 1.000 | -1.400 | 0.000 | -1.600 | 0.000 | 0.800 | 1.400 |
| \\(X_{8}\\) | 1.000 | -1.400 | 0.000 | -1.600 | 0.000 | 0.800 | 1.400 |
| \\(X_{2}\\) | 0.000 | 0.800 | 0.000 | 0.200 | 0.000 | 2.400 | -0.800 |
| \\(X_{5}\\) | 0.000 | -2.600 | 1.000 | -1.400 | 0.000 | 10.200 | 2.600 |
| \\(X_{0}\\) | 0.000 | -0.200 | 0.000 | 0.200 | 0.000 | 1.400 | 0.200 |
| \\(X_{7}\\) | 0.000 | 0.800 | 0.000 | 0.200 | 1.000 | 2.400 | -0.800 |


\\(Ĉ_J = \ \\)array([-1.6,  0. , -2.4, -1. ,  1.4])


\\(J = {[6, 1, 9, 3, 4]}\\)

### Pivot Operations

Variable to enter: \\(X_{4}\\)
Variable to leave: \\(X_{8}\\)
#### Row Operations:

1. \\(R_{X_{8}} \leftarrow \frac{R_{X_{8}}}{1.400}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 1.400R_{X_{8}}\\)
2. \\(R_{X_{2}} \leftarrow R_{X_{2}} + 0.800R_{X_{8}}\\)
3. \\(R_{X_{5}} \leftarrow R_{X_{5}} - 2.600R_{X_{8}}\\)
4. \\(R_{X_{0}} \leftarrow R_{X_{0}} - 0.200R_{X_{8}}\\)
5. \\(R_{X_{7}} \leftarrow R_{X_{7}} + 0.800R_{X_{8}}\\)


Computed Tableau:

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|
| Z | 0.000 | -0.000 | 0.000 | -0.000 | 0.000 | 0.000 |
| \\(X_{4}\\) | 0.714 | -1.000 | 0.000 | -1.143 | 0.000 | 0.571 |
| \\(X_{2}\\) | 0.571 | 0.000 | 0.000 | -0.714 | 0.000 | 2.857 |
| \\(X_{5}\\) | -1.857 | 0.000 | 1.000 | 1.571 | 0.000 | 8.714 |
| \\(X_{0}\\) | -0.143 | 0.000 | 0.000 | 0.429 | 0.000 | 1.286 |
| \\(X_{7}\\) | 0.571 | 0.000 | 0.000 | -0.714 | 1.000 | 2.857 |


## Iteration 3

Starting Tableau:

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|
| Z | 0.000 | -0.000 | 0.000 | -0.000 | 0.000 | 0.000 |
| \\(X_{4}\\) | 0.714 | -1.000 | 0.000 | -1.143 | 0.000 | 0.571 |
| \\(X_{2}\\) | 0.571 | 0.000 | 0.000 | -0.714 | 0.000 | 2.857 |
| \\(X_{5}\\) | -1.857 | 0.000 | 1.000 | 1.571 | 0.000 | 8.714 |
| \\(X_{0}\\) | -0.143 | 0.000 | 0.000 | 0.429 | 0.000 | 1.286 |
| \\(X_{7}\\) | 0.571 | 0.000 | 0.000 | -0.714 | 1.000 | 2.857 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
5.000 & -3.000 & 3.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -0.000 \\
-1.000 & -1.000 & 1.000 & 0.000 & -1.000 & 0.000 & 0.000 & 0.000 & -0.000 & 1.000 \\
3.000 & -4.000 & 4.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & -0.000 & -0.000 \\
4.000 & -1.000 & 1.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 & -0.000 & -0.000 \\
0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & -0.000 & -0.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
15.000\\
1.000\\
24.000\\
8.000\\
0.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 1.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}\\
X_{7}\\
X_{8}\\
X_{9}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal multiple solutions

Optimal Solution: \\(X^{*} = ['1.286', '0.000', '2.857', '0.000', '0.571', '8.714', '0.000', '2.857', '0.000', '0.000']\\)

Optimal Value: \\(Z^{*} = 0.000\\)

Optimal Basis: \\([4, 2, 5, 0, 7]\\)

Final non-basic variables set J: \\([6, 1, 9, 3, 8]\\)

\\(\hat{C}_{J}\\): \\([-0.0, 0.0, -1.0, -0.0, -1.0]\\)

Number of Iterations: 4



In [180]:
# segunda fase
iterations = list(solution.keys()).pop()
I = solution[iterations]['I']
# resolvendo problema original com o simplex tableau revisado com BIF

solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | 0.000 | -1.000 | 0.000 | 7.000 |
| \\(X_{4}\\) | 0.714 | -1.000 | 0.000 | -1.143 | 0.000 | 0.571 |
| \\(X_{2}\\) | 0.571 | 0.000 | 0.000 | -0.714 | 0.000 | 2.857 |
| \\(X_{5}\\) | -1.857 | 0.000 | 1.000 | 1.571 | 0.000 | 8.714 |
| \\(X_{0}\\) | -0.143 | 0.000 | 0.000 | 0.429 | 0.000 | 1.286 |
| \\(X_{7}\\) | 0.571 | 0.000 | 0.000 | -0.714 | 1.000 | 2.857 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
5.000 & -3.000 & 3.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
-1.000 & -1.000 & 1.000 & 0.000 & -1.000 & 0.000 & 0.000 & 0.000 \\
3.000 & -4.000 & 4.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 \\
4.000 & -1.000 & 1.000 & 0.000 & 0.000 & 0.000 & 1.000 & 0.000 \\
0.000 & 1.000 & -1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
15.000\\
1.000\\
24.000\\
8.000\\
0.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
1.000 & -2.000 & 2.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}\\
X_{7}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal multiple solutions

Optimal Solution: \\(X^{*} = ['1.286', '0.000', '2.857', '0.000', '0.571', '8.714', '0.000', '2.857']\\)

Optimal Value: \\(Z^{*} = 7.000\\)

Optimal Basis: \\([4, 2, 5, 0, 7]\\)

Final non-basic variables set J: \\([1, 3, 6]\\)

\\(\hat{C}_{J}\\): \\([0.0, -1.0, -1.0]\\)

Number of Iterations: 1



## Exercício 6.5)

In [3]:
C = -np.array([10, 24, 20, 20, 25])
A = np.array([
    [1, 1, 2, 3, 5],
    [2, 4, 3, 2, 1]
])
B = np.array([
    19,
    57
])
I = [1, 4]

solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | -4.000 | -5.000 | -361.000 |
| \\(X_{1}\\) | -0.053 | 0.263 | 14.000 |
| \\(X_{4}\\) | 0.211 | -0.053 | 1.000 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
1.000 & 1.000 & 2.000 & 3.000 & 5.000 \\
2.000 & 4.000 & 3.000 & 2.000 & 1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
19.000\\
57.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
-10.000 & -24.000 & -20.000 & -20.000 & -25.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal unique solution

Optimal Solution: \\(X^{*} = ['0.000', '14.000', '0.000', '0.000', '1.000']\\)

Optimal Value: \\(Z^{*} = -361.000\\)

Optimal Basis: \\([1, 4]\\)

Final non-basic variables set J: \\([0, 2, 3]\\)

\\(\hat{C}_{J}\\): \\([-4.0, -3.0, -2.0]\\)

Number of Iterations: 1



## Questão 6.7)

In [4]:
C = np.array([8, 6, 0, 0, 0])
A = np.array([
    [1,  1, -1,  0,  0],
    [2, -2,  0, -1,  0],
    [3,  2,  0,  0, -1]
])
B = np.array([
    2,
    3,
    5
])

# primeira fase, variáveis artificiais

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = [5, 6, 7]

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I, cycle_proof=True)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 1.000 | 1.000 | 1.000 | 10.000 |
| \\(X_{5}\\) | 1.000 | 0.000 | 0.000 | 2.000 |
| \\(X_{6}\\) | 0.000 | 1.000 | 0.000 | 3.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 1.000 | 5.000 |


Extended Tableau

|   |  | \\(A_I^{-1}\\) |   | RHS | \\(X_{0}\\)
|---|---|---|---|---|---|
| Z | 1.000 | 1.000 | 1.000 | 10.000 | 6.000 |
| \\(X_{5}\\) | 1.000 | 0.000 | 0.000 | 2.000 | 1.000 |
| \\(X_{6}\\) | 0.000 | 1.000 | 0.000 | 3.000 | 2.000 |
| \\(X_{7}\\) | 0.000 | 0.000 | 1.000 | 5.000 | 3.000 |


\\(Ĉ_J = \ \\)array([ 6.,  1., -1., -1., -1.])


\\(J = {[0, 1, 2, 3, 4]}\\)

### Pivot Operations

Variable to enter: \\(X_{0}\\)
Variable to leave: \\(X_{6}\\)
#### Row Operations:

1. \\(R_{X_{6}} \leftarrow \frac{R_{X_{6}}}{2.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 6.000R_{X_{6}}\\)
2. \\(R_{X_{5}} \leftarrow R_{X_{5}} - 1.000R_{X_{6}}\\)
3. \\(R_{X_{7}} \leftarrow R_{X_{7}} - 3.000R_{X_{6}}\\)


Computed Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 1.000 | -2.000 | 1.000 | 1.000 |
| \\(X_{5}\\) | 1.000 | -0.500 | 0.000 | 0.500 |
| \\(X_{0}\\) | 0.000 | 0.500 | 0.000 | 1.500 |
| \\(X_{7}\\) | 0.000 | -1.500 | 1.000 | 0.500 |


## Iteration 1

Starting Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 1.000 | -2.000 | 1.000 | 1.000 |
| \\(X_{5}\\) | 1.000 | -0.500 | 0.000 | 0.500 |
| \\(X_{0}\\) | 0.000 | 0.500 | 0.000 | 1.500 |
| \\(X_{7}\\) | 0.000 | -1.500 | 1.000 | 0.500 |


Extended Tableau

|   |  | \\(A_I^{-1}\\) |   | RHS | \\(X_{1}\\)
|---|---|---|---|---|---|
| Z | 1.000 | -2.000 | 1.000 | 1.000 | 7.000 |
| \\(X_{5}\\) | 1.000 | -0.500 | 0.000 | 0.500 | 2.000 |
| \\(X_{0}\\) | 0.000 | 0.500 | 0.000 | 1.500 | -1.000 |
| \\(X_{7}\\) | 0.000 | -1.500 | 1.000 | 0.500 | 5.000 |


\\(Ĉ_J = \ \\)array([-3.,  7., -1.,  2., -1.])


\\(J = {[6, 1, 2, 3, 4]}\\)

### Pivot Operations

Variable to enter: \\(X_{1}\\)
Variable to leave: \\(X_{7}\\)
#### Row Operations:

1. \\(R_{X_{7}} \leftarrow \frac{R_{X_{7}}}{5.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 7.000R_{X_{7}}\\)
2. \\(R_{X_{5}} \leftarrow R_{X_{5}} - 2.000R_{X_{7}}\\)
3. \\(R_{X_{0}} \leftarrow R_{X_{0}} + 1.000R_{X_{7}}\\)


Computed Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 1.000 | 0.100 | -0.400 | 0.300 |
| \\(X_{5}\\) | 1.000 | 0.100 | -0.400 | 0.300 |
| \\(X_{0}\\) | 0.000 | 0.200 | 0.200 | 1.600 |
| \\(X_{1}\\) | 0.000 | -0.300 | 0.200 | 0.100 |


## Iteration 2

Starting Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 1.000 | 0.100 | -0.400 | 0.300 |
| \\(X_{5}\\) | 1.000 | 0.100 | -0.400 | 0.300 |
| \\(X_{0}\\) | 0.000 | 0.200 | 0.200 | 1.600 |
| \\(X_{1}\\) | 0.000 | -0.300 | 0.200 | 0.100 |


Extended Tableau

|   |  | \\(A_I^{-1}\\) |   | RHS | \\(X_{4}\\)
|---|---|---|---|---|---|
| Z | 1.000 | 0.100 | -0.400 | 0.300 | 0.400 |
| \\(X_{5}\\) | 1.000 | 0.100 | -0.400 | 0.300 | 0.400 |
| \\(X_{0}\\) | 0.000 | 0.200 | 0.200 | 1.600 | -0.200 |
| \\(X_{1}\\) | 0.000 | -0.300 | 0.200 | 0.100 | -0.200 |


\\(Ĉ_J = \ \\)array([-0.9, -1.4, -1. , -0.1,  0.4])


\\(J = {[6, 7, 2, 3, 4]}\\)

### Pivot Operations

Variable to enter: \\(X_{4}\\)
Variable to leave: \\(X_{5}\\)
#### Row Operations:

1. \\(R_{X_{5}} \leftarrow \frac{R_{X_{5}}}{0.400}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 0.400R_{X_{5}}\\)
2. \\(R_{X_{0}} \leftarrow R_{X_{0}} + 0.200R_{X_{5}}\\)
3. \\(R_{X_{1}} \leftarrow R_{X_{1}} + 0.200R_{X_{5}}\\)


Computed Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 0.000 | 0.000 | -0.000 | -0.000 |
| \\(X_{4}\\) | 2.500 | 0.250 | -1.000 | 0.750 |
| \\(X_{0}\\) | 0.500 | 0.250 | 0.000 | 1.750 |
| \\(X_{1}\\) | 0.500 | -0.250 | 0.000 | 0.250 |


## Iteration 3

Starting Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 0.000 | 0.000 | -0.000 | -0.000 |
| \\(X_{4}\\) | 2.500 | 0.250 | -1.000 | 0.750 |
| \\(X_{0}\\) | 0.500 | 0.250 | 0.000 | 1.750 |
| \\(X_{1}\\) | 0.500 | -0.250 | 0.000 | 0.250 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
1.000 & 1.000 & -1.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 \\
2.000 & -2.000 & 0.000 & -1.000 & 0.000 & 0.000 & 1.000 & 0.000 \\
3.000 & 2.000 & 0.000 & 0.000 & -1.000 & 0.000 & 0.000 & 1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
2.000\\
3.000\\
5.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 1.000 & 1.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}\\
X_{7}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal multiple solutions

Optimal Solution: \\(X^{*} = ['1.750', '0.250', '0.000', '0.000', '0.750', '0.000', '0.000', '0.000']\\)

Optimal Value: \\(Z^{*} = -0.000\\)

Optimal Basis: \\([4, 0, 1]\\)

Final non-basic variables set J: \\([6, 7, 2, 3, 5]\\)

\\(\hat{C}_{J}\\): \\([-1.0, -1.0, 0.0, -0.0, -1.0]\\)

Number of Iterations: 4



In [5]:
# segunda fase
iterations = list(solution.keys()).pop()
I = solution[iterations]['I']
# resolvendo problema original com o simplex tableau revisado com BIF

solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 7.000 | 0.500 | 0.000 | 15.500 |
| \\(X_{4}\\) | 2.500 | 0.250 | -1.000 | 0.750 |
| \\(X_{0}\\) | 0.500 | 0.250 | 0.000 | 1.750 |
| \\(X_{1}\\) | 0.500 | -0.250 | 0.000 | 0.250 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
1.000 & 1.000 & -1.000 & 0.000 & 0.000 \\
2.000 & -2.000 & 0.000 & -1.000 & 0.000 \\
3.000 & 2.000 & 0.000 & 0.000 & -1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
2.000\\
3.000\\
5.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
8.000 & 6.000 & 0.000 & 0.000 & 0.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal unique solution

Optimal Solution: \\(X^{*} = ['1.750', '0.250', '0.000', '0.000', '0.750']\\)

Optimal Value: \\(Z^{*} = 15.500\\)

Optimal Basis: \\([4, 0, 1]\\)

Final non-basic variables set J: \\([2, 3]\\)

\\(\hat{C}_{J}\\): \\([-7.0, -0.5]\\)

Number of Iterations: 1



In [8]:
# problema primal
C = -np.array([2, 3, 5, 0, 0])
A = np.array([
    [1,  2, 3, 1, 0],
    [1, -2, 2, 0, 1]
])
B = np.array([
    8,
    6
])
I = [3, 4]
solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 |
| \\(X_{3}\\) | 1.000 | 0.000 | 8.000 |
| \\(X_{4}\\) | 0.000 | 1.000 | 6.000 |


Extended Tableau

|   |  | \\(A_I^{-1}\\) | RHS | \\(X_{2}\\)
|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 5.000 |
| \\(X_{3}\\) | 1.000 | 0.000 | 8.000 | 3.000 |
| \\(X_{4}\\) | 0.000 | 1.000 | 6.000 | 2.000 |


\\(Ĉ_J = \ \\)array([2., 3., 5.])


\\(J = {[0, 1, 2]}\\)

### Pivot Operations

Variable to enter: \\(X_{2}\\)
Variable to leave: \\(X_{3}\\)
#### Row Operations:

1. \\(R_{X_{3}} \leftarrow \frac{R_{X_{3}}}{3.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 5.000R_{X_{3}}\\)
2. \\(R_{X_{4}} \leftarrow R_{X_{4}} - 2.000R_{X_{3}}\\)


Computed Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | -1.667 | 0.000 | -13.333 |
| \\(X_{2}\\) | 0.333 | 0.000 | 2.667 |
| \\(X_{4}\\) | -0.667 | 1.000 | 0.667 |


## Iteration 1

Starting Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | -1.667 | 0.000 | -13.333 |
| \\(X_{2}\\) | 0.333 | 0.000 | 2.667 |
| \\(X_{4}\\) | -0.667 | 1.000 | 0.667 |


Extended Tableau

|   |  | \\(A_I^{-1}\\) | RHS | \\(X_{0}\\)
|---|---|---|---|---|
| Z | -1.667 | 0.000 | -13.333 | 0.333 |
| \\(X_{2}\\) | 0.333 | 0.000 | 2.667 | 0.333 |
| \\(X_{4}\\) | -0.667 | 1.000 | 0.667 | 0.333 |


\\(Ĉ_J = \ \\)array([ 0.333333, -0.333333, -1.666667])


\\(J = {[0, 1, 3]}\\)

### Pivot Operations

Variable to enter: \\(X_{0}\\)
Variable to leave: \\(X_{4}\\)
#### Row Operations:

1. \\(R_{X_{4}} \leftarrow \frac{R_{X_{4}}}{0.333}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 0.333R_{X_{4}}\\)
2. \\(R_{X_{2}} \leftarrow R_{X_{2}} - 0.333R_{X_{4}}\\)


Computed Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | -1.000 | -1.000 | -14.000 |
| \\(X_{2}\\) | 1.000 | -1.000 | 2.000 |
| \\(X_{0}\\) | -2.000 | 3.000 | 2.000 |


## Iteration 2

Starting Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | -1.000 | -1.000 | -14.000 |
| \\(X_{2}\\) | 1.000 | -1.000 | 2.000 |
| \\(X_{0}\\) | -2.000 | 3.000 | 2.000 |


Extended Tableau

|   |  | \\(A_I^{-1}\\) | RHS | \\(X_{1}\\)
|---|---|---|---|---|
| Z | -1.000 | -1.000 | -14.000 | 3.000 |
| \\(X_{2}\\) | 1.000 | -1.000 | 2.000 | 4.000 |
| \\(X_{0}\\) | -2.000 | 3.000 | 2.000 | -10.000 |


\\(Ĉ_J = \ \\)array([-0.999999,  2.999997, -1.000001])


\\(J = {[4, 1, 3]}\\)

### Pivot Operations

Variable to enter: \\(X_{1}\\)
Variable to leave: \\(X_{2}\\)
#### Row Operations:

1. \\(R_{X_{2}} \leftarrow \frac{R_{X_{2}}}{4.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 3.000R_{X_{2}}\\)
2. \\(R_{X_{0}} \leftarrow R_{X_{0}} + 10.000R_{X_{2}}\\)


Computed Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | -1.750 | -0.250 | -15.500 |
| \\(X_{1}\\) | 0.250 | -0.250 | 0.500 |
| \\(X_{0}\\) | 0.500 | 0.500 | 7.000 |


## Iteration 3

Starting Tableau:

|   |  | \\(A_I^{-1}\\) | RHS |
|---|---|---|---|
| Z | -1.750 | -0.250 | -15.500 |
| \\(X_{1}\\) | 0.250 | -0.250 | 0.500 |
| \\(X_{0}\\) | 0.500 | 0.500 | 7.000 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
1.000 & 2.000 & 3.000 & 1.000 & 0.000 \\
1.000 & -2.000 & 2.000 & 0.000 & 1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
8.000\\
6.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
-2.000 & -3.000 & -5.000 & 0.000 & 0.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal unique solution

Optimal Solution: \\(X^{*} = ['7.000', '0.500', '0.000', '0.000', '0.000']\\)

Optimal Value: \\(Z^{*} = -15.500\\)

Optimal Basis: \\([1, 0]\\)

Final non-basic variables set J: \\([4, 2, 3]\\)

\\(\hat{C}_{J}\\): \\([-0.25, -0.749999, -1.75]\\)

Number of Iterations: 4



In [13]:
A = np.array([
    [-1/2, -1, 1/4,  0,  0],
    [-1/6,  0, 1/4, -1,  0],
    [ 1/3,  0,   0,  0, -1],
    [-5/6, -1, 1/4,  0,  1],
    [   2,  0, 5/2,  0,  0]
])
B = np.array([
    -2,
    -4,
    -2,
     0,
    -35
])
#np.linalg.det(A)
np.linalg.solve(A, B)

array([-12. ,   6.9,  -4.4,   4.9,  -2. ])

## Questão 6.18)

In [18]:
from simplex_tableau import simplex as st
from simplex_tableau import markdown_repr_T as st_markdown_repr_T

C = np.array([120, 85, 0, 0])
A = np.array([
    [2,   1, -1, 0],
    [1, 1/2, 0, -1]
])
B = np.array([
    25,
    15
])

# primeira fase, variáveis artificiais

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = [4, 5]

# resolvendo problema artificial com o simplex tableau revisado
solution = st(A_artificial, B, C_artificial, I)
results_md = st_markdown_repr_T(solution['tableaus'])

display(
    Markdown(results_md)
)



## Iteration 1

Previous Tableau:

|   |\\(X_{0}\\) | \\(X_{1}\\) | \\(X_{2}\\) | \\(X_{3}\\) | \\(X_{4}\\) | \\(X_{5}\\) | RHS |
|---|---|---|---|---|---|---|---|
| Cost | 3.000 | 1.500 | -1.000 | -1.000 | 0.000 | 0.000 | 40.000 |
| \\(X_{4}\\) | 2.000 | 1.000 | -1.000 | 0.000 | 1.000 | 0.000 | 25.000 |
| \\(X_{5}\\) | 1.000 | 0.500 | 0.000 | -1.000 | 0.000 | 1.000 | 15.000 |
### Pivot Operations

Variable to enter: \\(X_{0}\\)
Variable to leave: \\(X_{4}\\)
#### Row Operations:

1. \\(R_{X_{4}} \leftarrow \frac{R_{X_{4}}}{2.0}\\)
1. \\(R_{C} \leftarrow R_{C} - 3.0R_{X_{4}}\\)
2. \\(R_{X_{5}} \leftarrow R_{X_{5}} - 1.0R_{X_{4}}\\)


Computed Tableau:

|   |\\(X_{0}\\) | \\(X_{1}\\) | \\(X_{2}\\) | \\(X_{3}\\) | \\(X_{4}\\) | \\(X_{5}\\) | RHS |
|---|---|---|---|---|---|---|---|
| Cost | 0.000 | 0.000 | 0.500 | -1.000 | -1.500 | 0.000 | 2.500 |
| \\(X_{0}\\) | 1.000 | 0.500 | -0.500 | 0.000 | 0.500 | 0.000 | 12.500 |
| \\(X_{5}\\) | 0.000 | 0.000 | 0.500 | -1.000 | -0.500 | 1.000 | 2.500 |


## Iteration 2

Previous Tableau:

|   |\\(X_{0}\\) | \\(X_{1}\\) | \\(X_{2}\\) | \\(X_{3}\\) | \\(X_{4}\\) | \\(X_{5}\\) | RHS |
|---|---|---|---|---|---|---|---|
| Cost | 0.000 | 0.000 | 0.500 | -1.000 | -1.500 | 0.000 | 2.500 |
| \\(X_{0}\\) | 1.000 | 0.500 | -0.500 | 0.000 | 0.500 | 0.000 | 12.500 |
| \\(X_{5}\\) | 0.000 | 0.000 | 0.500 | -1.000 | -0.500 | 1.000 | 2.500 |
### Pivot Operations

Variable to enter: \\(X_{2}\\)
Variable to leave: \\(X_{5}\\)
#### Row Operations:

1. \\(R_{X_{5}} \leftarrow \frac{R_{X_{5}}}{0.5}\\)
1. \\(R_{C} \leftarrow R_{C} - 0.5R_{X_{5}}\\)
2. \\(R_{X_{0}} \leftarrow R_{X_{0}} + 0.5R_{X_{5}}\\)


Computed Tableau:

|   |\\(X_{0}\\) | \\(X_{1}\\) | \\(X_{2}\\) | \\(X_{3}\\) | \\(X_{4}\\) | \\(X_{5}\\) | RHS |
|---|---|---|---|---|---|---|---|
| Cost | 0.000 | 0.000 | 0.000 | 0.000 | -1.000 | -1.000 | 0.000 |
| \\(X_{0}\\) | 1.000 | 0.500 | 0.000 | -1.000 | 0.000 | 1.000 | 15.000 |
| \\(X_{2}\\) | 0.000 | 0.000 | 1.000 | -2.000 | -1.000 | 2.000 | 5.000 |
